# Computational Creativity Project: Music Style Interpolation & Casual Creators

Welcome to my Computational Creativity project! Here are some notes on how to use this notebook:
*   ***Please save a copy if you wish to make edits!***
*   Be sure to connect to a GPU!
*   Give permission to access your G-Drive.
*   You can get started by going to Runtime >> Restart and Run All (or run all if not already connected to a runtime). This is beacuse the code in Section 1 generates the musical extracts which were saved to a database, to be used in Section 2's Casual Creator GUI. As it relies on paths to databases, it is fully commented out. That said, the inline comments link to the data sets, if you want to run this yourself.
* Section 2.2 contains the GUI. I reccomend simply hiding the code, playing with the sliders, and running the cell for the closest "Casual Creator" experience.
* Note that Sections 1.1 and 2.1 might require you to say yes (type y) to uninstalling mido. This is because miditok and magenta use different versions of mido --- I ensure that the version provided by miditok is installed first so that there are no compatiability issues. 
*   Details can be found in the accompanying report.  

## 1 Generating Extracts

NOTE: This is mostly an extenstion of the RNN text generation practical, combined with other resources to adapt it for music making --- notably miditok [which can be found here](https://github.com/Natooz/MidiTok).

### 1.1 Prerequisites

In [ ]:
# # Install dependencies 
# !pip uninstall mido # There is some interesting issues using mido for 
#                     # miditok versus with the magenta things... so 
#                     # we manipulate this as needed. 
# !pip install miditok
# print('🎉 Done!')

In [ ]:
# # Essensials
# import glob 
# import numpy as np
# import pandas as pd

# # Deep learning
# import tensorflow as tf
# import keras 
# from tensorflow.keras.utils import to_categorical
# from keras.layers import Input, Dense, Bidirectional, LSTM, GRU, LayerNormalization, Embedding
# from keras.activations import softplus, softmax
# from keras.models import Model 
# from tensorflow.keras.optimizers import Adam
# from keras import backend as K
# from keras.metrics import binary_crossentropy

# # Music & Tokenizing 
# from miditok import REMI, get_midi_programs
# import miditoolkit as mtk
# import music21 as m21
# from IPython.display import Audio

In [ ]:
# # mount drive and load midi files 
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

# # check gpus 
# device_name = tf.test.gpu_device_name()
# print('Found GPU at: {}'.format(device_name))
# print(tf.__version__)

### 1.2 Data Preperation

In [ ]:
# # Below is the code for stripping the percussion parts from the reggae music. 
# # I ran this locally, so do not call this function in the notebook. 

# def stip_drums_from_reggae_music():
#   import pathlib
#   path = str(pathlib.Path().resolve())

#   toIgnore = [None, 
# 		 "Crash",
# 		 "Timbal",
# 		 "Lo Timbal",
# 		 "Crash Bekken",
# 		 "Conga",
# 		 "Triangel",
# 		 "Conga 1",
# 		 "Conga 2",
# 		 "Conga 3",
# 		 "Conga's",
# 		 "Intro Conga's",
# 		 "Agogo",
# 		 "Tom in Beat",
# 		 "Maracas",
# 		 "Woodblock",
# 		 "Conga in Beat",
# 		 "Whistle",
# 		 "Maracas",
# 		 "Shehnai",
# 		 "Conga Mute ",
# 		 "Mel. Tom"]

#   filenames = glob.glob(path + '/*.mid*')

#   for files in range(len(filenames)):	
#     print(files, " of ", len(filenames))
#     mf = m21.converter.parse(filenames[files], format="midi")

#     newMf = m21.stream.Stream()

#     mf = m21.instrument.partitionByInstrument(mf)
#     for i in mf.recurse().parts:
#       if i.partName in toIgnore:
#         continue;
#       else:
#         newMf.append(i)

		
#     newFileName = "/Users/coreyford/Documents/PhD/_Year 2/5-Computational-Creativity/datas/newbies/" + str(files) + ".mid"
#     newMf.write('midi', fp=newFileName)

### 1.3 Tokenizing and Training Model

In [ ]:
# # Load and tokenize the data 

# CREATE_REGGAE = True #@param{type: 'boolean'}
# if CREATE_REGGAE:
#   # Data found here: https://drive.google.com/drive/folders/1aSVRE6DX9DbPW1XyCjTA9uH8t2mMZktO?usp=sharing
#   filenames = glob.glob(str('My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/reggae-tunes/*.mid*'))
# else:
#   # Data found here: https://drive.google.com/drive/folders/1MtzugNa16VdNUMy3cAWXspI35IgA7iO2?usp=sharing
#   filenames = glob.glob(str('My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/bach-tunes/*.mid*'))

# #TODO: This is largley adopted from the miditok example
# #      see https://github.com/Natooz/MidiTok/blob/main/README.md
# music = []
# for files in range(len(filenames)):	
#   # Get file from dataset
#   mf = mtk.MidiFile(filenames[files])
  
#   # Initialise the file 
#   pitch_range = range(21, 109)
#   beat_res = {(0, 4): 8, (4, 12): 4}
#   nb_velocities = 32
#   additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': False,
#                        'rest_range': (2, 16),  # (half, 16 beats)
#                        'nb_tempos': 32,  # nb of tempo bins
#                        'tempo_range': (40, 250),  # (min, max)
#                        'Program': True, 
#                        'TimeSignature' : True}

#   # Creates the tokenizer and loads a MIDI
#   tokenizer = REMI(pitch_range, beat_res, nb_velocities, additional_tokens) # REMI encoding

#   # Converts MIDI to tokens
#   tokens = tokenizer.midi_to_tokens(mf)

#   # Append to our dataset 
#   music.extend(tokens[0])

# # We also store one of the files so we can use the program numbers later to de-tokenize 
# mf = mtk.MidiFile(filenames[0])

In [ ]:
# # Output the number of unique chars in our dataset, and store the vocab size
# vocab = sorted(set(music))
# VOCAB_SIZE = len(vocab)

# # Convert to char2idx and idx2char arrays
# char2idx = {u:i for i, u in enumerate(vocab)}
# idx2char = np.array(vocab)

In [ ]:
# # Initialise Parameters (from Text RNN Tutorial)
# # size of batches
# BATCH_SIZE = 64 # @param {type: "integer"}
# # buffer size to shuffle our dataset, default 10000
# BUFFER_SIZE = 10000  # @param {type: "integer"}
# # number of RNN units, default 1024
# N_RNN_UNITS = 512  # @param {type: "integer"}
# # length of text chunks for training, default 100
# MAX_LENGTH =   32# @param {type: "integer"}
# # size of the embedding layer, default 256
# EMBEDDING_DIM = 256    # @param {type: "integer"}

In [ ]:
# # Get input and target data (from Text RNN Tutorial)
# input_text = []
# target_text = []

# for c in range(0, len(music)-MAX_LENGTH, MAX_LENGTH):
#     inps = music[c : c + MAX_LENGTH]
#     tars = music[c + 1 : c + 1 + MAX_LENGTH]

#     input_text.append([char2idx[i] for i in inps])
#     target_text.append([char2idx[t] for t in tars])
    
# print (np.array(input_text).shape)
# print (np.array(target_text).shape)

In [ ]:
# # Create batches from the dataset
# myData = tf.data.Dataset.from_tensor_slices((input_text, target_text)).shuffle(BUFFER_SIZE)
# myData = myData.batch(BATCH_SIZE, drop_remainder=True)
# myData

In [ ]:
# # Setup network generator structure...

# # Define loss function
# def loss_function(labels, logits):
#     return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# # Build model
# def build_model(batch_size):
#   model = tf.keras.Sequential([
#           Embedding(VOCAB_SIZE, 
#                     EMBEDDING_DIM,
#                     batch_input_shape=[batch_size, None]),
#           # Bidirectional(LSTM(N_RNN_UNITS, 
#                             #  return_sequences=True, 
#                             #  stateful=True,
#                             #  recurrent_activation='sigmoid',
#                             #  recurrent_initializer='glorot_uniform')),
#           # GRU(N_RNN_UNITS,
#           #     return_sequences=True,
#           #     stateful=True,
#           #     recurrent_activation='sigmoid',
#           #     recurrent_initializer='glorot_uniform'),
#           LSTM(N_RNN_UNITS,
#               return_sequences=True,
#               stateful=True,
#               recurrent_activation='sigmoid',
#               recurrent_initializer='glorot_uniform'),
#           LSTM(N_RNN_UNITS,
#               return_sequences=True,
#               stateful=True,
#               recurrent_activation='sigmoid',
#               recurrent_initializer='glorot_uniform'),
#           LSTM(N_RNN_UNITS,
#               return_sequences=True,
#               stateful=True,
#               recurrent_activation='sigmoid',
#               recurrent_initializer='glorot_uniform'),
#         Dense(VOCAB_SIZE)
#   ])
#   model.summary()
#   return model

# model = build_model(BATCH_SIZE)

# # Compile model
# opt_learning_rate = 0.005 # @param {type: ""}
# opt_beta = 0.5
# optimizer = Adam(opt_learning_rate, beta_1=opt_beta)

# # Compile the model
# model.compile(optimizer, loss_function)

In [ ]:
# #@title Save checkpoints during training
# # from google.colab import drive
# # drive.mount('/content/gdrive')
# path = 'My Drive/Colab/TextGen/' #@param{type: 'string'}
# full_path = path + "ckpt_{epoch}" 

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#                       filepath=full_path,
#                       save_weights_only=True)

# # Do the training... 
# n_epochs =   150#@param{type: 'integer'}
# history = model.fit(myData, epochs=n_epochs, callbacks=[checkpoint_callback])
# # history = model.fit(myData, epochs=n_epochs)

### 1.4 Generating data

In [ ]:
# # Load the model 
# batch_size = 1
# model = build_model(batch_size=batch_size)
# model.load_weights(tf.train.latest_checkpoint(path))
# model.build(tf.TensorShape([batch_size, None]))

In [ ]:
# #@title Generate text!

# # create a function for generating the text
# def generate_text(model, input_text, n_characters_output=1000):
#     # First, vectorize the input text as before
#     input_eval = [char2idx[s] for s in input_text]
#     input_eval = tf.expand_dims(input_eval, 0)

#     text_generated = []
#     # Generate the number of characters desired
#     model.reset_states()
#     for i in range(n_characters_output):
#       # Run input through model
#       predictions = model(input_eval)

#       # Remove the batch dimension
#       predictions = tf.squeeze(predictions, 0)

#       # Using a categorical distribution to predict the character returned by the model
#       predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

#       # Pass the predicted character as the next input to the model
#       input_eval = tf.expand_dims([predicted_id], 0)

#       # Add the predicted character to the output
#       text_generated.append(idx2char[predicted_id])

#     # Return output
#     return text_generated

# def generate_music():
#   input_text = [128]
#   n_output =  250#@param 
#   output = generate_text(model, input_text=input_text, n_characters_output=n_output)
#   return output

# # # Generate an example (used to make Figures 4 and 5 in the report)
# # output = generate_music()
# # converted_midi_path = "My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/generated.mid"
# # converted_back_midi = tokenizer.tokens_to_midi([output], get_midi_programs(mf))
# # converted_back_midi.dump(converted_midi_path)
# # myMidiFile = m21.converter.parse(converted_midi_path, format="midi")
# # myMidiFile.plot()

In [ ]:
# features = {
#     "midiTokens" : [],
#     "pitchCount" : [], # Number of pitches used at least once
#     "pitchRange" : [], # Difference between the highest and lowest pitches in semitones
#     "averageNoteDuration" : [],
#     "stepwiseMotion" : []
# }
# #===============================================================================

# for i in range(100):
#   output = generate_music()

#   converted_midi_path = "My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/generated.mid"
#   converted_back_midi = tokenizer.tokens_to_midi([output], get_midi_programs(mf))
#   converted_back_midi.dump(converted_midi_path)

#   myMidiFile = m21.converter.parse(converted_midi_path, format="midi")

#   features["midiTokens"].append(output)
#   features["pitchCount"].append(m21.features.jSymbolic.PitchVarietyFeature(myMidiFile).extract().vector[0]) 
#   features["pitchRange"].append(m21.features.jSymbolic.RangeFeature(myMidiFile).extract().vector[0])
#   features["averageNoteDuration"].append(m21.features.jSymbolic.AverageNoteDurationFeature(myMidiFile).extract().vector[0])
#   features["stepwiseMotion"].append(m21.features.jSymbolic.StepwiseMotionFeature(myMidiFile).extract().vector[0])

# # Save the file
# df = pd.DataFrame.from_dict(features)
# if CREATE_REGGAE:
#   df.to_csv("My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/reggae.csv")
# else:
#   df.to_csv("My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/bach.csv")

In [ ]:
# df #<-- print dataframe

## 2 Casual Creator Interface

NOTE: This uses some code drawn from the Hello Magenta notebook [found here](https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb), linked from the music generation practical.

### 2.1 Prerequisites

In [ ]:
# Install dependencies for GUI
!pip uninstall mido # There is some interesting issues using mido for 
                    # miditok versus with the magenta things... so 
                    # we manipulate this as needed. 
!pip install miditok
print('🎉 Done!')

#Setup libraries needed for the mock GUI
import pandas as pd
from miditok import REMI, get_midi_programs
import miditoolkit as mtk
import random

# mount drive and load midi files 
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 213 kB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
🎉 Done!
Mounted at /gdrive
/gdrive


In [ ]:
#
# Install requirements for magenta
# Note, we are strict on installing 
# mido first (through miditok) to avoid
# confusion with the magenta installation 
#
# See: https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb
#

!pip install magenta

import pretty_midi as pm
from IPython.display import Audio
# import fluidsynth

# Google Magenta 
from google.colab import files
import magenta
import note_seq

from note_seq.protobuf import music_pb2

print('🎉 Done!')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.9 MB/s 
     |████████████████████████████████| 3.6 MB 51.2 MB/s 
     |████████████████████████████████| 5.6 MB 14.9 MB/s 
     |████████████████████████████████| 1.6 MB 47.9 MB/s 
     |████████████████████████████████| 90 kB 13.2 MB/s 
     |████████████████████████████████| 2.3 MB 50.9 MB/s 
     |████████████████████████████████| 204 kB 74.5 MB/s 
     |████████████████████████████████| 254 kB 65.5 MB/s 
     |████████████████████████████████| 69 kB 8.7 MB/s 
     |████████████████████████████████| 352 kB 77.1 MB/s 
     |████████████████████████████████| 1.4 MB 54.8 MB/s 
     |████████████████████████████████| 210 kB 73.4 MB/s 
     |████████████████████████████████| 20.2 MB 2.0 MB/s 
     |████████████████████████████████| 385 kB 58.4 MB/s 
     |████████████████████████████████| 48 kB 2.3 MB/s 
     |███████████████████████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


🎉 Done!


In [ ]:
# Download checkpoint from magenta 
print('Copying checkpoint from GCS. This will take less than a minute...')
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

# Import dependencies.
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# Initialize the model.
print("Initializing Music VAE...")
mel_2bar_config = configs.CONFIG_MAP['cat-mel_2bar_big']
music_vae = TrainedModel(
      mel_2bar_config, 
      batch_size=4, 
      checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')

print('🎉 Done!')

Copying checkpoint from GCS. This will take less than a minute...
Initializing Music VAE...
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


INFO:tensorflow:Restoring parameters from /content/mel_2bar_big.ckpt
🎉 Done!


In [ ]:
# Setup Tokens
# mf = mtk.MidiFile("My Drive/Papers/Up-To-Stage-2/Comp Creativity Module/reggae-tunes/reggae_107.mid")

# Initialise the file 
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': False,
                       'rest_range': (2, 16),  # (half, 16 beats)
                       'nb_tempos': 32,  # nb of tempo bins
                       'tempo_range': (40, 250),  # (min, max)
                       'Program': True, 
                       'TimeSignature' : True}

# Creates the tokenizer and loads a MIDI
tokenizer = REMI(pitch_range, beat_res, nb_velocities, additional_tokens) # REMI encoding

### 2.2 GUI

In [ ]:
# For ease when marking, we hardcode the generated tokens and metrics calculated
# by following the steps in the previous section of this notebook. 

# It makes the cell initially slightly slow, but once you scroll past the dataframe, there are no issues. For interaction, I reccomend hiding the code alltogether.

# Load the dataframes 
reggaeDf = pd.DataFrame([[0, '[202, 10, 110, 124, 190, 5, 110, 123, 194, 5, 110, 123, 235, 202, 7, 113, 127, 210, 7, 113, 129, 1, 186, 7, 113, 133, 235, 202, 7, 113, 127, 210, 7, 113, 123, 214, 12, 113, 125, 1, 186, 14, 113, 124, 190, 14, 113, 124, 194, 14, 113, 124, 198, 14, 113, 124, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 10, 113, 124, 194, 5, 113, 124, 198, 10, 113, 124, 202, 5, 113, 124, 206, 10, 113, 124, 210, 10, 113, 124, 214, 9, 113, 124, 1, 186, 5, 113, 127, 194, 5, 113, 127, 202, 5, 113, 131, 214, 2, 112, 125, 1, 186, 7, 113, 133, 235, 202, 7, 113, 127, 210, 7, 113, 123, 214, 12, 113, 125, 1, 186, 14, 113, 124, 190, 14, 113, 124, 194, 14, 113, 124, 198, 14, 113, 124, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 10, 113, 124, 194, 5, 113, 124, 198, 10, 113, 124, 202, 5, 113, 124, 206, 10, 113, 124, 206, 10, 113, 124, 210, 10, 113, 124, 214, 9, 113, 124, 1, 186, 5, 113, 127, 194, 5, 113, 127, 202, 5, 113, 131, 214, 2, 112, 125, 1, 186, 7, 113, 133, 235, 202, 7, 113, 127, 210, 7, 113, 123, 214, 12, 113, 125, 1, 186, 14, 113, 124, 190, 14, 113, 124, 194, 14, 113, 124, 198, 14, 113, 124, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 14, 113]', 7, 12, 0.2865497076023392, 0.1111111111111111], [1, '[216, 19, 95, 123, 24, 117, 123, 216, 19, 114, 123, 1, 186, 23, 121, 123, 189, 19, 114, 123, 191, 24, 121, 123, 194, 226, 19, 113, 123, 24, 95, 123, 26, 110, 124, 188, 19, 121, 124, 190, 19, 121, 124, 194, 19, 121, 127, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194]', 5, 8, 0.2105263157894737, 0.0], [2, '[50, 121, 123, 57, 121, 123, 235, 194, 50, 121, 123, 57, 121, 123, 236, 210, 50, 119, 123, 59, 119, 123, 214, 26, 119, 123, 1, 186, 17, 119, 129, 23, 121, 125, 45, 121, 123, 53, 121, 123, 62, 119, 123, 188, 218, 43, 121, 123, 52, 121, 123, 60, 119, 123, 190, 23, 104, 125, 41, 121, 123, 50, 121, 123, 57, 121, 123, 190, 23, 104, 125, 194, 17, 119, 129, 23, 121, 123, 196, 23, 104, 127, 198, 9, 112, 124, 21, 121, 137, 27, 104, 131, 202, 218, 14, 112, 127, 17, 119, 129, 23, 121, 127, 206, 230, 23, 104, 123, 26, 108, 123, 29, 108, 123, 33, 108, 123, 208, 27, 104, 127, 210, 9, 110, 129, 17, 121, 123, 214, 17, 121, 123, 1, 186, 18, 121, 127, 194, 25, 121, 123, 48, 91, 123, 53, 91, 123, 55, 91, 123, 57, 90, 123, 60, 91, 123, 210, 220, 45, 91, 124, 48, 90, 124, 53, 91, 123, 57, 91, 123, 60, 91, 123, 215, 222, 53, 90, 123, 57, 90, 123, 60, 90, 123, 1, 189, 194, 19, 117, 125, 235, 202, 16, 115, 127, 235, 214, 19, 114, 123, 1, 186, 23, 116, 123, 190, 23, 114, 123, 194, 19, 118, 125, 235, 202, 16, 116, 125, 235, 210, 16, 116, 123, 214, 14, 115, 123, 1, 186, 12, 117, 127, 235, 198, 12, 116, 125, 237, 235, 1, 186, 47, 110, 123, 235, 194, 47, 110, 124, 236]', 27, 53, 0.2589285714285714, 0.0], [3, '[47, 114, 124, 51, 114, 124, 1, 189, 218, 42, 114, 123, 47, 120, 123, 51, 120, 123, 194, 218, 41, 113, 124, 44, 119, 124, 49, 113, 124, 200, 218, 42, 113, 124, 47, 114, 124, 51, 114, 124, 202, 9, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 124, 190, 9, 114, 124, 194, 9, 114, 124, 198, 9, 114, 124, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 124, 190, 9, 114, 124, 194, 9, 114, 124, 198, 9, 114, 124, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9]', 10, 49, 0.3020833333333333, 0.1063829787234042], [4, '[212, 218, 41, 121, 124, 44, 114, 124, 49, 121, 124, 215, 218, 42, 119, 124, 47, 119, 124, 51, 113, 124, 1, 189, 218, 42, 116, 123, 194, 47, 117, 123, 197, 47, 110, 123, 202, 47, 114, 125, 206, 49, 114, 126, 207, 52, 114, 125, 210, 36, 114, 127, 215, 35, 114, 185, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 251, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 185, 1, 1, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 185, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 185, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 185, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 185, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214, 26, 114, 169]', 10, 26, 3.403153153153153, 0.03125], [5, '[210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206]', 8, 14, 0.1840277777777777, 0.0677966101694915], [6, '[196, 49, 115, 123, 52, 115, 123, 235, 202, 47, 121, 123, 52, 121, 123, 235, 210, 47, 121, 123, 50, 121, 123, 236, 190, 49, 121, 127, 55, 121, 127, 196, 54, 121, 129, 214, 7, 121, 125, 1, 186, 14, 121, 123, 190, 14, 121, 124, 194, 14, 121, 127, 202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 12, 121, 124, 202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 14, 121, 124, 202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 14, 121, 124, 202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 12, 121, 124, 202, 14, 121, 125, 207, 14, 121, 124, 210, 14, 121, 125, 235, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12]', 11, 48, 0.3060606060606061, 0.22], [7, '[194, 14, 121, 127, 202, 14, 121, 129, 236, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 202, 16, 121, 129, 237, 235, 198, 8, 121, 124, 202, 11, 121, 124, 206, 9, 121, 124, 210, 11, 121, 125, 235, 186, 16, 121, 127, 194, 16, 121, 127, 202, 16, 121, 129, 238, 202, 23, 121, 124, 206, 21, 121, 124, 210, 20, 121, 125, 235, 186, 13, 121, 129, 194, 13, 121, 127, 202, 13, 121, 129, 237, 235, 198, 18, 121, 124, 202, 20, 121, 124, 206, 18, 121, 124, 210, 20, 121, 126, 1, 186, 13, 121, 127, 194, 13, 121, 127, 202, 13, 121, 129, 246, 202, 20, 121, 124, 206, 20, 121, 124, 210, 20, 121, 125, 235, 186, 18, 121, 129, 194, 18, 121, 129, 202, 18, 121, 129, 238, 202, 13, 121, 124, 206, 10, 121, 124, 210, 14, 121, 125, 235, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121]', 13, 16, 0.2770467836257309, 0.2678571428571428], [8, '[202, 7, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 121, 125, 235, 202, 14, 121, 125, 206, 12, 121, 123, 210, 9, 121, 125, 237, 235, 202, 10, 121, 123, 206, 10, 121, 123, 210, 10, 121, 123, 214, 10, 121, 123, 1, 186, 17, 121, 125, 190, 17, 121, 123, 194, 17, 121, 127, 202, 10, 121, 125, 235, 210, 10, 121, 127, 237, 202, 14, 121, 125, 206, 12, 121, 123, 210, 9, 121, 123, 214, 12, 121, 123, 1, 186, 14, 121, 123, 190, 14, 121, 124, 194, 14, 121, 123, 198, 14, 121, 124, 202, 14, 121, 129, 237, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 12, 121, 124, 202, 14, 121, 125, 207, 14, 121, 124, 210, 14, 121, 125, 235, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 127, 210, 10, 121, 124, 214]', 8, 11, 0.2579022988505748, 0.2105263157894736], [9, '[206, 15, 113, 124, 210, 18, 114, 124, 214, 17, 114, 124, 1, 186, 17, 112, 126, 194, 17, 115, 126, 202, 22, 117, 124, 206, 23, 113, 124, 210, 22, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 190, 14, 112, 127, 198, 14, 112, 127, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 7, 112, 124, 198, 12, 112, 124, 202, 7, 112, 124, 206, 12, 112, 124, 210, 7, 112, 124, 214, 12, 112, 124, 1, 186, 7, 112, 124, 190, 6, 112, 124, 194, 14, 112, 124, 198, 14, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 127, 210, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 18, 112, 124, 210, 19, 112, 124, 214, 18, 112, 124, 1, 186, 14, 112, 124, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112]', 10, 17, 0.2690677966101697, 0.2241379310344827], [10, '[200, 47, 110, 124, 235, 206, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235]', 1, 0, 0.125, 0.0], [11, '[52, 121, 124, 242, 235, 198, 28, 121, 124, 40, 121, 124, 200, 29, 121, 124, 41, 121, 124, 202, 31, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 114, 145, 251, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 214, 26, 111, 127, 1, 186, 22, 107, 124, 1, 1, 1, 1, 1, 1, 1]', 12, 30, 0.2197712418300654, 0.0851063829787234], [12, '[236, 235, 194, 34, 121, 123, 236, 206, 34, 121, 123, 208, 34, 121, 123, 210, 34, 121, 123, 236, 235, 194, 34, 121, 123, 236, 206, 34, 121, 123, 208, 34, 121, 123, 210, 34, 121, 123, 236, 235, 194, 34, 121, 123, 236, 206, 34, 121, 123, 208, 34, 121, 123, 210, 34, 121, 123, 251, 235, 1, 1, 1, 1, 1, 194, 43, 121, 123, 200, 58, 110, 125, 202, 53, 109, 125, 204, 58, 109, 125, 208, 53, 110, 125, 210, 58, 109, 126, 212, 53, 109, 123, 216, 58, 109, 125, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 23, 117, 123, 190, 23, 113, 123, 194, 19, 118, 125, 235, 202, 16, 115, 125, 235, 210]', 10, 48, 0.2625786163522011, 0.0444444444444444], [13, '[45, 114, 123, 48, 113, 123, 53, 114, 123, 57, 116, 123, 60, 114, 123, 199, 41, 110, 123, 45, 113, 123, 48, 113, 123, 53, 114, 123, 57, 115, 123, 60, 114, 123, 205, 41, 110, 123, 45, 113, 123, 48, 113, 123, 53, 113, 123, 57, 113, 123, 60, 113, 123, 207, 57, 110, 123, 45, 100, 123, 50, 102, 123, 235, 190, 45, 121, 129, 50, 121, 129, 196, 45, 114, 123, 50, 114, 123, 235, 202, 45, 108, 129, 50, 108, 129, 208, 45, 114, 123, 50, 109, 123, 210, 45, 121, 123, 50, 121, 123, 214, 45, 108, 123, 50, 104, 123, 216, 45, 114, 123, 50, 114, 123, 1, 188, 45, 102, 123, 50, 99, 123, 190, 45, 108, 123, 50, 108, 123, 194, 45, 108, 123, 50, 108, 123, 198, 47, 108, 123, 50, 108, 123, 200, 45, 108, 123, 50, 108, 123, 202, 43, 117, 123, 55, 108, 123, 204, 43, 117, 123, 55, 108, 123, 208, 43, 117, 123, 55, 108, 123, 210, 43, 117, 123, 55, 108, 123, 214, 43, 117, 123, 55, 108, 123, 216, 43, 117, 123, 55, 108, 123, 1, 188, 43, 117, 123, 55, 108, 123, 190, 43, 117, 123, 55, 108, 123, 194, 43, 117, 123, 55, 108, 123, 196, 43, 117, 123, 55, 108, 123, 200, 43, 117, 123, 55, 108, 123, 202, 43, 117, 123, 55, 108, 123, 204, 43, 117, 123, 55, 108, 123, 208, 43, 117, 123, 55, 108, 123, 210]', 10, 19, 0.1330645161290322, 0.0], [14, '[202, 11, 114, 129, 210, 7, 114, 129, 1, 186, 12, 114, 131, 235, 202, 12, 114, 126, 208, 7, 114, 126, 214, 2, 114, 126, 1, 186, 7, 114, 125, 190, 7, 114, 125, 194, 7, 114, 125, 198, 7, 114, 124, 202, 11, 114, 129, 210, 7, 114, 127, 1, 1, 186, 46, 121, 125, 206, 48, 121, 125, 210, 52, 121, 124, 214, 52, 121, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 198, 43, 121, 123, 202, 52, 121, 124, 204, 50, 121, 123, 206, 48, 121, 126, 210, 52, 121, 129, 240, 194, 46, 121, 123, 198, 46, 121, 123, 202, 47, 121, 124, 206, 46, 121, 123, 45, 121, 123, 52, 121, 123, 207, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 208, 218, 43, 121, 122, 48, 121, 122, 52, 121, 122, 235, 213, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 1, 186, 222, 43, 118, 151, 47, 121, 151, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124]', 15, 50, 0.2658333333333327, 0.1666666666666666], [15, '[214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 124, 190, 9, 114, 124, 194, 9, 114, 124, 198, 9, 114, 124, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 124, 190, 9, 114, 124, 194, 9, 114, 124, 198, 9, 114, 124, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186]', 4, 14, 0.3267543859649125, 0.1071428571428571], [16, '[206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 20, 121, 125, 235, 186, 13, 121, 129, 194, 13, 121, 127, 202, 13, 121, 129, 237, 235, 198, 18, 121, 124, 202, 20, 121, 124, 206, 18, 121, 124, 210, 20, 121, 126, 1, 186, 13, 121, 127, 194, 13, 121, 127, 202, 13, 121, 129, 237, 235, 198, 18, 121, 124, 202, 20, 121, 124, 206, 18, 121, 124, 210, 18, 121, 125, 235, 186, 16, 121, 127, 194, 16, 121, 127, 202, 16, 121, 129, 238, 202, 23, 121, 124, 206, 21, 121, 124, 210, 20, 121, 125, 235, 186, 11, 121, 129, 194, 11, 121, 127, 202, 11, 121, 129, 237, 235, 198, 16, 121, 124, 202, 18, 121, 124, 206, 16, 121, 124, 210, 18, 121, 125, 235, 186, 16, 121, 129, 194, 16, 121, 129, 202, 16, 121, 129, 237, 235, 198, 8, 121]', 9, 13, 0.2690058479532164, 0.3035714285714285], [17, '[216, 218, 41, 113, 124, 44, 119, 124, 49, 113, 124, 1, 190, 218, 41, 110, 124, 44, 116, 124, 49, 110, 124, 194, 218, 41, 112, 123, 44, 112, 123, 202, 10, 116, 126, 210, 16, 117, 125, 235, 186, 23, 116, 124, 190, 18, 115, 123, 194, 18, 114, 125, 235, 202, 13, 117, 126, 210, 13, 114, 126, 1, 186, 19, 114, 124, 190, 19, 114, 124, 194, 19, 114, 124, 198, 18, 114, 124, 202, 16, 114, 127, 210, 16, 114, 127, 236, 194, 16, 114, 127, 202, 11, 114, 129, 210, 16, 114, 126, 1, 186, 19, 114, 124, 190, 19, 114, 124, 194, 19, 114, 124, 251, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 192, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 132, 1, 202, 12, 110, 126, 210, 12, 110, 126, 1, 186, 17, 110, 129, 235, 198, 17, 110, 123, 202, 17, 110, 123, 206, 14, 110, 123, 210, 17, 110, 123, 214, 14, 110, 125, 1, 186, 17, 110, 123, 190, 16, 110, 123, 194, 14, 110, 125, 235, 202, 12, 110, 126, 210, 12, 110, 126, 1, 186, 17, 110, 129, 235, 198, 17, 110, 123, 202, 17, 110, 123, 206, 14, 110, 123, 210, 17, 110, 123, 214, 14, 110, 125]', 13, 39, 0.2549019607843136, 0.1276595744680851], [18, '[214, 17, 114, 124, 1, 186, 15, 114, 126, 194, 15, 114, 126, 202, 20, 115, 124, 206, 22, 114, 124, 210, 20, 114, 125, 214, 18, 113, 123, 1, 186, 20, 114, 125, 235, 194, 17, 114, 125, 235, 202, 17, 117, 124, 206, 21, 115, 123, 210, 17, 114, 123, 214, 21, 116, 123, 1, 186, 21, 112, 123, 190, 16, 116, 123, 194, 9, 118, 126, 202, 9, 116, 129, 210, 12, 116, 126, 1, 186, 14, 117, 129, 235, 198, 14, 116, 125, 202, 17, 116, 123, 206, 21, 115, 123, 210, 17, 115, 123, 214, 21, 118, 123, 1, 186, 21, 114, 123, 190, 16, 117, 123, 194, 9, 117, 126, 202, 9, 117, 129, 210, 12, 115, 129, 1, 186, 14, 117, 131, 235, 202, 17, 118, 124, 206, 21, 117, 123, 210, 17, 116, 123, 214, 21, 116, 123, 1, 186, 21, 113, 123, 190, 16, 116, 123, 194, 9, 114, 126, 202, 9, 118, 129, 210, 12, 117, 127, 236, 194, 14, 117, 126, 202, 17, 118, 125, 235, 202, 21, 118, 123, 206, 19, 115, 123, 210, 21, 116, 125, 235, 186, 14, 118, 131, 198, 14, 116, 123, 60, 113, 123, 236, 235, 210, 43, 111, 123, 46, 114, 123, 50, 113, 123, 55, 114, 123, 58, 114, 123, 62, 113, 123, 236, 235, 194, 43, 113, 123, 46, 116, 123, 50, 114, 123, 55, 113, 123, 58, 114, 123, 62, 113, 123, 236, 235, 210, 43, 112, 123, 46]', 18, 53, 0.238095238095238, 0.2272727272727272], [19, '[191, 19, 121, 123, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202]', 6, 8, 0.1909090909090909, 0.1111111111111111], [20, '[198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36]', 6, 12, 0.1812499999999999, 0.0677966101694915], [21, '[50, 121, 123, 43, 109, 123, 46, 113, 123, 50, 111, 123, 55, 114, 123, 58, 114, 123, 62, 114, 123, 236, 235, 210, 43, 113, 123, 46, 114, 123, 50, 114, 123, 55, 114, 123, 58, 114, 123, 62, 113, 123, 236, 235, 194, 43, 113, 123, 46, 116, 123, 50, 114, 123, 55, 113, 123, 58, 114, 123, 62, 113, 123, 236, 235, 210, 43, 112, 123, 46, 114, 123, 50, 114, 123, 55, 114, 123, 58, 114, 123, 62, 113, 123, 236, 235, 194, 43, 113, 123, 46, 116, 123, 50, 114, 123, 55, 113, 123, 58, 114, 123, 62, 113, 123, 236, 235, 210, 43, 112, 123, 46, 114, 123, 50, 114, 123, 55, 114, 123, 58, 114, 123, 62, 113, 123, 236, 235, 194, 43, 113, 123, 46, 114, 123, 50, 113, 123, 55, 114, 123, 58, 114, 123, 62, 113, 123, 236, 235, 210, 36, 113, 123, 236, 235, 194, 33, 112, 123, 45, 113, 123, 48, 113, 123, 53, 114, 123, 57, 115, 123, 60, 114, 123, 199, 41, 112, 123, 45, 113, 123, 48, 113, 123, 53, 114, 123, 57, 114, 123, 60, 113, 123, 236, 210, 41, 113, 123, 214, 16, 108, 124, 202, 16, 108, 127, 210, 16, 108, 127, 1, 186, 11, 108, 127, 194, 11, 108, 127, 202, 11, 108, 132, 214, 17, 108, 124, 1, 186, 16, 108, 124, 190, 16, 108, 124, 194, 16, 108, 124, 198, 16, 108, 124, 202, 16, 108, 127, 210, 16]', 17, 51, 0.2228260869565216, 0.0833333333333333], [22, '[1, 186, 14, 112, 124, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 127, 251, 1, 1, 1, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 18, 112, 124, 210, 19, 112, 124, 214, 18, 112, 124, 1, 186, 14, 112, 124, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112]', 4, 7, 0.2995689655172413, 0.1929824561403508], [23, '[50, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121]', 5, 32, 0.2132768361581922, 0.1896551724137931], [24, '[198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117]', 4, 8, 0.3238304093567252, 0.1071428571428571], [25, '[202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141]', 4, 8, 0.3545197740112997, 0.0689655172413793], [26, '[208, 14, 113, 124, 210, 16, 114, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110]', 5, 8, 0.3577586206896555, 0.0714285714285714], [27, '[49, 119, 124, 205, 218, 42, 117, 123, 47, 117, 123, 51, 117, 123, 210, 218, 42, 108, 124, 47, 114, 124, 51, 114, 124, 216, 218, 42, 110, 124, 47, 110, 124, 51, 110, 124, 236, 194, 218, 41, 114, 124, 44, 121, 124, 49, 114, 123, 236, 235, 210, 218, 41, 121, 124, 44, 114, 124, 49, 114, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 114, 124, 236, 235, 210, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 251, 235, 1, 1, 1, 1, 194, 218, 44, 121, 124, 47, 121, 124, 51, 121, 124, 236, 235, 210, 218, 42, 121, 124, 47, 121, 124, 51, 121, 123, 236, 235, 194, 218, 44, 121, 124, 47, 121, 124, 52, 121, 124, 236, 235, 210, 218, 44, 121, 124, 47, 121, 124, 52, 114, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 186, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 188, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 192, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 196, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 198, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 202, 218, 43, 121, 123, 48, 121, 123, 52, 121]', 9, 11, 0.1479166666666669, 0.5], [28, '[198, 58, 92, 123, 235, 214, 13, 108, 123, 1, 186, 14, 108, 123, 190, 14, 108, 123, 194, 14, 108, 123, 198, 14, 108, 123, 202, 14, 108, 127, 210, 14, 108, 123, 214, 14, 108, 124, 1, 186, 16, 108, 124, 190, 13, 108, 124, 194, 9, 108, 127, 202, 9, 108, 129, 235, 214, 13, 108, 123, 1, 186, 14, 108, 123, 190, 14, 108, 123, 194, 14, 108, 123, 198, 14, 108, 123, 202, 14, 108, 127, 210, 14, 108, 123, 214, 14, 108, 124, 1, 186, 16, 108, 124, 190, 13, 108, 124, 194, 9, 108, 127, 202, 9, 108, 129, 235, 214, 13, 108, 123, 1, 186, 14, 108, 123, 190, 14, 108, 123, 194, 14, 108, 123, 198, 14, 108, 123, 202, 14, 108, 127, 210, 14, 108, 123, 214, 14, 108, 124, 1, 186, 16, 108, 124, 190, 13, 108, 124, 194, 9, 108, 127, 202, 9, 108, 129, 235, 214, 13, 108, 123, 1, 186, 14, 108, 123, 190, 14, 108, 123, 194, 14, 108, 123, 198, 14, 108, 123, 202, 14, 108, 127, 210, 14, 108, 123, 214, 14, 108, 124, 1, 186, 16, 108, 124, 190, 13, 108, 124, 190, 9, 108, 123, 194, 9, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124]', 9, 49, 0.1949152542372883, 0.1578947368421052], [29, '[210, 12, 110, 126, 1, 186, 17, 110, 129, 235, 198, 17, 110, 123, 202, 17, 110, 129, 235, 214, 17, 110, 123, 1, 186, 17, 110, 123, 190, 16, 110, 123, 194, 14, 110, 125, 235, 202, 12, 110, 126, 210, 12, 110, 126, 1, 186, 17, 110, 129, 235, 198, 10, 114, 124, 202, 11, 114, 124, 206, 10, 114, 124, 210, 11, 114, 127, 1, 186, 15, 114, 135, 202, 15, 114, 123, 206, 15, 114, 123, 210, 15, 114, 123, 214, 17, 114, 124, 1, 186, 18, 114, 127, 194, 18, 114, 127, 202, 15, 114, 124, 206, 15, 114, 124, 210, 18, 114, 131, 1, 190, 47, 110, 123, 191, 48, 110, 122, 194, 48, 110, 123, 198, 48, 110, 123, 202, 48, 104, 123, 204, 48, 103, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202]', 10, 38, 0.3074712643678163, 0.2], [30, '[191, 50, 121, 123, 194, 50, 121, 124, 198, 48, 121, 124, 202, 43, 121, 127, 210, 16, 114, 127, 1, 186, 15, 114, 135, 202, 15, 114, 123, 206, 15, 114, 123, 210, 15, 114, 123, 214, 15, 113, 123, 1, 186, 21, 119, 127, 194, 21, 118, 125, 235, 202, 21, 118, 123, 206, 19, 115, 123, 210, 21, 116, 125, 235, 186, 23, 118, 125, 235, 194, 19, 117, 125, 235, 202, 16, 115, 129, 235, 214, 19, 120, 123, 1, 186, 23, 117, 123, 190, 23, 114, 123, 194, 19, 119, 125, 235, 202, 16, 116, 125, 235, 210, 16, 116, 123, 214, 14, 117, 123, 1, 186, 12, 116, 127, 194, 12, 117, 127, 235, 206, 16, 116, 123, 210, 19, 118, 123, 214, 16, 114, 123, 1, 186, 21, 118, 125, 235, 194, 21, 118, 125, 235, 202, 21, 117, 123, 206, 19, 116, 123, 210, 21, 116, 123, 235, 1, 186, 23, 117, 127, 194, 19, 116, 125, 235, 202, 16, 115, 129, 235, 214, 19, 116, 123, 1, 186, 23, 117, 123, 190, 23, 114, 123, 194, 19, 118, 125, 235, 202, 16, 116, 125, 235, 210, 16, 116, 123, 214, 14, 117, 123, 1, 186, 12, 116, 127, 194, 12, 117, 127, 235, 206, 16, 114, 123, 210, 19, 118, 123, 214, 16, 115, 123, 1, 186, 21, 120, 125, 235, 194, 21, 118, 125, 235, 202, 21, 117, 123, 206, 19, 116, 123, 210, 21, 116, 123, 235, 1, 186]', 10, 38, 0.2196969696969696, 0.2592592592592592], [31, '[44, 121, 124, 49, 121, 124, 236, 235, 210, 218, 42, 114, 124, 47, 121, 124, 51, 121, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 236, 235, 210, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 236, 235, 194, 218, 41, 121, 124, 44, 121, 124, 49, 121, 124, 236, 235, 210, 218, 42, 121, 124, 47, 121, 124, 51, 121, 124, 236, 235, 194, 218, 42, 114, 124, 47, 121, 124, 51, 121, 124, 236, 235, 210, 218, 42, 121, 124, 47, 121, 124, 51, 121, 124, 236, 235, 194, 218, 42, 121, 123, 48, 121, 123, 52, 121, 123, 236, 210, 218, 44, 121, 124, 47, 121, 124, 52, 114, 124, 206, 218, 44, 105, 124, 47, 103, 124, 52, 103, 124, 210, 47, 98, 124, 52, 98, 124, 216, 47, 110, 129, 55, 108, 123, 196, 43, 117, 123, 55, 108, 123, 198, 43, 117, 123, 55, 108, 123, 202, 43, 117, 123, 55, 108, 123, 204, 43, 117, 123, 55, 108, 123, 208, 43, 117, 123, 55, 108, 123, 210, 43, 117, 123, 55, 108, 123, 214, 43, 117, 123, 55, 108, 123, 216, 43, 117, 123, 55, 108, 123, 1, 188, 43, 117, 123, 55, 108, 123, 190, 43, 117, 123, 55, 108, 123, 194, 43, 117, 123, 55, 108, 123, 196, 43, 117, 123, 55, 108, 123, 200, 43, 117, 123, 55, 108, 123, 202, 43, 117, 123, 55, 108, 123, 206, 43, 117, 123]', 10, 14, 0.1781609195402298, 0.0], [32, '[198, 14, 121, 124, 202, 14, 121, 129, 236, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 127, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 123, 190, 14, 121, 124, 194, 14, 121, 127, 202, 14, 121, 129, 236, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 127, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 123, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 9, 121, 127, 210, 9, 121, 127, 237, 202, 14, 121, 125, 206, 12, 121, 123, 210, 9, 121, 123, 214, 12, 121, 123, 1, 186, 14, 121, 123, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 9, 121, 127, 210, 9, 121, 127, 237, 202, 10, 121, 123, 206, 10, 121, 123, 210, 10, 121, 123, 214, 10, 121, 123, 1, 186, 17]', 5, 7, 0.252155172413793, 0.1929824561403508], [33, '[44, 121, 124, 49, 114, 124, 236, 235, 210, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 236, 235, 210, 218, 41, 121, 124, 44, 121, 124, 49, 110, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 123, 48, 121, 123, 52, 121, 124, 251, 235, 1, 1, 1, 1, 186, 218, 44, 114, 123, 48, 121, 123, 53, 114, 123, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 127, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 123, 190, 14, 121, 124, 194, 14, 121, 123, 198, 14, 121, 124, 202, 14, 121, 129, 236, 186, 7]', 11, 46, 0.2025862068965504, 0.0476190476190476], [34, '[208, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 135, 237, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 135, 237, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117]', 3, 3, 0.2157738095238097, 0.2909090909090909], [35, '[50, 121, 123, 235, 202, 43, 121, 124, 50, 121, 124, 206, 43, 120, 123, 50, 119, 123, 235, 210, 47, 116, 123, 50, 116, 123, 235, 216, 41, 109, 123, 235, 202, 15, 117, 125, 235, 210, 15, 118, 125, 235, 186, 15, 118, 133, 235, 202, 17, 118, 129, 210, 18, 115, 127, 236, 194, 17, 118, 123, 235, 202, 17, 119, 129, 236, 186, 17, 119, 129, 43, 119, 123, 50, 119, 123, 58, 119, 123, 188, 38, 108, 123, 41, 108, 123, 43, 117, 123, 55, 108, 123, 214, 43, 117, 123, 55, 108, 123, 216, 43, 117, 123, 55, 108, 123, 1, 188, 43, 117, 123, 55, 108, 123, 190, 43, 117, 123, 55, 90, 123, 58, 90, 123, 194, 55, 103, 127, 198, 12, 112, 124, 202, 12, 112, 124, 206, 7, 112, 124, 210, 12, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 214, 12, 117, 124, 1, 186, 22, 114, 125, 235, 194, 22, 114, 126, 202, 18, 114, 124, 206, 17, 113, 124, 210, 18, 114, 124, 214, 17, 114, 124, 1, 186, 11, 115, 125, 235, 194, 11, 114, 123, 236, 206, 15, 115, 123, 210, 18, 113, 123, 214, 36, 112, 123, 48, 112, 123, 216, 36, 111, 123, 48, 111, 123, 1, 188, 36, 110, 123, 48, 110, 123, 190, 36, 112, 123, 48, 112, 123, 194, 36, 111, 123, 48, 111, 123, 198, 36, 110, 123, 48, 110, 123, 202, 43, 109, 123, 204]', 16, 51, 0.2035984848484847, 0.2222222222222222], [36, '[53, 118, 124, 206, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 210, 220, 43, 121, 123, 48, 121, 123, 52, 121, 123, 58, 118, 126, 212, 55, 121, 127, 214, 219, 43, 121, 123, 48, 121, 123, 52, 121, 123, 57, 121, 127, 214, 58, 118, 126, 215, 55, 121, 128, 1, 186, 40, 121, 123, 52, 121, 123, 190, 41, 121, 123, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 14, 121, 123, 190, 19, 121, 123, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 14, 121, 123, 190, 19, 121, 123, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 14, 121, 123, 190, 19, 121, 123, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210]', 11, 44, 0.1633986928104575, 0.0], [37, '[194, 47, 107, 124, 52, 107, 124, 200, 47, 106, 124, 52, 106, 124, 206, 47, 105, 124, 52, 105, 123, 215, 50, 105, 123, 216, 50, 105, 123, 1, 186, 50, 105, 123, 190, 50, 105, 123, 192, 50, 105, 123, 196, 50, 105, 123, 198, 50, 105, 123, 202, 50, 105, 123, 206, 53, 105, 123, 210, 50, 105, 123, 212, 46, 105, 123, 216, 50, 105, 123, 1, 188, 50, 105, 123, 190, 50, 105, 123, 194, 50, 105, 123, 198, 50, 105, 123, 200, 50, 105, 123, 204, 50, 105, 123, 206, 21, 116, 123, 210, 17, 116, 123, 214, 21, 115, 123, 1, 186, 21, 111, 123, 190, 16, 117, 123, 194, 9, 117, 126, 202, 9, 117, 129, 210, 12, 115, 129, 1, 186, 14, 117, 131, 235, 202, 17, 118, 124, 206, 21, 117, 123, 210, 17, 116, 123, 214, 21, 116, 123, 1, 186, 21, 113, 123, 190, 16, 116, 123, 194, 9, 118, 126, 202, 9, 117, 129, 210, 12, 116, 126, 1, 186, 14, 117, 129, 235, 198, 14, 116, 125, 202, 17, 117, 129, 210, 19, 115, 125, 235, 186, 21, 117, 125, 190, 22, 114, 123, 194, 22, 117, 124, 235, 202, 22, 117, 126, 210, 22, 114, 124, 214, 20, 115, 123, 1, 186, 22, 114, 125, 235, 194, 22, 116, 125, 235, 202, 22, 115, 124, 206, 23, 112, 124, 210, 22, 112, 124, 214, 18, 112, 124, 1, 186, 14, 112, 124, 190, 12, 112]', 16, 44, 0.199404761904762, 0.173076923076923], [38, '[198, 18, 112, 124, 202, 14, 112, 124, 206, 18, 112, 124, 210, 19, 112, 124, 214, 18, 112, 124, 1, 186, 14, 112, 124, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 124, 206, 12, 112, 124, 210, 7, 112, 124, 214, 12, 112, 124, 1, 186, 7, 112, 124, 190, 6, 112, 124, 194, 14, 112, 124, 198, 14, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 124, 206, 7, 112, 124, 210, 12, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 190, 14, 112, 127, 198, 14, 112, 127, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 7, 112, 124, 198, 12, 112, 124, 202, 7, 112, 124, 206, 12, 112, 124, 210, 7, 112, 124, 214, 12, 112, 124, 1, 186, 7, 112, 124, 190, 6, 112, 124, 194, 14, 112, 124, 198, 14, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 124, 206, 7, 112, 124, 210, 12, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 190, 14, 112, 127]', 6, 13, 0.2430555555555557, 0.1864406779661017], [39, '[204, 50, 121, 123, 214, 50, 121, 123, 215, 54, 121, 123, 1, 186, 55, 121, 137, 202, 50, 121, 129, 210, 47, 121, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 198, 50, 110, 123, 202, 43, 114, 123, 50, 114, 123, 206, 47, 114, 123, 50, 114, 123, 210, 47, 112, 123, 50, 112, 123, 214, 47, 110, 123, 50, 117, 123, 216, 47, 118, 123, 50, 118, 123, 1, 186, 45, 113, 123, 50, 112, 123, 188, 47, 116, 123, 50, 112, 123, 190, 47, 118, 123, 50, 117, 123, 192, 47, 116, 123, 50, 116, 123, 196, 47, 116, 123, 50, 116, 123, 198, 47, 116, 123, 50, 94, 123, 202, 48, 94, 123, 55, 92, 124, 206, 48, 92, 123, 192, 45, 92, 123, 52, 92, 124, 196, 48, 92, 123, 198, 45, 91, 123, 52, 91, 124, 202, 48, 91, 123, 204, 45, 92, 123, 52, 91, 124, 206, 48, 90, 123, 210, 45, 90, 123, 52, 90, 124, 251, 235, 1, 186, 12, 115, 123, 190, 12, 116, 123, 194, 12, 117, 125, 202, 12, 105, 125, 206, 7, 105, 125, 210, 12, 105, 126, 214, 7, 105, 125, 1, 186, 5, 105, 124, 190, 5, 105, 123, 194, 5, 105, 127, 202, 5, 105, 127, 251, 1, 186, 12, 105, 126, 194, 12, 105, 127, 202]', 11, 50, 0.2239583333333338, 0.0], [40, '[194, 9, 108, 127, 202, 9, 108, 129, 235, 214, 13, 108, 123, 1, 186, 14, 108, 124, 190, 9, 108, 123, 194, 14, 108, 129, 202, 14, 108, 127, 210, 14, 108, 135, 1, 194, 10, 113, 124, 198, 10, 113, 124, 202, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 1, 186, 48, 105, 124, 190, 50, 105, 124, 194, 45, 105, 127, 198, 50, 105, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 242, 202, 18, 113, 123, 206, 13, 111, 123, 210, 13, 114, 123, 214, 13, 114, 123, 1, 186, 12, 116, 129, 236, 202, 12, 119, 123, 206, 12, 115, 123, 210, 14, 116, 123, 214, 12, 116, 123, 1, 186, 14, 117, 132, 202, 14, 117, 124, 206, 12, 113, 123, 210, 14, 117, 123, 214, 19, 116, 123, 1, 186, 14, 117, 132, 235, 202, 14, 119, 124, 206, 12, 114, 123, 210, 14, 117, 123, 214, 16, 116, 123, 1, 186, 17, 117, 131, 235, 202, 17, 119, 123, 206, 17, 115, 123, 210, 17, 105, 123, 194, 36]', 15, 41, 0.2641369047619047, 0.28], [41, '[236, 202, 10, 115, 123, 206, 10, 110, 125, 210, 13, 117, 129, 1, 186, 15, 114, 127, 236, 202, 10, 115, 123, 206, 10, 111, 125, 210, 13, 117, 129, 1, 186, 10, 115, 125, 235, 194, 10, 116, 123, 235, 202, 10, 116, 125, 206, 8, 114, 125, 210, 10, 117, 125, 237, 235, 202, 10, 115, 123, 206, 10, 110, 125, 210, 13, 117, 129, 1, 186, 15, 114, 127, 236, 202, 10, 115, 123, 206, 10, 112, 125, 210, 13, 117, 125, 235, 186, 14, 118, 131, 235, 202, 14, 119, 124, 206, 12, 114, 123, 210, 14, 118, 123, 214, 16, 116, 123, 1, 186, 17, 114, 125, 235, 194, 17, 121, 125, 235, 202, 12, 121, 123, 206, 12, 121, 123, 210, 14, 121, 125, 214, 16, 121, 123, 1, 186, 17, 121, 127, 194, 24, 121, 123, 198, 24, 121, 123, 202, 24, 121, 123, 206, 24, 121, 123, 210, 24, 121, 123, 214, 21, 121, 123, 1, 186, 22, 121, 125, 235, 194, 17, 121, 127, 202, 10, 121, 125, 235, 210, 10, 121, 127, 237, 202, 14, 121, 125, 206, 12, 121, 123, 210, 9, 121, 123, 214, 12, 121, 123, 1, 186, 14, 121, 123, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 208, 17, 121, 123, 235, 214, 19, 121, 123, 1, 186, 22, 121, 127, 236, 202, 19, 121, 125, 206, 17, 121, 123, 210, 14, 121, 127, 1, 186, 21, 121, 125]', 13, 16, 0.228422619047619, 0.3090909090909091], [42, '[17, 121, 125, 236, 235, 186, 17, 121, 126, 236, 199, 17, 121, 123, 202, 17, 121, 127, 236, 1, 186, 9, 121, 125, 191, 10, 121, 123, 194, 9, 121, 125, 235, 202, 12, 121, 129, 210, 9, 121, 127, 251, 1, 1, 1, 186, 22, 121, 127, 236, 202, 19, 121, 125, 206, 17, 121, 123, 210, 14, 121, 127, 249, 202, 11, 121, 123, 206, 11, 121, 123, 210, 11, 121, 123, 214, 11, 121, 123, 1, 186, 18, 121, 125, 190, 18, 121, 123, 194, 18, 121, 125, 235, 202, 11, 121, 127, 210, 11, 121, 127, 237, 202, 15, 121, 125, 206, 13, 121, 123, 210, 10, 121, 125, 237, 235, 202, 11, 121, 123, 206, 11, 121, 123, 210, 11, 121, 123, 214, 11, 121, 123, 1, 186, 18, 121, 125, 190, 18, 121, 123, 194, 18, 121, 127, 202, 15, 121, 125, 235, 210, 15, 121, 126, 239, 1, 186, 12, 121, 124, 191, 14, 121, 123, 194, 15, 121, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11]', 12, 13, 0.2651515151515145, 0.1481481481481481], [43, '[190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 135, 237, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 135, 237, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124]', 3, 3, 0.2147988505747128, 0.2807017543859649], [44, '[191, 14, 112, 127, 198, 14, 112, 127, 206, 14, 117, 123, 210, 14, 117, 123, 214, 19, 117, 123, 1, 186, 19, 117, 132, 202, 14, 120, 123, 206, 14, 115, 123, 210, 14, 117, 125, 235, 186, 14, 115, 124, 50, 114, 124, 1, 186, 43, 110, 134, 238, 194, 50, 110, 123, 198, 50, 121, 123, 202, 50, 121, 125, 208, 47, 121, 124, 214, 50, 114, 124, 1, 186, 43, 110, 129, 194, 47, 114, 125, 238, 235, 194, 50, 110, 123, 198, 50, 121, 123, 202, 50, 121, 125, 208, 47, 121, 124, 202, 49, 121, 129, 210, 52, 110, 131, 1, 186, 50, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 142, 59, 109, 127, 1, 190, 218, 42, 106, 124, 47, 110, 124, 51, 110]', 9, 45, 0.2067610062893081, 0.0212765957446808], [45, '[212, 19, 92, 122, 213, 19, 90, 124, 214, 23, 103, 123, 24, 108, 122, 215, 24, 90, 122, 216, 19, 100, 123, 26, 108, 124, 1, 186, 19, 91, 124, 24, 96, 123, 188, 23, 90, 123, 24, 90, 126, 189, 19, 114, 122, 190, 19, 103, 125, 191, 24, 121, 123, 192, 23, 119, 123, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121]', 9, 17, 0.1604166666666665, 0.0], [46, '[50, 114, 125, 192, 41, 110, 123, 44, 94, 123, 235, 198, 33, 92, 123, 45, 93, 123, 235, 204, 33, 93, 123, 45, 93, 123, 235, 210, 33, 92, 123, 45, 94, 123, 214, 58, 109, 126, 190, 53, 109, 123, 200, 58, 109, 126, 202, 53, 110, 123, 206, 58, 110, 126, 210, 53, 109, 123, 214, 58, 109, 126, 1, 186, 53, 108, 123, 190, 58, 107, 126, 192, 53, 111, 123, 235, 198, 53, 108, 123, 235, 204, 53, 107, 123, 235, 214, 53, 106, 123, 235, 188, 53, 105, 123, 235, 192, 53, 108, 123, 235, 194, 53, 108, 123, 235, 202, 26, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 14, 121, 123, 190, 19, 121, 123, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 18, 121, 123, 206, 18, 121, 123, 210, 18, 121, 127]', 14, 45, 0.1802884615384615, 0.0454545454545454], [47, '[45, 121, 123, 50, 121, 123, 57, 121, 123, 1, 186, 17, 105, 127, 194, 17, 105, 125, 235, 202, 24, 105, 123, 206, 26, 105, 123, 210, 24, 105, 123, 214, 21, 105, 123, 1, 186, 22, 105, 125, 235, 194, 22, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 22, 105, 123, 1, 186, 21, 105, 125, 235, 194, 21, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 16, 105, 125, 1, 186, 14, 105, 125, 235, 194, 14, 105, 125, 235, 202, 21, 105, 123, 206, 22, 105, 123, 210, 21, 105, 123, 214, 19, 105, 123, 1, 186, 17, 105, 125, 235, 194, 17, 105, 125, 235, 202, 24, 105, 123, 206, 26, 105, 123, 210, 24, 105, 123, 214, 21, 105, 123, 1, 186, 22, 105, 125, 235, 194, 22, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 22, 105, 123, 1, 186, 21, 105, 127, 194, 21, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 16, 105, 123, 1, 186, 14, 105, 125, 235, 194, 14, 105, 127, 202, 21, 105, 125, 206, 22, 105, 123, 210, 21, 105, 123, 214, 19, 105, 123, 1, 186, 17, 105, 127, 194, 17, 105, 125, 235, 202, 24, 105, 123, 206, 26, 105, 123, 210, 24, 105, 123, 214, 21, 105, 123, 1, 186]', 11, 43, 0.1886363636363636, 0.5660377358490566], [48, '[43, 105, 129, 1, 186, 5, 105, 124, 190, 5, 105, 123, 194, 5, 105, 127, 202, 5, 105, 129, 246, 235, 194, 16, 110, 123, 190, 21, 110, 123, 194, 26, 109, 123, 196, 21, 109, 123, 199, 26, 108, 123, 200, 21, 108, 123, 205, 26, 108, 123, 206, 21, 107, 123, 210, 26, 107, 123, 212, 21, 107, 123, 215, 26, 106, 123, 216, 21, 106, 123, 1, 189, 26, 105, 123, 189, 19, 105, 123, 192, 53, 101, 126, 198, 52, 98, 126, 200, 48, 95, 125, 55, 94, 126, 202, 53, 93, 126, 204, 52, 92, 126, 206, 48, 92, 125, 55, 91, 126, 208, 53, 91, 126, 208, 52, 91, 126, 210, 48, 91, 126, 235, 186, 53, 114, 125, 190, 52, 114, 125, 194, 50, 114, 145, 236, 194, 53, 114, 123, 198, 53, 114, 123, 202, 53, 114, 123, 206, 53, 114, 123, 210, 53, 114, 124, 214, 46, 121, 124, 50, 121, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 198, 43, 121, 123, 202, 52, 121, 124, 204, 50, 121, 123, 206, 48, 121, 126, 210, 52, 121, 129, 240, 235, 198, 43, 121, 123, 202, 52, 121, 124, 204, 50, 121, 123, 206, 48, 121, 125, 210, 52, 121, 124, 214, 52, 121, 124, 251, 235, 1, 1, 1]', 12, 50, 0.24113475177305, 0.0], [49, '[202, 52, 121, 124, 204, 50, 121, 123, 206, 48, 121, 125, 210, 52, 121, 124, 214, 52, 121, 124, 251, 235, 1, 1, 1, 1, 198, 43, 121, 123, 202, 52, 121, 124, 204, 50, 121, 123, 206, 48, 121, 125, 210, 52, 121, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 198, 55, 121, 124, 200, 57, 121, 124, 202, 58, 118, 126, 206, 57, 121, 125, 208, 55, 121, 123, 212, 58, 118, 126, 214, 53, 121, 127, 1, 186, 40, 121, 123, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 210, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26]', 10, 32, 0.2423469387755107, 0.0], [50, '[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 123, 206, 24, 121, 124, 210, 24, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121]', 9, 18, 0.1937134502923976, 0.125], [51, '[214, 14, 105, 124, 1, 186, 9, 105, 124, 190, 7, 105, 124, 194, 5, 105, 126, 202, 9, 105, 125, 206, 7, 105, 125, 210, 12, 105, 125, 214, 7, 105, 125, 1, 186, 5, 105, 127, 194, 5, 105, 124, 198, 5, 105, 123, 202, 5, 105, 125, 206, 7, 105, 125, 210, 9, 105, 127, 1, 186, 12, 105, 127, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 202, 26, 114, 124, 206, 17, 114, 124, 210, 24, 114, 124, 240, 192, 17, 114, 123, 194, 24, 104, 124, 198, 17, 121, 123, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194]', 11, 21, 0.2199074074074071, 0.2264150943396226], [52, '[192, 19, 121, 123, 19, 92, 122, 24, 110, 123, 48, 113, 123, 202, 52, 113, 124, 204, 53, 113, 123, 207, 47, 117, 124, 210, 48, 116, 124, 212, 48, 116, 123, 216, 52, 116, 124, 1, 186, 48, 116, 123, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127, 210, 16, 110, 131, 1, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 129, 235, 198, 16, 114, 124, 202, 16, 114, 127, 210, 16, 114, 129, 235, 190, 23, 121, 124, 194, 23, 114, 127, 202, 23, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127, 210, 16, 110, 131, 1, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 129, 235, 198, 16, 114, 124, 202, 16, 114, 127, 210, 16, 114, 129, 235, 190, 23, 121, 124, 194, 23, 114, 127, 202, 23, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127, 210, 16, 110, 131, 1, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 129, 235, 198, 16, 114, 124, 202, 16, 114, 127, 210, 16, 114, 129, 235, 190, 23, 121, 124, 194, 23, 114, 127, 202, 23, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127]', 9, 42, 0.3444940476190477, 0.0192307692307692], [53, '[194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124]', 4, 8, 0.3361581920903958, 0.0689655172413793], [54, '[1, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 48, 110, 123, 210, 36, 109, 123, 48, 109, 123, 214, 36, 108, 123, 48, 108, 123, 216, 36, 108, 123, 48, 108, 123, 1, 188, 36, 107, 123, 48, 107, 123, 190, 36, 108, 122]', 7, 19, 0.1800595238095237, 0.1], [55, '[190, 6, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 123, 202, 16, 110, 127, 210, 16, 110, 131, 1, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 127, 194, 16, 114, 127, 202, 16, 114, 127, 235, 214, 18, 114, 123, 216, 11, 114, 123, 1, 188, 18, 114, 123, 190, 11, 114, 123, 194, 18, 114, 123, 198, 11, 114, 123, 202, 18, 114, 127, 210, 11, 114, 127, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]', 5, 12, 0.3197463768115941, 0.0444444444444444], [56, '[210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 135, 237, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 135, 237, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7, 117, 124, 206, 5, 117, 124, 210, 4, 117, 124, 235, 1, 186, 5, 117, 124, 190, 7, 117, 124, 194, 7, 117, 124, 198, 7, 117, 124, 202, 7, 117, 124, 206, 7, 117, 124, 210, 7, 117, 124, 214, 7, 117, 124, 1, 186, 7, 117, 131, 235, 202, 7]', 3, 3, 0.223684210526316, 0.2857142857142857], [57, '[202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 10, 113, 124, 194, 10, 113, 124, 198, 10, 113, 124, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 10, 113, 124, 194, 10, 113, 124, 198, 10, 113, 124, 202, 5, 113, 124, 206, 10, 113, 124, 210, 10, 113, 124, 214, 10, 113, 124, 1, 186, 5, 113, 127, 194, 5, 113, 127, 202, 5, 113, 129, 236, 186, 14, 113, 129, 194, 14, 113, 127, 202, 14, 113, 134, 1, 186, 14, 113, 129, 194, 14, 113, 127, 202, 14, 113, 134, 1, 186, 14, 113, 125, 190, 12, 113, 125, 194, 14, 113, 127, 202, 14, 113, 127, 210, 14, 113, 125, 214, 12, 113, 125, 1, 186, 10, 113, 125, 190, 10, 109, 125, 194, 10, 113, 124, 198, 10, 113, 124, 202, 10, 113, 124, 206, 10, 113, 124, 210, 10, 113, 124, 214, 10, 113, 124, 1, 186, 5, 113, 127, 194, 5, 113, 127, 202, 5, 113, 129, 210, 9, 112, 129, 1, 186, 7, 113, 133, 235, 202, 7, 113, 127, 210, 7, 113, 123, 214, 12, 113, 125, 1, 186, 14, 113, 124, 190, 14, 113, 124, 194, 14, 113, 124, 198, 14, 113, 124, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 10, 113, 124, 194, 5, 113, 124, 198, 10, 113, 124, 202, 5, 113, 124, 206, 10, 113, 124, 210, 10, 113, 124, 214, 9, 113, 124, 1, 186, 5, 113, 127, 194]', 6, 9, 0.2994350282485877, 0.1206896551724138], [58, '[47, 110, 123, 235, 198, 47, 110, 123, 236, 208, 47, 110, 123, 236, 186, 47, 110, 123, 236, 197, 47, 110, 123, 236, 210, 47, 110, 123, 236, 189, 47, 110, 123, 235, 198, 47, 110, 123, 236, 235, 210, 47, 110, 123, 236, 189, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 192, 47, 110, 123, 235, 198, 47, 110, 123, 235, 205, 47, 110, 123, 210, 47, 110, 123, 235, 216, 47, 110, 123, 235, 190, 47, 110, 123, 235, 197, 47, 110, 123, 202, 47, 110, 123, 235, 208, 47, 110, 123, 235, 214, 47, 110, 123, 235, 189, 47, 110, 123, 194, 47, 110, 123, 235, 200, 47, 110, 123, 235, 206, 47, 110, 123, 235, 213, 47, 110, 123, 1, 186, 47, 110, 123, 235, 194, 47, 110, 124, 241, 235, 1, 186, 47, 110, 123, 235]', 1, 0, 0.1258169934640523, 0.0], [59, '[206, 22, 115, 123, 210, 20, 113, 124, 214, 18, 114, 124, 1, 186, 17, 116, 126, 194, 22, 116, 125, 235, 202, 17, 115, 124, 206, 15, 113, 123, 210, 13, 111, 124, 214, 15, 113, 124, 1, 186, 22, 115, 125, 235, 194, 22, 114, 125, 235, 202, 18, 116, 124, 206, 17, 114, 123, 210, 18, 114, 124, 214, 17, 114, 124, 1, 186, 15, 114, 126, 194, 15, 114, 126, 202, 20, 115, 124, 206, 22, 114, 124, 210, 20, 114, 125, 214, 18, 113, 123, 1, 186, 20, 114, 125, 235, 194, 17, 114, 125, 235, 202, 17, 117, 124, 206, 21, 115, 123, 210, 17, 115, 123, 214, 21, 116, 123, 1, 186, 21, 113, 123, 190, 16, 116, 123, 194, 9, 118, 126, 202, 9, 117, 129, 210, 12, 116, 126, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 186, 10, 110, 124, 190, 11, 110, 124, 194, 9, 121, 127, 202, 15, 121, 125, 235, 210, 15, 121, 126, 239, 1, 186, 12, 121, 124, 191, 14, 121, 123, 194, 15, 121, 124, 235, 202, 17, 121, 125, 235, 210, 12, 121, 125, 235, 186, 17, 121, 129, 235, 199, 17, 121, 123, 202, 17, 121, 129, 236, 186, 9, 121, 125, 191, 10, 121, 124, 194, 9, 121, 129, 202, 15, 121, 125, 235, 210, 15, 121, 126, 239, 1, 186, 12, 121, 124, 191, 14, 121, 123]', 13, 13, 0.226762820480769, 0.4509803921568627], [60, '[198, 9, 117, 123, 202, 10, 114, 129, 210, 12, 117, 129, 1, 186, 12, 116, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 129, 202, 17, 117, 123, 206, 21, 117, 123, 210, 17, 116, 123, 214, 21, 115, 123, 1, 186, 21, 111, 123, 190, 16, 116, 123, 194, 9, 114, 126, 202, 9, 117, 129, 210, 12, 115, 127, 1, 186, 14, 116, 127, 235, 198, 14, 116, 125, 202, 17, 117, 129, 210, 19, 115, 125, 235, 186, 21, 117, 125, 190, 22, 112, 123, 194, 19, 117, 124, 235, 202, 22, 117, 124, 206, 23, 116, 123, 210, 42, 121, 124, 202, 23, 105, 124, 206, 19, 105, 124, 210, 19, 105, 124, 214, 19, 105, 124, 1, 186, 14, 105, 129, 194, 14, 105, 129, 202, 14, 105, 129, 236, 186, 18, 105, 125, 190, 19, 105, 125, 194, 23, 105, 129, 202, 23, 105, 129, 210, 23, 105, 129, 1, 186, 23, 105, 131, 235, 202, 18, 105, 125]', 13, 33, 0.3221726190476191, 0.1923076923076923], [61, '[206, 50, 114, 124, 210, 53, 114, 124, 214, 45, 114, 124, 241, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 127, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26, 121, 124, 237, 202, 26, 114, 127, 235, 214, 26]', 4, 27, 0.270833333333333, 0.0], [62, '[206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 9, 114, 124, 194, 9, 114, 124, 198, 9, 114, 124, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 16, 114, 127, 1, 186, 9, 114, 135, 235, 206, 2, 114, 124, 210, 9, 114, 124, 214, 2, 114, 124, 1, 186, 9, 114, 127, 194, 9, 114, 127, 202, 14, 114, 127, 210, 12, 105, 125, 214, 7]', 8, 16, 0.3455459770114945, 0.1228070175438596], [63, '[46, 105, 123, 1, 186, 22, 105, 125, 235, 194, 22, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 22, 105, 123, 1, 186, 21, 105, 125, 235, 194, 21, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 16, 105, 125, 1, 186, 14, 105, 125, 235, 194, 14, 105, 125, 235, 202, 21, 105, 123, 206, 22, 105, 123, 210, 21, 105, 123, 214, 19, 105, 123, 1, 186, 17, 105, 125, 235, 194, 17, 105, 125, 235, 202, 24, 105, 123, 206, 26, 105, 123, 210, 24, 105, 123, 214, 21, 105, 123, 1, 186, 22, 105, 125, 235, 194, 22, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 22, 105, 123, 1, 186, 21, 105, 125, 235, 194, 21, 105, 125, 235, 202, 17, 105, 125, 206, 16, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129]', 12, 36, 0.240327380952381, 0.3653846153846153], [64, '[214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 236, 194, 16, 110, 129, 202, 16, 110, 133, 235, 186, 10, 110, 124, 190, 11, 110, 124, 194, 11, 110, 124, 198, 11, 110, 124, 202, 11, 110, 141, 235, 194, 16, 110, 129, 202, 16, 110, 133, 236, 190, 11, 110, 124, 188, 58, 106, 126, 190, 53, 106, 123, 194, 58, 105, 127, 245, 1, 186, 46, 105, 124, 190, 41, 105, 124, 194, 46, 105, 124, 198, 46, 105, 124, 202, 41, 105, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214]', 14, 48, 0.2789548022598873, 0.0], [65, '[50, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206]', 7, 21, 0.1847222222222223, 0.0847457627118644], [66, '[241, 202, 47, 119, 125, 206, 49, 113, 124, 207, 48, 117, 123, 208, 47, 117, 133, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 235, 194, 14, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117]', 7, 43, 0.3209064327485381, 0.1296296296296296], [67, '[202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24]', 6, 12, 0.1749999999999999, 0.0677966101694915], [68, '[202, 11, 114, 129, 210, 7, 114, 127, 1, 186, 12, 114, 131, 235, 202, 12, 114, 126, 208, 7, 114, 126, 214, 2, 114, 126, 1, 186, 7, 114, 125, 190, 7, 114, 125, 194, 7, 114, 125, 198, 7, 114, 124, 202, 11, 114, 129, 210, 7, 114, 129, 1, 186, 12, 114, 131, 235, 202, 12, 114, 126, 208, 7, 114, 126, 214, 2, 114, 126, 1, 186, 7, 114, 125, 190, 7, 114, 125, 194, 7, 114, 125, 198, 7, 114, 124, 202, 11, 114, 129, 210, 7, 114, 127, 1, 186, 12, 114, 131, 235, 202, 12, 114, 126, 208, 7, 114, 126, 214, 2, 114, 126, 1, 186, 7, 114, 125, 190, 7, 114, 125, 194, 7, 114, 125, 198, 7, 114, 124, 202, 11, 114, 129, 210, 7, 114, 129, 251, 235, 1, 1, 1, 186, 7, 113, 133, 235, 202, 7, 113, 127, 210, 7, 113, 123, 214, 12, 113, 125, 1, 186, 14, 113, 124, 194, 14, 113, 127, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 10, 113, 124, 194, 5, 113, 124, 198, 10, 113, 124, 202, 5, 113, 124, 206, 10, 113, 124, 210, 10, 113, 124, 214, 9, 113, 124, 1, 186, 5, 113, 127, 194, 5, 113, 127, 202, 5, 113, 131, 214, 2, 112, 125, 1, 186, 7, 113, 133, 235, 202, 7, 113, 127, 210, 7, 113, 123, 214, 12, 113, 125, 1, 186, 14, 113, 124, 190, 14, 113, 124, 194]', 8, 12, 0.3318713450292394, 0.0566037735849056], [69, '[50, 121, 124, 216, 43, 121, 123, 50, 121, 124, 212, 48, 114, 124, 215, 43, 121, 123, 50, 121, 124, 216, 48, 121, 124, 1, 188, 43, 121, 123, 50, 121, 124, 188, 48, 121, 124, 192, 43, 121, 123, 50, 121, 124, 194, 48, 121, 124, 198, 43, 121, 128, 50, 121, 127, 202, 43, 114, 127, 46, 114, 127, 50, 113, 127, 206, 43, 113, 127, 212, 19, 112, 123, 207, 19, 112, 123, 207, 15, 113, 123, 210, 18, 111, 124, 214, 17, 113, 124, 1, 186, 15, 114, 126, 194, 15, 113, 126, 202, 20, 116, 124, 206, 22, 114, 124, 210, 18, 115, 124, 214, 17, 114, 123, 1, 186, 15, 113, 126, 194, 15, 115, 125, 235, 202, 20, 114, 123, 206, 22, 114, 123, 210, 18, 114, 124, 214, 17, 113, 123, 1, 186, 15, 112, 126, 194, 15, 114, 125, 235, 202, 20, 114, 124, 206, 22, 114, 124, 210, 18, 114, 124, 214, 17, 113, 123, 1, 186, 10, 115, 127, 236, 202, 10, 116, 129, 210, 13, 116, 129, 1, 186, 15, 115, 127, 236, 202, 10, 116, 123, 206, 10, 113, 125, 210, 13, 117, 129, 1, 186, 10, 115, 125, 235, 196, 12, 116, 126, 204, 19, 117, 129, 236, 204, 15, 119, 125, 208, 14, 114, 123, 212, 15, 116, 123, 215, 14, 117, 123, 216, 14, 117, 123, 1, 188, 12, 117, 126, 196, 12, 116, 125, 235, 204, 19, 117, 129, 235, 216, 19]', 14, 40, 0.2254901960784313, 0.0], [70, '[1, 1, 1, 1, 188, 15, 118, 129, 235, 200, 15, 118, 124, 204, 22, 118, 125, 210, 22, 117, 124, 215, 20, 115, 124, 1, 188, 19, 117, 129, 236, 204, 15, 119, 125, 208, 14, 114, 123, 212, 15, 116, 123, 216, 14, 116, 123, 1, 188, 12, 115, 126, 196, 12, 114, 127, 204, 19, 119, 126, 212, 19, 116, 123, 216, 17, 116, 123, 1, 188, 15, 118, 129, 235, 200, 15, 118, 125, 204, 22, 117, 125, 210, 22, 115, 124, 215, 20, 117, 124, 1, 188, 19, 117, 129, 236, 204, 15, 118, 125, 207, 14, 110, 123, 212, 15, 114, 123, 216, 14, 117, 123, 1, 188, 12, 117, 126, 196, 12, 116, 126, 204, 19, 118, 129, 235, 216, 19, 116, 125, 1, 188, 12, 115, 124, 192, 12, 113, 123, 196, 12, 115, 126, 204, 19, 119, 126, 212, 19, 114, 123, 216, 38, 104, 125, 1, 186, 19, 103, 123, 188, 24, 100, 123, 189, 23, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202]', 12, 27, 0.2865497076023392, 0.2264150943396226], [71, '[47, 121, 125, 1, 186, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 198, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 202, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 204, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 210, 24, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 50, 121, 125, 202, 42, 121, 129, 54, 121, 129, 210, 38, 121, 124, 50, 121, 124, 235, 1, 186, 42, 121, 129, 54, 121, 129, 194, 38, 121, 123, 50, 121, 123, 235, 202, 42, 121, 129, 54, 121, 129, 210, 43, 114, 129, 55, 114, 129, 1, 186, 45, 114, 160, 57, 114, 160, 1, 202, 42, 114, 124, 54, 114, 124, 206, 45, 114, 124, 57, 114, 124, 237, 194, 38, 121, 123, 50, 121, 123, 198, 38, 121, 125, 50, 121, 125, 202, 42, 121, 129, 54, 121, 129, 210, 38, 121, 124, 50, 121, 124, 235, 1, 186, 42, 121, 129, 54, 121, 129, 194, 38, 121, 123, 50, 121, 123, 235, 202, 42, 121, 129, 54, 121, 129, 210, 43, 114, 129, 55, 114, 129, 1, 186, 45, 114, 160, 57, 114, 160, 1, 202, 42, 114, 124, 54, 114, 124, 206, 45, 114, 123, 52, 114, 123, 210, 53, 114, 124, 236, 235, 194, 15, 114, 126, 202, 18, 114, 124, 206, 17, 113, 124, 210, 18, 114, 124, 214]', 19, 42, 0.3923611111111111, 0.3], [72, '[198, 18, 118, 124, 202, 17, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 16, 105, 123, 1, 186, 14, 105, 125, 235, 194, 14, 105, 125, 235, 202, 21, 105, 125, 206, 22, 105, 123, 210, 21, 105, 123, 214, 19, 105, 123, 245, 235, 206, 16, 105, 123, 210, 17, 105, 125, 214, 16, 105, 123, 1, 186, 14, 105, 125, 235, 194, 14, 105, 125, 235, 202, 21, 105, 123, 206, 22, 105, 123, 210, 21, 105, 123, 214, 19, 105, 123, 1, 186, 17, 105, 127, 194, 17, 105, 125, 235, 202, 24, 105, 123, 206, 26, 105, 123, 210, 24, 105, 123, 214, 21, 105, 123, 1, 186, 22, 105, 127, 194, 22, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 22, 105, 123, 1, 186, 21, 105, 127, 194, 21, 116, 124, 206, 22, 114, 124, 210, 18, 115, 124, 214, 17, 114, 123, 1, 186, 15, 113, 126, 194, 15, 115, 125, 235, 202, 20, 114, 123, 206, 22, 114, 123, 210, 18, 114, 124, 214, 17, 113, 123, 1, 186, 15, 112, 126, 194, 15, 114, 124, 206, 15, 114, 123, 210, 18, 113, 123, 214, 17, 113, 123, 1, 186, 15, 108, 126, 208, 53, 107, 126, 210, 52, 107, 126, 212, 52, 106, 126, 214, 48, 105, 125, 55, 105, 126, 216, 53, 104, 126, 1, 186, 52, 104, 126, 188, 48, 103, 125]', 15, 41, 0.2023809523809523, 0.0], [73, '[190, 52, 121, 123, 194, 48, 121, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 186, 47, 121, 124, 188, 48, 114, 124, 191, 47, 114, 124, 237, 235, 214, 47, 121, 125, 1, 186, 48, 114, 124, 190, 48, 121, 123, 194, 48, 121, 124, 251, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 186, 47, 121, 124, 188, 48, 114, 124, 191, 47, 114, 124, 237, 235, 190, 47, 114, 123, 194, 47, 121, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 24, 114, 123, 189, 19, 114, 123, 191, 24, 121, 123, 194, 19, 121, 123, 237, 235, 216, 19, 114, 123, 1, 186, 23, 121, 123, 189, 19, 114, 123, 191, 24, 121, 123, 194, 26, 121, 124, 237, 235, 1, 186, 24, 121, 123, 189, 19, 114, 123, 191, 24, 114, 123, 194, 19, 114, 123, 241, 215, 19, 114, 124, 251, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 214, 26, 114, 169, 1, 1, 214]', 10, 41, 0.3069105691056909, 0.131578947368421], [74, '[44, 114, 124, 49, 110, 124, 199, 218, 41, 114, 123, 44, 120, 123, 49, 108, 123, 236, 215, 218, 41, 110, 124, 44, 116, 124, 49, 110, 124, 1, 194, 41, 112, 123, 45, 114, 123, 48, 114, 123, 53, 115, 123, 57, 115, 123, 60, 114, 123, 188, 41, 112, 123, 45, 113, 123, 48, 113, 123, 53, 113, 123, 57, 114, 123, 60, 103, 123, 194, 41, 103, 123, 45, 113, 123, 48, 113, 123, 53, 113, 123, 57, 114, 123, 60, 113, 123, 236, 210, 41, 113, 123, 214, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 210, 21, 121, 123, 214, 21, 121, 123, 1, 186, 12, 121, 125, 190, 7, 121, 123, 238, 235, 1, 186, 17, 121, 125, 190, 17, 121, 127, 194, 17, 121, 127, 202, 17, 121, 129, 236, 186, 24, 121, 129, 235, 198, 26, 121, 123, 202, 28, 121, 123, 206, 26, 121, 123, 210, 24, 121, 125, 235, 186, 17, 121, 127, 194, 17, 121, 127, 202, 17, 121, 129, 236, 186, 24, 121, 129, 235, 198, 26, 121, 123, 202, 28, 121, 123, 206, 26, 121, 123, 210, 24, 121, 125, 235, 186, 11, 121, 129, 194, 11, 121, 127, 202, 11, 121, 129, 237, 235, 198, 16, 121, 124, 202, 18, 121, 124, 206, 16, 121, 124, 210, 18, 121, 126, 1, 186, 11, 121, 127, 194, 11, 121, 127, 202]', 18, 53, 0.2654761904761907, 0.3793103448275862], [75, '[1, 1, 194, 43, 112, 123, 198, 43, 118, 123, 202, 50, 112, 125, 206, 43, 118, 124, 210, 50, 118, 129, 1, 186, 52, 112, 126, 239, 194, 43, 112, 123, 198, 43, 118, 123, 202, 50, 112, 125, 206, 43, 118, 124, 210, 50, 118, 129, 1, 186, 52, 112, 126, 247, 194, 43, 112, 123, 198, 43, 118, 123, 202, 50, 112, 125, 206, 43, 118, 124, 210, 19, 116, 129, 1, 186, 11, 114, 123, 214, 16, 115, 123, 1, 186, 21, 120, 125, 235, 194, 21, 114, 125, 235, 202, 21, 116, 123, 206, 19, 118, 123, 210, 21, 117, 123, 235, 1, 186, 23, 119, 127, 194, 19, 117, 125, 235, 202, 16, 117, 127, 235, 214, 19, 117, 123, 1, 186, 23, 116, 123, 190, 23, 114, 123, 194, 19, 117, 125, 235, 202, 16, 115, 127, 235, 214, 19, 114, 123, 1, 186, 23, 114, 123, 190, 19, 109, 123, 194, 19, 112, 123, 198, 19, 112, 123, 202, 14, 116, 126, 210, 16, 116, 123, 214, 17, 115, 123, 1, 186, 18, 115, 125, 235, 194, 18, 115, 125, 235, 202, 13, 117, 129, 236, 186, 18, 117, 124, 190, 13, 115, 123, 194, 18, 114, 125, 235, 202, 13, 117, 126, 210, 13, 114, 126, 1, 186, 19, 114, 127, 194, 19, 118, 125, 235, 202, 16, 116, 127, 235, 214, 19, 116, 123, 243, 1, 188, 12, 117, 127, 196, 12, 117, 127, 204, 19, 118, 129, 236, 204]', 13, 41, 0.2378787878787878, 0.1481481481481481], [76, '[194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 29, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 90, 123, 45, 90, 123, 48, 90, 123, 53, 90, 123, 57, 90, 123, 60, 90, 123, 1]', 11, 31, 0.1959876543209876, 0.0784313725490196], [77, '[196, 49, 115, 123, 52, 111, 123, 235, 198, 49, 113, 123, 52, 111, 123, 235, 204, 49, 111, 123, 52, 108, 123, 235, 210, 49, 119, 123, 52, 92, 123, 212, 49, 91, 123, 52, 91, 123, 235, 214, 49, 90, 123, 52, 90, 123, 235, 188, 49, 90, 123, 52, 90, 123, 236, 1, 186, 10, 114, 124, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 129, 235, 198, 16, 114, 124, 202, 16, 114, 127, 210, 16, 114, 129, 235, 190, 23, 121, 124, 194, 23, 114, 127, 202, 23, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127, 210, 16, 110, 131, 1, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 127, 235, 214, 18, 114, 123, 216, 11, 114, 123, 1, 188, 18, 114, 123, 190, 11, 114, 123, 194, 18, 114, 123, 198, 11, 114, 123, 202, 18, 114, 127, 210, 11, 114, 127, 1, 186, 16, 114, 127, 194, 16, 114, 127, 202, 16, 114, 127, 235, 214, 18, 114, 123, 216, 11, 114, 123, 1, 188, 18, 114, 123, 190, 11, 114, 123, 194, 18, 114, 123, 198, 11, 114, 123, 202, 18, 114, 127, 210, 11, 114, 127, 1, 186, 16, 114, 127, 194, 16, 114, 127, 202, 16, 114, 127, 235, 214, 18, 114, 123, 216, 11, 114, 123, 1, 188, 18, 114, 123, 190, 11, 114, 123]', 7, 42, 0.2588141025641025, 0.0909090909090909], [78, '[214, 18, 112, 124, 1, 186, 14, 112, 124, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 124, 206, 7, 112, 124, 210, 12, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 190, 14, 112, 127, 198, 14, 112, 127, 206, 15, 115, 123, 210, 18, 113, 123, 214, 18, 112, 123, 1, 186, 20, 117, 125, 235, 194, 20, 118, 125, 206, 13, 113, 125, 210, 13, 117, 127, 1, 186, 15, 117, 129, 236, 202, 10, 115, 123]', 9, 13, 0.2831920903954804, 0.1724137931034483], [79, '[214, 22, 112, 124, 1, 186, 7, 112, 124, 190, 6, 112, 124, 194, 14, 112, 124, 198, 14, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 124, 206, 7, 112, 124, 210, 12, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 190, 14, 112, 127, 198, 14, 112, 127, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 7, 112, 124, 198, 12, 112, 124, 202, 7, 112, 124, 206, 12, 112, 124, 210, 7, 112, 124, 214, 12, 112, 124, 1, 186, 7, 112, 124, 190, 6, 112, 124, 194, 14, 112, 124, 198, 14, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 132, 1, 194, 12, 112, 124, 198, 12, 112, 124, 202, 12, 112, 124, 206, 7, 112, 124, 210, 12, 112, 124, 214, 7, 112, 124, 1, 186, 12, 112, 124, 190, 19, 92, 126, 191, 24, 90, 126, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 186, 24, 121, 123, 189, 19, 114, 123, 191, 24, 114, 123, 194, 19, 114, 123, 241, 215, 19, 114, 124, 1, 186, 24, 114, 123, 189, 19, 114, 123, 191, 24, 121, 123, 194, 19]', 7, 18, 0.2193877550680268, 0.1063829787234042], [80, '[52, 121, 124, 251, 235, 1, 1, 1, 190, 49, 114, 123, 52, 121, 123, 239, 235, 198, 55, 121, 124, 200, 57, 121, 124, 202, 58, 118, 126, 206, 57, 121, 125, 210, 55, 121, 127, 1, 186, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 189, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 1, 186, 15, 117, 126, 194, 15, 114, 125, 235, 202, 20, 114, 124, 206, 22, 114, 124, 210, 18, 114, 124, 214, 17, 114, 124, 1, 186, 11, 115, 125, 235, 194, 11, 114, 123, 236, 206, 15, 115, 123, 210, 18, 113, 123, 214, 18, 112, 123, 1, 186, 20, 114, 125, 235, 194, 20, 114, 123, 236, 206, 18, 115, 123, 210, 15, 113, 123, 214, 18, 114, 123, 1, 186, 20, 115, 125, 235, 194, 20, 115, 123, 236, 206, 15, 111, 123, 210, 20, 116, 123, 214, 21, 118, 123, 1, 186, 22, 114, 125, 235, 194, 22, 117, 123, 202, 10, 121, 125, 206, 50, 107, 127, 192, 48, 106, 126, 52, 106, 127, 214, 48, 102, 123, 52, 101, 126, 1, 186, 48, 105, 125, 235, 202, 19, 116, 125, 235, 202, 16, 116, 127, 235, 214, 19, 116, 123, 1, 186, 23, 116, 123, 190, 23, 113, 123, 194, 19, 118, 125, 235, 202, 16, 116, 125, 235, 210, 16, 115, 123, 214, 14, 115, 123, 1, 186, 12, 117, 127, 194, 12, 116, 127, 1, 186, 16, 114, 127, 194, 14]', 21, 48, 0.2083333333333333, 0.0], [81, '[50, 102, 124, 197, 41, 99, 123, 44, 100, 123, 198, 36, 97, 123, 48, 97, 123, 200, 33, 97, 123, 45, 97, 123, 204, 36, 96, 123, 48, 97, 123, 206, 33, 97, 123, 45, 96, 123, 208, 36, 95, 123, 48, 95, 123, 210, 218, 36, 94, 123, 45, 120, 123, 48, 94, 123, 214, 36, 94, 123, 48, 94, 123, 216, 36, 94, 123, 48, 94, 123, 1, 188, 36, 94, 123, 48, 94, 123, 190, 52, 94, 123, 194, 48, 93, 123, 198, 53, 92, 123, 202, 48, 92, 123, 55, 92, 123, 204, 48, 92, 123, 208, 52, 91, 185, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166, 1, 1, 192, 43, 113, 127, 198, 43, 113, 166]', 11, 22, 1.323643410852713, 0.032258064516129], [82, '[198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 18, 112, 124, 198, 14, 112, 124, 202, 18, 112, 124, 206, 14, 112, 124, 210, 14, 112, 124, 214, 14, 112, 127, 1, 190, 12, 112, 127, 198, 12, 112, 127, 206, 19, 112, 127, 214, 12, 112, 132, 1, 194, 14, 112, 124, 198, 18, 112, 124, 202, 14, 112, 124, 206, 14, 112, 127, 214, 14, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 210, 13, 117, 131, 1, 190, 14, 117, 124, 194, 14, 117, 124, 198, 14, 117, 124, 202, 9, 117, 131, 214, 6, 117, 124, 1, 186, 9, 117, 129, 235, 198, 9, 117, 125, 202, 13, 117, 124, 206, 13, 117, 124, 188, 58, 119, 123, 48, 95, 123, 194, 36, 94, 123, 48, 94, 123, 196, 36, 94, 123, 48, 94, 123]', 10, 52, 0.2887426900584796, 0.1372549019607843], [83, '[194, 18, 110, 124, 198, 11, 110, 124, 202, 18, 110, 124, 206, 11, 110, 124, 210, 18, 110, 124, 214, 11, 110, 125, 1, 186, 7, 110, 124, 190, 6, 110, 124, 194, 4, 110, 129, 202, 4, 110, 133, 251, 1, 186, 18, 114, 123, 190, 18, 109, 123, 194, 18, 115, 125, 235, 202, 18, 116, 125, 235, 210, 11, 114, 123, 214, 12, 115, 123, 1, 186, 13, 115, 126, 194, 19, 116, 125, 235, 202, 18, 115, 126, 210, 13, 114, 123, 214, 18, 114, 123, 1, 186, 11, 114, 126, 194, 11, 116, 126, 237, 1, 186, 19, 114, 123, 190, 19, 109, 123, 194, 19, 111, 123, 198, 19, 111, 123, 202, 14, 116, 129, 235, 214, 14, 114, 123, 1, 186, 19, 114, 123, 190, 19, 111, 123, 194, 19, 112, 123, 38, 108, 123, 41, 108, 123, 45, 108, 123, 190, 230, 23, 104, 123, 26, 108, 123, 29, 110, 123, 33, 108, 123, 194, 17, 119, 129, 202, 18, 121, 124, 206, 16, 121, 124, 210, 18, 121, 126, 1, 186, 11, 121, 127, 194, 11, 121, 127, 202, 11, 121, 129, 238, 202, 18, 121, 124, 206, 18, 121, 124, 210, 18, 121, 125, 235, 186, 16, 121, 129, 194, 16, 121, 129, 202, 16, 121, 129, 237, 235, 198, 8, 121, 124, 202, 11, 121, 124, 206, 9, 121, 124, 210, 11, 121, 125, 235, 186, 16, 121, 127, 194, 16, 121, 127, 202, 16, 121, 129, 238]', 20, 41, 0.2596153846153847, 0.2448979591836734], [84, '[17, 119, 129, 23, 121, 125, 190, 23, 104, 125, 194, 17, 119, 129, 23, 121, 123, 196, 23, 104, 127, 198, 9, 112, 124, 21, 121, 137, 27, 104, 131, 202, 7, 112, 124, 17, 119, 129, 23, 121, 125, 206, 9, 112, 124, 23, 104, 125, 31, 108, 129, 34, 108, 125, 38, 108, 125, 208, 27, 104, 127, 210, 10, 112, 133, 17, 119, 129, 23, 121, 123, 212, 23, 104, 125, 214, 21, 121, 137, 27, 104, 137, 216, 23, 104, 123, 1, 186, 21, 119, 127, 194, 21, 118, 125, 210, 47, 116, 123, 235, 1, 186, 52, 118, 123, 188, 54, 113, 123, 190, 52, 114, 123, 191, 51, 113, 123, 194, 52, 116, 123, 237, 235, 1, 186, 47, 119, 123, 190, 47, 118, 123, 194, 47, 117, 123, 198, 47, 118, 123, 202, 47, 118, 123, 206, 47, 118, 123, 210, 47, 116, 123, 214, 47, 116, 123, 1, 186, 49, 118, 125, 190, 47, 114, 125, 194, 49, 117, 123, 238, 235, 194, 51, 119, 129, 202, 47, 115, 123, 235, 210, 51, 118, 123, 235, 1, 186, 52, 118, 123, 188, 54, 114, 123, 190, 52, 114, 123, 191, 51, 114, 123, 194, 52, 117, 123, 237, 235, 1, 186, 47, 120, 123, 190, 47, 117, 123, 194, 47, 120, 123, 198, 47, 117, 123, 202, 47, 120, 123, 206, 47, 120, 123, 210, 47, 116, 123, 214, 47, 116, 123, 1, 186, 49, 118, 125, 190, 47, 114, 125]', 15, 47, 0.2457627118644068, 0.0], [85, '[202, 11, 114, 129, 210, 7, 114, 127, 1, 186, 12, 114, 131, 235, 202, 12, 114, 126, 208, 7, 114, 126, 214, 2, 114, 126, 1, 186, 7, 114, 125, 190, 7, 114, 125, 194, 7, 114, 125, 198, 7, 114, 124, 202, 11, 114, 129, 210, 7, 114, 129, 1, 186, 12, 114, 131, 235, 202, 12, 114, 126, 208, 7, 114, 126, 214, 2, 114, 126, 1, 186, 7, 114, 125, 190, 7, 114, 125, 194, 14, 105, 125, 235, 202, 21, 105, 123, 206, 22, 105, 123, 210, 21, 105, 123, 214, 19, 105, 123, 1, 186, 17, 105, 125, 235, 194, 17, 105, 125, 235, 202, 24, 105, 125, 206, 26, 105, 123, 210, 24, 105, 123, 214, 21, 105, 123, 1, 186, 22, 105, 127, 194, 22, 105, 127, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 22, 105, 123, 1, 186, 21, 105, 125, 235, 194, 21, 105, 125, 235, 202, 17, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 16, 105, 123, 1, 186, 14, 105, 125, 235, 194, 14, 105, 125, 235, 202, 21, 105, 125, 206, 16, 105, 123, 210, 17, 105, 123, 214, 16, 105, 123, 1, 186, 14, 105, 125, 245, 235, 186, 12, 112, 124, 202, 7, 112, 124, 206, 12, 112, 124, 210, 7, 112, 124, 214, 12, 112, 124, 1, 186, 7, 112, 124, 190, 6, 112, 124, 194, 14, 112, 124, 198, 14, 112, 124, 202]', 13, 24, 0.2419590643274856, 0.3518518518518518], [86, '[206, 22, 105, 124, 198, 22, 105, 124, 202, 19, 105, 127, 210, 16, 105, 129, 235, 190, 14, 105, 125, 194, 19, 105, 123, 198, 19, 105, 123, 202, 19, 105, 123, 206, 19, 105, 123, 210, 19, 105, 123, 214, 19, 105, 124, 1, 186, 14, 105, 127, 194, 14, 105, 126, 202, 14, 105, 131, 235, 1, 186, 16, 105, 127, 194, 16, 105, 127, 202, 16, 105, 129, 235, 214, 19, 105, 124, 1, 186, 23, 105, 124, 192, 23, 105, 124, 198, 23, 105, 124, 202, 19, 105, 127, 210, 16, 105, 129, 235, 190, 14, 105, 125, 194, 19, 105, 124, 198, 19, 105, 124, 202, 19, 105, 124, 206, 19, 105, 124, 210, 19, 105, 124, 214, 19, 105, 124, 1, 186, 14, 105, 129, 194, 14, 105, 129, 202, 14, 105, 129, 236, 186, 18, 105, 125, 190, 19, 105, 125, 194, 23, 105, 129, 202, 23, 105, 129, 210, 23, 105, 129, 1, 186, 23, 105, 131, 235, 202, 18, 105, 125, 206, 23, 105, 125, 236, 235, 190, 19, 105, 125, 194, 24, 105, 124, 198, 24, 105, 124, 202, 24, 105, 124, 206, 24, 105, 124, 212, 15, 119, 125, 216, 219, 55, 103, 123, 59, 107, 123, 60, 96, 127, 62, 102, 125, 1, 188, 62, 103, 125, 189, 218, 55, 102, 123, 59, 106, 123, 60, 96, 126, 194, 219, 55, 106, 123, 59, 107, 123, 60, 98, 126, 62, 107, 125, 199, 219, 55, 105, 123]', 12, 48, 0.2738095237797618, 0.0], [87, '[214, 14, 114, 124, 202, 16, 114, 127, 210, 16, 114, 129, 235, 190, 23, 121, 124, 194, 23, 114, 127, 202, 23, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127, 210, 16, 110, 131, 1, 190, 11, 114, 124, 194, 11, 114, 127, 202, 11, 121, 129, 236, 186, 16, 114, 129, 235, 198, 16, 114, 124, 202, 16, 114, 127, 210, 16, 114, 129, 235, 190, 23, 121, 124, 194, 23, 114, 127, 202, 23, 114, 129, 236, 186, 16, 114, 129, 197, 16, 110, 123, 202, 16, 110, 127, 210, 18, 110, 127, 1, 186, 12, 110, 129, 194, 17, 110, 124, 198, 17, 110, 123, 202, 17, 110, 123, 206, 17, 110, 123, 210, 17, 110, 123, 214, 17, 110, 125, 1, 186, 15, 110, 127, 194, 15, 110, 129, 202, 10, 110, 129, 210, 15, 110, 129, 1, 186, 12, 110, 129, 194, 17, 110, 124, 198, 17, 110, 123, 202, 17, 110, 123, 206, 17, 110, 123, 210, 17, 110, 123, 214, 17, 110, 123, 1, 186, 17, 110, 123, 190, 16, 110, 123, 194, 14, 110, 125, 235, 202, 12, 110, 126, 210, 12, 110, 126, 1, 1, 1, 1, 1, 1, 1, 1, 186, 53, 121, 125, 190, 50, 121, 123, 235, 198, 50, 121, 123, 236, 210, 50, 121, 123, 214, 50, 121, 123, 1, 186, 53, 121, 125, 190, 50, 121, 123, 235, 198, 50, 121, 123, 236, 210, 50, 121, 123, 214]', 11, 43, 0.2827380952380952, 0.0925925925925925], [88, '[214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 126, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 15, 114, 123, 206, 18, 114, 124, 210, 18, 114, 129, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214, 11, 114, 124, 1, 186, 11, 114, 135, 202, 11, 114, 123, 206, 11, 114, 123, 210, 11, 114, 123, 214]', 3, 7, 0.3347457627118644, 0.0], [89, '[49, 119, 124, 205, 218, 42, 117, 123, 47, 117, 123, 51, 117, 123, 210, 218, 42, 113, 124, 47, 119, 124, 51, 119, 124, 216, 218, 41, 110, 124, 44, 116, 124, 49, 110, 124, 236, 194, 218, 41, 114, 124, 44, 121, 124, 49, 114, 123, 236, 235, 210, 218, 41, 121, 124, 44, 114, 124, 49, 121, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 114, 124, 236, 235, 210, 218, 41, 114, 124, 44, 121, 124, 49, 114, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 114, 124, 236, 235, 210, 218, 41, 121, 124, 44, 121, 124, 49, 110, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 114, 124, 236, 235, 210, 218, 41, 114, 124, 44, 121, 124, 49, 114, 124, 236, 235, 196, 218, 41, 113, 124, 44, 113, 124, 49, 120, 123, 236, 215, 218, 41, 110, 124, 44, 116, 124, 49, 116, 124, 236, 194, 218, 41, 109, 124, 44, 114, 124, 49, 109, 124, 236, 235, 194, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 236, 235, 210, 218, 41, 114, 124, 44, 121, 124, 49, 121, 124, 236, 235, 194, 218, 41, 121, 124, 44, 121, 124, 49, 121, 124, 236, 235, 210, 218, 42, 121, 124, 47, 121, 124, 51, 121, 123, 236, 235, 194, 218, 42, 121, 124, 47, 121, 124, 238, 235, 1, 186, 47, 121, 124, 188, 48, 121, 123, 190]', 7, 10, 0.171875, 0.25], [90, '[190, 14, 121, 124, 194, 14, 121, 127, 202, 14, 121, 129, 236, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 129, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 123, 190, 14, 121, 124, 194, 14, 121, 127, 202, 14, 121, 129, 236, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 127, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 123, 190, 14, 121, 124, 194, 14, 121, 127, 202, 9, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 12, 121, 124, 202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 117, 127, 202, 9, 117, 127, 210, 14, 117, 127, 1, 186, 9, 117, 129, 235, 198, 9, 117]', 5, 7, 0.300287356321839, 0.2280701754385964], [91, '[198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 1, 186, 17, 121, 129, 236, 202, 24, 121, 123, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 127, 210, 36, 121, 126, 1, 186, 24, 121, 124, 190, 24, 121, 123, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 124, 214, 31, 121, 124, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123, 214, 31, 121, 123, 1, 186, 29, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127]', 10, 19, 0.1993055555555556, 0.1206896551724138], [92, '[50, 110, 123, 213, 48, 114, 123, 55, 114, 123, 216, 47, 113, 123, 50, 107, 123, 210, 47, 106, 123, 50, 106, 123, 214, 47, 106, 123, 50, 106, 123, 216, 47, 114, 123, 50, 114, 123, 1, 188, 49, 102, 123, 52, 99, 123, 190, 219, 45, 108, 123, 49, 114, 127, 50, 108, 123, 52, 108, 127, 235, 216, 218, 41, 102, 124, 44, 104, 124, 49, 102, 124, 1, 190, 42, 101, 124, 194, 218, 41, 99, 124, 44, 101, 124, 49, 99, 124, 236, 206, 218, 41, 94, 124, 44, 96, 124, 49, 94, 124, 235, 216, 218, 41, 94, 124, 44, 95, 124, 49, 95, 124, 1, 186, 42, 99, 124, 47, 96, 124, 51, 96, 124, 196, 218, 41, 92, 124, 44, 92, 124, 49, 92, 124, 236, 210, 218, 41, 121, 124, 44, 114, 124, 49, 121, 124, 215, 218, 41, 119, 124, 44, 119, 124, 216, 218, 41, 113, 124, 44, 119, 124, 49, 113, 124, 1, 190, 218, 41, 110, 124, 44, 116, 124, 49, 110, 124, 194, 218, 41, 112, 123, 44, 107, 123, 235, 205, 43, 104, 123, 206, 43, 103, 123, 235, 212, 43, 101, 123, 235, 189, 43, 101, 123, 191, 43, 100, 123, 235, 197, 43, 99, 123, 235, 204, 43, 98, 123, 208, 43, 97, 123, 235, 214, 43, 96, 123, 235, 216, 43, 95, 123, 235, 189, 43, 97, 123, 191, 43, 97, 123, 235, 196, 43, 94, 123, 200, 50, 94, 123]', 12, 14, 0.148989898989899, 0.0], [93, '[198, 10, 113, 124, 202, 14, 113, 127, 210, 14, 113, 129, 235, 190, 33, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 123, 198, 36, 121, 125, 202, 33, 121, 124, 206, 36, 121, 124, 210, 36, 121, 127, 1, 186, 24, 121, 123, 190, 24, 121, 124, 194, 24, 121, 124, 198, 24, 121, 124, 202, 28, 121, 123, 206, 24, 121, 124, 210, 28, 121, 123]', 10, 28, 0.1916666666666667, 0.0677966101694915], [94, '[202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 13, 121, 125, 235, 210, 13, 121, 125, 235, 186, 13, 121, 123, 190, 16, 121, 123, 194, 18, 121, 125, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 21, 121, 123, 190, 18, 121, 123, 194, 13, 121, 125, 235, 202, 18, 121, 123, 206, 18, 121, 123, 210, 18, 121, 127, 236, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123, 214, 19, 121, 123, 1, 186, 19, 121, 125, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 235, 186, 14, 121, 123, 190, 19, 121, 123, 194, 19, 121, 123, 198, 19, 121, 123, 202, 19, 121, 123, 206, 19, 121, 123, 210, 19, 121, 123]', 6, 8, 0.1852678571428571, 0.0727272727272727], [95, '[202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 12, 121, 124, 202, 14, 121, 125, 207, 14, 121, 124, 210, 14, 121, 125, 235, 186, 12, 121, 124, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 9, 121, 127, 243, 1, 186, 12, 121, 125, 190, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 129, 210, 7, 121, 129, 235, 191, 12, 121, 124, 194, 14, 121, 124, 198, 12, 121, 124, 202, 14, 121, 129, 236, 186, 7, 121, 127, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 124, 210, 10, 121, 124, 214, 7, 121, 125, 1, 186, 14, 121, 126, 194, 14, 121, 127, 202, 14, 121, 129, 235, 214, 10, 121, 125, 1, 186, 12, 121, 125, 190, 7, 121, 124, 194, 7, 121, 127, 202, 7, 121, 125, 206, 9, 121, 125, 210, 10, 121, 127, 237, 202, 14, 121, 125, 206, 12, 121, 123, 210, 9, 121, 123, 214, 12, 121, 123, 1, 186, 14, 121, 123, 190, 14, 121, 123, 194, 14, 121, 125, 235, 202, 14, 121, 125, 235, 210, 14, 121, 125, 237, 235, 202, 18, 121, 125, 235, 210, 18, 121, 125, 235, 186, 17, 121]', 6, 11, 0.2745535714285715, 0.2909090909090909], [96, '[50, 121, 123, 236, 235, 194, 33, 121, 124, 1, 190, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 210, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 214, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 216, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 216, 218, 43, 121, 123, 48, 121, 123, 52, 121, 123, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 215, 23, 121, 124, 1, 186, 24, 121, 129, 236, 202, 28, 121, 123, 206, 26, 121, 123, 210, 24, 121, 125, 235, 186, 17, 121, 127, 194, 17, 121, 127, 202, 17, 121, 129, 237, 1, 1, 1, 1, 1, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202]', 12, 35, 0.2268518518518528, 0.2580645161290322], [97, '[40, 121, 123, 206, 36, 121, 124, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124, 198, 34, 121, 124, 202, 31, 121, 129, 236, 186, 34, 121, 124, 190, 36, 121, 124, 194, 36, 121, 124, 198, 36, 121, 124, 202, 36, 121, 124, 206, 36, 121, 123, 210, 36, 121, 124, 236, 190, 33, 121, 123, 194, 29, 121, 123, 198, 33, 121, 123, 202, 29, 121, 129, 236, 186, 36, 121, 124, 190, 38, 121, 124, 194, 38, 121, 124, 198, 38, 121, 123, 202, 38, 121, 124, 206, 38, 121, 124, 210, 38, 121, 124, 236, 190, 34, 121, 123, 194, 31, 121, 124]', 7, 11, 0.1819444444444446, 0.0847457627118644], [98, '[50, 121, 124, 236, 235, 210, 43, 114, 124, 48, 121, 124, 236, 235, 210, 43, 114, 124, 48, 121, 124, 248, 235, 194, 43, 121, 124, 48, 121, 124, 200, 43, 114, 124, 48, 121, 124, 236, 235, 210, 43, 114, 124, 48, 121, 124, 236, 235, 194, 43, 114, 123, 50, 121, 123, 236, 235, 210, 43, 121, 124, 50, 121, 124, 236, 235, 194, 43, 114, 124, 48, 121, 124, 236, 204, 43, 121, 123, 50, 121, 124, 208, 48, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206, 24, 121, 124, 210, 24, 121, 129, 236, 194, 18, 121, 124, 198, 19, 121, 123, 202, 19, 121, 124, 206, 19, 121, 124, 210, 19, 121, 126, 236, 194, 23, 120, 124, 198, 24, 121, 124, 202, 24, 121, 124, 206]', 7, 32, 0.2009803921568632, 0.2], [99, '[194, 14, 105, 124, 198, 14, 105, 124, 202, 11, 105, 124, 206, 11, 105, 124, 210, 14, 105, 124, 214, 11, 105, 124, 1, 186, 7, 105, 127, 194, 7, 105, 127, 202, 7, 105, 127, 236, 235, 190, 14, 105, 124, 194, 11, 105, 124, 198, 14, 105, 124, 202, 11, 105, 124, 206, 11, 105, 124, 210, 14, 105, 124, 214, 11, 105, 124, 1, 186, 7, 105, 127, 194, 7, 105, 127, 202, 7, 105, 127, 238, 202, 11, 105, 124, 206, 11, 105, 124, 210, 14, 105, 124, 214, 11, 105, 124, 1, 186, 7, 105, 127, 194, 7, 105, 127, 202, 7, 105, 127, 238, 202, 11, 105, 124, 206, 11, 105, 124, 210, 14, 105, 124, 214, 11, 105, 123, 1, 186, 6, 105, 125, 190, 7, 105, 125, 194, 7, 105, 127, 202, 7, 105, 127, 238, 202, 11, 105, 124, 206, 11, 105, 124, 210, 14, 105, 124, 214, 11, 105, 124, 1, 186, 7, 105, 127, 194, 7, 105, 127, 202, 7, 105, 127, 238, 202, 11, 105, 124, 206, 11, 105, 124, 210, 14, 105, 124, 214, 11, 105, 123, 1, 186, 6, 105, 125, 190, 7, 105, 124, 194, 5, 105, 123, 198, 9, 105, 124, 202, 9, 115, 125, 190, 10, 113, 123, 235, 198, 10, 115, 123, 202, 21, 117, 123, 206, 19, 115, 123, 210, 21, 114, 123, 235, 1, 186, 23, 117, 125, 235, 194, 19, 118, 125, 235, 202, 16, 116, 127, 235]', 11, 18, 0.2234195402298852, 0.0925925925925925]], columns=['Unnamed: 0', 'midiTokens', 'pitchCount', 'pitchRange', 'averageNoteDuration', 'stepwiseMotion'])

bachDf = pd.DataFrame([[0, '[202, 48, 117, 144, 210, 53, 117, 135, 1, 186, 51, 117, 155, 194, 52, 117, 128, 202, 53, 117, 135, 1, 186, 51, 117, 135, 190, 56, 117, 125, 194, 58, 117, 125, 198, 46, 117, 125, 202, 58, 117, 125, 206, 56, 117, 125, 210, 56, 117, 125, 214, 55, 117, 125, 1, 186, 53, 117, 125, 190, 51, 117, 125, 194, 53, 117, 125, 198, 54, 117, 125, 202, 53, 117, 125, 1, 186, 51, 117, 155, 54, 117, 143, 1, 194, 48, 117, 132, 198, 50, 117, 124, 210, 63, 117, 131, 212, 53, 117, 123, 214, 51, 117, 123, 216, 53, 117, 123, 56, 117, 123, 1, 186, 55, 117, 131, 58, 117, 131, 196, 56, 117, 123, 198, 55, 117, 123, 200, 53, 117, 123, 202, 55, 117, 131, 214, 53, 117, 124, 1, 186, 52, 117, 127, 194, 53, 117, 124, 198, 55, 117, 123, 200, 57, 117, 123, 63, 121, 123, 188, 64, 121, 123, 190, 62, 121, 123, 192, 60, 121, 123, 194, 59, 121, 123, 196, 57, 121, 123, 198, 56, 121, 123, 200, 57, 121, 123, 202, 50, 121, 123, 204, 56, 121, 123, 206, 59, 121, 123, 208, 56, 121, 123, 210, 62, 121, 123, 212, 59, 121, 123, 214, 56, 121, 123, 216, 59, 121, 123, 1, 186, 52, 121, 123, 188, 56, 121, 123, 190, 52, 121, 123, 192, 50, 121, 123, 194, 62, 121, 124, 198, 62, 121, 124, 202, 62, 121, 124, 206, 60]', 16, 18, 0.3511904761904762, 0.0], [1, '[202, 43, 118, 125, 206, 56, 117, 126, 216, 58, 117, 123, 1, 186, 51, 117, 127, 194, 44, 117, 127, 53, 117, 127, 235, 204, 43, 117, 123, 206, 44, 117, 123, 208, 41, 117, 123, 210, 48, 117, 126, 216, 51, 117, 123, 1, 186, 56, 117, 131, 192, 41, 117, 123, 194, 43, 117, 126, 196, 55, 117, 123, 198, 53, 117, 123, 200, 44, 117, 123, 53, 117, 123, 194, 43, 117, 127, 58, 117, 123, 196, 55, 117, 123, 198, 53, 117, 123, 200, 55, 117, 123, 202, 60, 117, 143, 1, 210, 48, 117, 127, 1, 186, 50, 117, 127, 194, 52, 117, 127, 202, 51, 117, 134, 210, 55, 117, 127, 1, 186, 57, 117, 140, 235, 210, 55, 117, 127, 1, 186, 53, 117, 127, 194, 55, 117, 127, 202, 50, 117, 134, 57, 117, 127, 210, 55, 117, 127, 1, 186, 50, 117, 140, 194, 42, 117, 127, 202, 46, 117, 140, 210, 50, 117, 127, 1, 186, 55, 117, 127, 194, 50, 117, 127, 202, 48, 117, 127, 210, 46, 117, 127, 57, 117, 127, 1, 186, 48, 117, 127, 55, 117, 127, 194, 48, 117, 127, 55, 117, 127, 202, 48, 117, 127, 55, 117, 127, 210, 48, 117, 127, 56, 117, 127, 1, 186, 55, 117, 124, 190, 53, 117, 124, 194, 51, 117, 124, 198, 53, 117, 124, 202, 55, 117, 124, 214, 57, 117, 124, 1, 186, 48, 117, 127, 53, 117, 131, 194, 48, 117]', 15, 19, 0.4091666666666668, 0.0], [2, '[194, 56, 117, 153, 1, 202, 54, 117, 138, 1, 186, 54, 117, 125, 190, 56, 117, 125, 194, 58, 117, 125, 198, 46, 118, 127, 200, 44, 118, 123, 202, 222, 51, 118, 137, 55, 118, 133, 58, 118, 133, 206, 54, 118, 123, 208, 52, 118, 123, 210, 53, 118, 123, 212, 52, 118, 123, 214, 53, 118, 123, 216, 52, 118, 123, 1, 186, 50, 118, 129, 53, 118, 147, 198, 45, 118, 125, 202, 50, 118, 125, 206, 48, 118, 125, 210, 47, 118, 129, 212, 52, 118, 123, 214, 48, 118, 123, 216, 45, 118, 123, 1, 186, 48, 118, 153, 192, 47, 114, 123, 204, 45, 114, 123, 206, 50, 114, 123, 55, 114, 145, 208, 46, 114, 123, 210, 50, 114, 123, 212, 43, 114, 123, 214, 50, 114, 123, 216, 40, 114, 123, 1, 186, 42, 114, 123, 188, 40, 114, 123, 190, 41, 114, 127, 192, 52, 112, 125, 235, 198, 60, 112, 124, 202, 55, 112, 131, 212, 57, 116, 123, 196, 52, 116, 123, 198, 48, 116, 123, 200, 52, 116, 123, 202, 60, 116, 131, 212, 53, 116, 123, 214, 50, 116, 123, 216, 53, 116, 123, 1, 186, 52, 116, 124, 190, 60, 116, 124, 196, 53, 116, 123, 198, 48, 116, 123, 200, 51, 116, 123, 202, 50, 116, 124, 206, 62, 116, 124, 210, 60, 116, 123, 212, 62, 116, 123, 214, 60, 116, 123, 216, 62, 116, 123, 1, 186, 60, 116, 123, 188]', 20, 22, 0.330409356725146, 0.0], [3, '[125, 55, 117, 123, 206, 50, 117, 124, 62, 117, 123, 208, 60, 117, 123, 210, 59, 117, 123, 212, 62, 117, 123, 214, 63, 117, 123, 216, 65, 117, 123, 1, 186, 63, 117, 123, 188, 62, 117, 123, 190, 60, 117, 123, 192, 58, 117, 123, 194, 60, 117, 123, 196, 62, 117, 123, 198, 64, 117, 123, 200, 65, 117, 123, 202, 62, 117, 123, 204, 63, 117, 123, 206, 62, 117, 123, 208, 63, 117, 123, 210, 58, 117, 123, 212, 62, 117, 123, 214, 63, 117, 127, 1, 188, 62, 117, 123, 190, 60, 117, 123, 192, 62, 117, 123, 194, 63, 117, 123, 196, 62, 117, 123, 198, 60, 117, 123, 200, 62, 117, 123, 202, 58, 117, 123, 204, 60, 117, 123, 206, 59, 117, 123, 208, 60, 117, 123, 210, 64, 117, 123, 212, 60, 117, 123, 214, 59, 117, 123, 216, 55, 117, 123, 1, 186, 53, 117, 123, 188, 52, 117, 123, 190, 50, 117, 123, 192, 48, 117, 123, 194, 50, 117, 123, 196, 53, 117, 122, 198, 58, 117, 122, 199, 56, 117, 122, 200, 58, 117, 122, 201, 61, 117, 122, 202, 61, 117, 122, 203, 60, 117, 122, 205, 58, 117, 122, 206, 55, 117, 122, 61, 117, 122, 207, 52, 121, 122, 207, 53, 121, 122, 208, 62, 121, 123, 210, 61, 121, 123, 212, 60, 121, 123, 214, 59, 121, 123, 216, 60, 121, 123, 1, 186, 54, 121, 123, 188, 57, 121, 123]', 16, 17, 0.1090277777777777, 0.6909090909090909], [4, '[202, 55, 117, 141, 214, 53, 117, 125, 1, 186, 54, 117, 135, 190, 50, 117, 125, 194, 52, 117, 125, 198, 53, 117, 125, 202, 50, 117, 128, 210, 55, 117, 129, 1, 186, 51, 117, 132, 55, 117, 128, 194, 55, 117, 128, 202, 53, 117, 141, 214, 58, 117, 125, 1, 186, 60, 117, 125, 190, 51, 117, 125, 194, 54, 117, 125, 60, 117, 129, 198, 49, 117, 125, 59, 117, 125, 202, 49, 117, 125, 58, 117, 141, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 52, 117, 151, 190, 58, 117, 125, 194, 60, 117, 125, 198, 61, 117, 125, 202, 60, 117, 125, 206, 58, 117, 125, 210, 51, 117, 128, 55, 117, 128, 1, 186, 48, 117, 128, 194, 58, 117, 128, 202, 48, 117, 128, 202, 53, 117, 128, 210, 52, 117, 128, 1, 186, 53, 117, 125, 190, 60, 117, 125, 194, 58, 117, 125, 198, 60, 117, 125, 202, 58, 117, 128, 210, 52, 117, 125, 214, 53, 117, 125, 1, 186, 55, 117, 135, 190, 53, 121, 123, 192, 55, 121, 123, 194, 56, 121, 123, 196, 53, 121, 123, 198, 50, 121, 123, 200, 46, 121, 123, 202, 55, 121, 123, 204, 45, 121, 123, 206, 49, 121, 123, 208, 45, 121, 123, 210, 48, 121, 123, 212, 50, 121, 123, 214, 52, 121, 123, 216, 54, 121, 123, 1, 186, 55, 121, 124, 190, 60, 121, 124, 194, 57, 121]', 15, 16, 0.3486842105263159, 0.4615384615384615], [5, '[194, 48, 117, 128, 202, 46, 117, 133, 214, 43, 117, 125, 46, 117, 125, 1, 186, 46, 117, 125, 190, 55, 117, 125, 194, 56, 117, 125, 198, 58, 117, 125, 202, 56, 117, 125, 206, 58, 117, 125, 210, 60, 117, 125, 214, 62, 117, 125, 1, 186, 65, 117, 125, 190, 65, 117, 125, 194, 64, 117, 125, 198, 61, 117, 125, 202, 60, 117, 125, 206, 58, 117, 125, 210, 55, 117, 128, 1, 186, 48, 117, 128, 194, 58, 117, 128, 202, 55, 117, 128, 210, 53, 117, 128, 1, 186, 51, 117, 128, 194, 56, 117, 148, 206, 53, 117, 125, 210, 55, 117, 125, 214, 56, 117, 125, 1, 186, 55, 117, 141, 190, 46, 117, 125, 194, 51, 117, 125, 198, 49, 117, 125, 202, 51, 117, 125, 206, 49, 117, 125, 210, 49, 117, 132, 1, 186, 48, 117, 147, 51, 117, 125, 198, 49, 118, 123, 200, 48, 118, 123, 202, 49, 118, 123, 204, 46, 118, 123, 206, 44, 118, 123, 208, 49, 118, 123, 210, 43, 118, 123, 212, 49, 118, 123, 214, 43, 118, 123, 216, 49, 118, 123, 1, 186, 48, 118, 123, 188, 46, 118, 123, 190, 44, 118, 123, 192, 43, 118, 123, 194, 44, 118, 123, 196, 48, 118, 123, 198, 44, 118, 123, 200, 48, 118, 123, 200, 51, 121, 122, 204, 49, 121, 123, 206, 47, 121, 123, 208, 45, 121, 123, 210, 48, 121, 123, 212, 39, 121, 123, 214]', 18, 26, 0.2744252873563218, 0.0], [6, '[202, 60, 117, 122, 208, 49, 117, 122, 58, 117, 122, 210, 46, 117, 122, 61, 117, 122, 211, 43, 117, 122, 58, 117, 122, 213, 44, 117, 122, 60, 117, 122, 213, 44, 117, 122, 60, 117, 122, 214, 44, 117, 122, 60, 117, 122, 215, 46, 117, 122, 61, 117, 122, 216, 44, 117, 122, 61, 117, 122, 1, 186, 46, 117, 122, 61, 117, 122, 187, 48, 117, 122, 60, 117, 122, 189, 49, 117, 122, 58, 117, 122, 190, 49, 117, 122, 58, 117, 122, 191, 60, 117, 122, 193, 60, 117, 122, 194, 58, 117, 122, 195, 60, 117, 122, 197, 58, 117, 122, 198, 60, 117, 122, 199, 58, 117, 122, 200, 57, 117, 122, 201, 55, 117, 122, 202, 55, 117, 122, 56, 117, 122, 203, 55, 117, 122, 56, 117, 122, 204, 55, 117, 122, 205, 53, 117, 122, 206, 56, 117, 122, 207, 55, 117, 123, 62, 116, 123, 214, 57, 116, 123, 216, 58, 116, 122, 217, 57, 116, 122, 1, 186, 55, 116, 123, 188, 53, 116, 123, 190, 51, 116, 123, 192, 50, 116, 123, 194, 51, 115, 125, 214, 48, 115, 125, 64, 115, 125, 1, 186, 57, 115, 125, 64, 115, 125, 190, 57, 115, 125, 62, 115, 123, 192, 60, 115, 123, 194, 57, 115, 125, 62, 115, 125, 198, 57, 115, 125, 62, 115, 125, 202, 57, 115, 125, 62, 115, 125, 206, 55, 115, 123, 208, 53, 115, 123, 210, 55, 115, 125]', 16, 21, 0.0815217391304347, 0.7916666666666666], [7, '[194, 54, 121, 124, 196, 55, 121, 123, 198, 57, 121, 123, 200, 48, 117, 127, 56, 117, 127, 194, 48, 117, 127, 55, 117, 127, 202, 48, 117, 127, 56, 117, 127, 210, 49, 117, 127, 55, 117, 127, 1, 186, 54, 112, 125, 190, 55, 112, 123, 192, 35, 121, 123, 194, 50, 121, 123, 196, 47, 121, 123, 198, 55, 121, 128, 206, 58, 113, 123, 208, 56, 113, 123, 210, 58, 113, 124, 214, 53, 113, 124, 1, 186, 51, 113, 122, 188, 49, 113, 135, 190, 48, 113, 123, 192, 46, 113, 123, 194, 48, 113, 123, 196, 51, 113, 123, 198, 48, 113, 123, 200, 51, 113, 123, 202, 44, 113, 123, 204, 55, 113, 123, 206, 53, 113, 123, 208, 51, 113, 123, 210, 53, 113, 123, 212, 55, 113, 123, 214, 53, 113, 123, 216, 56, 113, 123, 1, 186, 50, 113, 123, 188, 56, 113, 123, 190, 53, 113, 123, 192, 56, 113, 123, 194, 48, 113, 124, 196, 53, 113, 123, 198, 53, 113, 123, 200, 56, 113, 123, 202, 46, 113, 127, 204, 55, 113, 123, 206, 53, 113, 123, 208, 51, 113, 122, 210, 49, 113, 123, 53, 113, 127, 212, 48, 113, 123, 214, 46, 113, 123, 216, 44, 113, 123, 1, 186, 49, 113, 127, 188, 51, 113, 123, 190, 56, 113, 123, 192, 49, 113, 123, 55, 113, 123, 194, 48, 113, 134, 56, 113, 127, 202, 49, 113, 127, 55, 113, 127, 210, 48]', 14, 23, 0.1875, 0.0], [8, '[236, 235, 186, 59, 121, 126, 192, 57, 121, 123, 194, 60, 121, 123, 196, 57, 121, 123, 198, 56, 121, 123, 200, 57, 121, 123, 202, 52, 121, 123, 204, 56, 121, 123, 206, 59, 121, 123, 208, 56, 121, 123, 210, 62, 121, 123, 212, 59, 121, 123, 214, 56, 121, 123, 216, 59, 121, 123, 1, 186, 60, 121, 124, 190, 48, 121, 124, 194, 62, 121, 132, 206, 60, 121, 123, 208, 59, 121, 123, 210, 60, 121, 124, 214, 55, 121, 124, 1, 186, 56, 117, 123, 194, 46, 117, 134, 53, 117, 127, 202, 55, 117, 127, 64, 117, 127, 202, 55, 117, 156, 61, 117, 140, 210, 43, 117, 127, 1, 186, 48, 117, 140, 194, 50, 117, 127, 202, 51, 117, 127, 210, 51, 117, 127, 1, 186, 41, 117, 134, 50, 117, 140, 235, 210, 60, 117, 127, 1, 186, 62, 117, 127, 194, 58, 117, 127, 64, 117, 127, 202, 52, 117, 127, 61, 117, 140, 210, 54, 117, 127, 1, 186, 55, 117, 140, 235, 210, 53, 117, 127, 1, 186, 51, 117, 127, 194, 48, 117, 127, 202, 55, 117, 140, 235, 202, 53, 117, 140, 235, 194, 50, 117, 127, 202, 51, 117, 127, 210, 53, 117, 127, 1, 186, 55, 117, 140, 194, 43, 117, 127, 202, 48, 117, 150, 210, 55, 117, 127, 1, 186, 56, 117, 127, 194, 56, 117, 127, 202, 51, 117, 140, 210, 50, 117, 127, 1, 186, 55, 117, 127]', 18, 23, 0.4619252873563219, 0.0], [9, '[203, 59, 121, 122, 196, 41, 121, 123, 198, 40, 121, 123, 200, 34, 121, 123, 204, 38, 121, 123, 206, 42, 121, 123, 208, 45, 121, 123, 210, 48, 121, 123, 212, 45, 121, 123, 214, 42, 121, 123, 216, 45, 121, 123, 1, 186, 35, 121, 123, 188, 45, 121, 123, 190, 48, 121, 123, 192, 52, 121, 123, 194, 50, 121, 123, 196, 48, 121, 123, 198, 47, 121, 123, 200, 44, 121, 123, 202, 50, 121, 123, 204, 53, 121, 123, 206, 43, 121, 123, 208, 53, 121, 123, 210, 43, 121, 124, 212, 50, 121, 124, 214, 38, 121, 124, 216, 50, 121, 123, 1, 186, 43, 121, 124, 188, 50, 121, 123, 190, 47, 121, 123, 192, 43, 121, 123, 194, 55, 121, 123, 196, 50, 121, 123, 198, 48, 121, 123, 200, 47, 121, 123, 202, 43, 121, 127, 210, 48, 121, 123, 212, 52, 121, 123, 214, 53, 121, 123, 216, 52, 121, 123, 1, 186, 53, 121, 123, 188, 50, 121, 123, 190, 55, 121, 123, 192, 50, 121, 123, 194, 54, 121, 123, 196, 50, 121, 123, 198, 60, 121, 123, 200, 50, 121, 123, 204, 40, 117, 123, 206, 60, 118, 123, 208, 62, 118, 123, 210, 60, 118, 123, 212, 64, 118, 123, 214, 48, 118, 125, 57, 118, 123, 216, 64, 118, 123, 1, 186, 47, 118, 125, 61, 118, 123, 188, 57, 118, 123, 190, 59, 118, 123, 192, 60, 118, 123, 194, 51, 118, 122]', 23, 30, 0.1317204301075268, 0.2407407407407407], [10, '[202, 55, 105, 125, 206, 58, 105, 126, 210, 57, 105, 125, 214, 55, 105, 125, 1, 186, 54, 105, 125, 190, 51, 105, 124, 194, 50, 105, 126, 198, 48, 105, 126, 202, 46, 105, 125, 206, 45, 105, 126, 194, 47, 105, 125, 198, 50, 105, 124, 202, 54, 105, 126, 206, 55, 105, 122, 208, 53, 105, 123, 210, 53, 117, 124, 198, 52, 117, 124, 202, 53, 117, 124, 206, 55, 117, 124, 210, 52, 117, 134, 236, 202, 50, 117, 127, 210, 52, 117, 127, 1, 186, 54, 117, 125, 202, 56, 117, 153, 1, 202, 53, 117, 146, 1, 198, 55, 117, 131, 210, 60, 117, 145, 1, 202, 58, 117, 153, 1, 202, 56, 117, 153, 1, 202, 54, 117, 125, 63, 117, 141, 214, 56, 117, 145, 1, 198, 55, 117, 124, 202, 56, 117, 134, 206, 59, 117, 127, 214, 56, 117, 124, 1, 186, 58, 117, 124, 190, 56, 117, 124, 194, 65, 117, 131, 204, 63, 117, 123, 206, 62, 117, 123, 208, 60, 117, 123, 210, 59, 117, 123, 212, 62, 117, 123, 214, 63, 117, 123, 216, 65, 117, 123, 1, 186, 56, 117, 124, 190, 55, 117, 127, 196, 56, 117, 123, 198, 58, 117, 123, 200, 54, 117, 123, 202, 51, 117, 127, 54, 117, 127, 204, 49, 117, 123, 206, 54, 121, 123, 208, 52, 121, 123, 210, 56, 121, 123, 212, 54, 121, 123, 214, 48, 121, 123, 216, 54, 121, 123, 1]', 19, 20, 0.4181818181818182, 0.0], [11, '[202, 60, 117, 125, 206, 58, 117, 125, 210, 48, 117, 125, 214, 47, 117, 125, 1, 186, 48, 117, 125, 190, 55, 117, 125, 194, 53, 117, 125, 198, 55, 117, 125, 202, 51, 117, 128, 210, 60, 117, 133, 1, 186, 50, 117, 141, 190, 60, 117, 125, 194, 58, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 1, 190, 56, 117, 125, 194, 58, 117, 125, 198, 60, 117, 125, 202, 58, 117, 128, 210, 65, 117, 154, 214, 55, 117, 125, 1, 186, 57, 117, 125, 190, 58, 117, 125, 194, 60, 117, 125, 198, 57, 117, 125, 202, 58, 117, 125, 206, 56, 117, 125, 210, 54, 117, 125, 214, 53, 117, 125, 1, 186, 54, 117, 135, 190, 50, 117, 125, 194, 52, 117, 125, 198, 53, 117, 125, 202, 50, 117, 131, 214, 48, 117, 123, 216, 50, 117, 123, 1, 186, 48, 117, 127, 243, 235, 214, 64, 117, 124, 1, 186, 55, 117, 124, 61, 117, 124, 190, 53, 117, 124, 61, 117, 124, 194, 53, 117, 124, 60, 117, 124, 198, 58, 117, 124, 202, 57, 117, 123, 204, 58, 117, 123, 206, 56, 117, 123, 208, 55, 117, 123, 210, 56, 117, 123, 212, 54, 117, 123, 214, 43, 121, 124, 1, 186, 41, 121, 124, 50, 121, 134, 190, 43, 121, 124, 194, 45, 121, 124, 198, 42, 121, 123, 196, 55, 121, 123, 198, 59, 121, 123, 200, 57, 121, 123, 202, 59, 121, 123]', 20, 24, 0.3216374269005849, 0.0], [12, '[194, 52, 117, 128, 236, 235, 214, 57, 121, 123, 216, 54, 121, 123, 1, 186, 50, 121, 123, 188, 53, 121, 123, 190, 57, 121, 123, 192, 58, 121, 123, 194, 57, 121, 123, 196, 55, 121, 123, 198, 53, 121, 123, 200, 55, 121, 123, 202, 57, 121, 123, 204, 60, 121, 123, 206, 53, 121, 123, 208, 62, 121, 123, 210, 52, 121, 123, 212, 60, 121, 123, 214, 59, 121, 123, 216, 60, 121, 123, 1, 186, 50, 121, 123, 188, 52, 121, 123, 190, 50, 121, 123, 192, 52, 121, 123, 194, 53, 121, 123, 196, 55, 121, 123, 198, 52, 121, 123, 200, 55, 121, 123, 202, 58, 121, 124, 206, 57, 121, 124, 210, 54, 121, 124, 214, 48, 121, 124, 235, 190, 57, 121, 124, 194, 54, 121, 124, 198, 47, 121, 124, 235, 206, 57, 121, 124, 210, 54, 121, 124, 214, 48, 121, 124, 235, 190, 57, 121, 124, 194, 54, 121, 124, 198, 47, 121, 124, 235, 206, 57, 121, 124, 210, 54, 121, 124, 214, 48, 121, 124, 235, 190, 57, 121, 124, 194, 54, 121, 124, 198, 47, 121, 124, 235, 206, 57, 121, 124, 210, 54, 121, 124, 214, 48, 121, 124, 235, 190, 57, 121, 124, 194, 54, 121, 124, 198, 47, 121, 124, 235, 206, 54, 121, 124, 210, 51, 121, 124, 214, 47, 121, 124, 1, 186, 55, 121, 124, 190, 57, 121, 124, 194, 54, 121, 124, 198, 50, 121, 124, 235]', 13, 15, 0.1518361581920902, 0.2586206896551724], [13, '[50, 121, 123, 194, 47, 121, 123, 196, 50, 121, 123, 198, 48, 121, 123, 200, 47, 121, 123, 202, 45, 121, 124, 235, 210, 57, 121, 131, 1, 188, 52, 121, 123, 190, 57, 121, 123, 192, 54, 121, 123, 194, 62, 121, 123, 196, 57, 121, 123, 198, 54, 121, 123, 200, 57, 121, 123, 202, 50, 121, 123, 204, 54, 121, 123, 206, 57, 121, 123, 208, 54, 121, 123, 210, 60, 121, 123, 212, 57, 121, 123, 214, 54, 121, 123, 216, 57, 121, 123, 1, 186, 60, 121, 123, 188, 57, 121, 123, 190, 59, 121, 123, 192, 55, 121, 123, 194, 62, 121, 123, 196, 59, 121, 123, 198, 55, 121, 123, 200, 59, 121, 123, 202, 50, 121, 123, 204, 56, 121, 123, 206, 59, 121, 123, 208, 56, 121, 123, 210, 62, 121, 123, 212, 59, 121, 123, 214, 56, 121, 123, 216, 59, 121, 123, 1, 186, 60, 121, 123, 188, 62, 121, 123, 190, 60, 121, 123, 192, 59, 121, 123, 194, 57, 121, 123, 196, 55, 121, 123, 198, 53, 121, 123, 200, 57, 121, 123, 202, 50, 121, 123, 204, 54, 121, 123, 206, 57, 121, 123, 208, 54, 121, 123, 210, 60, 121, 123, 212, 57, 121, 123, 214, 54, 121, 123, 216, 54, 121, 123, 1, 186, 53, 121, 123, 188, 57, 121, 123, 190, 62, 121, 123, 192, 57, 121, 123, 194, 65, 121, 123, 196, 62, 121, 123, 198, 57, 121, 123, 200, 62]', 14, 20, 0.133879781420765, 0.2], [14, '[202, 55, 117, 128, 60, 117, 128, 194, 49, 117, 128, 52, 117, 128, 202, 48, 117, 128, 53, 117, 125, 206, 56, 110, 122, 201, 55, 110, 122, 202, 52, 110, 127, 55, 110, 127, 235, 214, 49, 110, 131, 56, 110, 130, 1, 192, 58, 110, 123, 194, 49, 110, 127, 55, 110, 127, 235, 206, 49, 110, 124, 210, 46, 110, 125, 214, 44, 110, 125, 1, 186, 43, 110, 129, 53, 110, 137, 194, 46, 110, 129, 202, 44, 110, 129, 51, 110, 137, 210, 44, 110, 137, 1, 186, 51, 110, 130, 192, 58, 110, 123, 193, 51, 116, 122, 194, 55, 116, 123, 196, 54, 116, 123, 198, 52, 116, 123, 200, 55, 116, 123, 202, 49, 116, 123, 204, 50, 116, 123, 206, 47, 116, 123, 208, 49, 116, 123, 210, 45, 116, 124, 214, 52, 116, 124, 1, 186, 53, 116, 123, 188, 48, 116, 123, 190, 50, 116, 123, 192, 52, 116, 123, 194, 50, 116, 123, 196, 53, 116, 123, 198, 48, 116, 123, 200, 47, 116, 123, 202, 48, 116, 127, 235, 214, 50, 116, 124, 1, 186, 52, 116, 123, 188, 48, 116, 123, 190, 52, 116, 123, 192, 55, 116, 123, 194, 60, 116, 131, 204, 57, 116, 123, 206, 62, 116, 123, 208, 60, 116, 123, 210, 62, 116, 123, 212, 64, 116, 123, 214, 62, 116, 123, 216, 64, 116, 123, 216, 62, 116, 123, 1, 186, 60, 116, 123, 188, 59, 116, 123, 190]', 20, 21, 0.2552083333333333, 0.0], [15, '[202, 50, 110, 122, 52, 110, 122, 56, 110, 122, 199, 50, 118, 122, 200, 52, 118, 122, 201, 54, 118, 122, 202, 51, 118, 125, 56, 118, 153, 206, 51, 118, 123, 208, 53, 118, 123, 210, 55, 118, 127, 216, 53, 118, 123, 1, 186, 55, 118, 122, 53, 112, 123, 188, 56, 112, 123, 190, 51, 112, 123, 192, 56, 112, 123, 194, 50, 112, 123, 196, 55, 112, 123, 198, 46, 117, 123, 200, 41, 117, 126, 202, 44, 117, 124, 48, 117, 124, 206, 219, 43, 117, 124, 46, 117, 124, 52, 117, 124, 58, 117, 124, 212, 58, 117, 123, 214, 60, 117, 123, 216, 62, 117, 123, 1, 186, 58, 117, 123, 188, 62, 117, 123, 190, 65, 117, 127, 196, 63, 117, 123, 198, 62, 117, 123, 200, 63, 117, 123, 202, 61, 117, 123, 204, 63, 117, 123, 206, 62, 117, 123, 208, 63, 117, 123, 210, 61, 117, 123, 212, 63, 117, 123, 214, 62, 117, 123, 216, 63, 117, 123, 1, 186, 56, 117, 124, 190, 55, 117, 127, 58, 117, 123, 192, 56, 117, 123, 194, 55, 117, 123, 196, 53, 117, 123, 198, 51, 117, 123, 200, 50, 117, 123, 56, 117, 123, 202, 48, 117, 127, 56, 117, 123, 204, 63, 117, 123, 206, 61, 117, 126, 208, 56, 117, 123, 210, 44, 117, 127, 55, 117, 123, 212, 58, 117, 123, 214, 56, 117, 123, 216, 55, 117, 123, 1, 186, 53, 117, 123, 188]', 18, 24, 0.1525297619047619, 0.0], [16, '[202, 48, 121, 124, 53, 121, 150, 206, 54, 121, 124, 210, 52, 121, 124, 214, 50, 121, 124, 1, 186, 49, 121, 124, 190, 50, 121, 124, 194, 52, 121, 124, 198, 49, 121, 124, 202, 46, 121, 134, 210, 55, 121, 127, 1, 186, 45, 121, 134, 53, 121, 140, 57, 121, 134, 202, 50, 121, 143, 210, 46, 121, 124, 214, 45, 121, 124, 1, 186, 55, 121, 127, 194, 45, 121, 134, 53, 121, 127, 202, 50, 121, 134, 210, 59, 121, 127, 1, 186, 48, 121, 127, 59, 121, 127, 194, 45, 121, 134, 53, 121, 127, 194, 57, 121, 124, 198, 55, 121, 124, 202, 57, 121, 127, 210, 53, 121, 127, 62, 121, 135, 1, 186, 50, 121, 127, 194, 49, 121, 124, 51, 121, 127, 202, 48, 121, 124, 51, 121, 127, 206, 45, 121, 124, 210, 43, 121, 124, 214, 41, 121, 124, 1, 186, 40, 121, 124, 190, 48, 121, 124, 194, 38, 121, 124, 198, 47, 121, 124, 202, 36, 121, 124, 206, 48, 121, 123, 208, 47, 121, 123, 210, 50, 121, 123, 212, 47, 121, 123, 214, 55, 121, 123, 216, 45, 121, 123, 1, 186, 53, 121, 123, 188, 57, 121, 123, 190, 62, 121, 123, 192, 57, 121, 123, 194, 65, 121, 123, 196, 62, 121, 123, 198, 57, 121, 123, 200, 62, 121, 123, 202, 53, 121, 123, 204, 57, 121, 123, 206, 54, 121, 123, 208, 63, 121, 123, 210, 64, 121, 123]', 22, 29, 0.3210382513661199, 0.6666666666666666], [17, '[202, 46, 117, 128, 56, 117, 125, 206, 55, 117, 125, 210, 53, 117, 125, 60, 117, 133, 214, 52, 117, 125, 1, 186, 53, 117, 136, 202, 50, 117, 135, 206, 53, 117, 125, 210, 55, 117, 128, 1, 186, 56, 117, 135, 190, 55, 117, 125, 194, 53, 117, 125, 198, 55, 117, 125, 202, 56, 117, 125, 206, 58, 117, 125, 210, 56, 117, 125, 214, 55, 117, 125, 58, 117, 125, 1, 186, 56, 117, 125, 63, 117, 128, 190, 53, 117, 125, 194, 51, 117, 125, 55, 117, 135, 202, 48, 117, 135, 57, 117, 135, 1, 186, 54, 117, 128, 194, 56, 117, 128, 202, 53, 117, 144, 1, 186, 60, 117, 151, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 48, 117, 128, 194, 53, 117, 128, 202, 52, 117, 135, 206, 61, 117, 125, 210, 60, 117, 125, 214, 58, 117, 125, 1, 186, 56, 117, 149, 190, 48, 117, 125, 194, 53, 117, 125, 198, 51, 117, 125, 202, 49, 117, 141, 206, 53, 117, 125, 210, 58, 117, 125, 214, 56, 117, 125, 1, 186, 55, 117, 141, 190, 46, 117, 125, 194, 51, 117, 125, 198, 49, 117, 125, 202, 51, 117, 125, 206, 49, 117, 125, 210, 47, 117, 125, 214, 46, 117, 125, 1, 186, 47, 117, 128, 194, 51, 117, 133, 214, 43, 117, 125, 51, 117, 125, 1, 186, 46, 117, 125, 55, 117, 124, 190, 51, 117, 123]', 17, 20, 0.4498587570621469, 0.5], [18, '[236, 235, 186, 42, 117, 127, 188, 58, 117, 123, 190, 56, 117, 123, 192, 55, 117, 123, 194, 53, 117, 126, 196, 51, 117, 123, 198, 49, 117, 123, 200, 48, 117, 123, 202, 49, 117, 124, 1, 192, 42, 117, 123, 49, 117, 123, 194, 51, 117, 131, 196, 49, 117, 123, 198, 48, 117, 123, 200, 46, 117, 123, 202, 48, 117, 124, 206, 56, 117, 124, 210, 49, 117, 124, 214, 53, 117, 124, 60, 117, 124, 1, 186, 53, 117, 147, 56, 117, 140, 235, 210, 54, 117, 127, 1, 186, 51, 117, 127, 55, 117, 127, 194, 47, 117, 127, 50, 117, 127, 202, 48, 117, 127, 51, 117, 127, 210, 48, 117, 127, 53, 117, 127, 56, 117, 127, 1, 186, 49, 117, 134, 52, 117, 127, 194, 55, 117, 127, 202, 48, 117, 127, 52, 117, 127, 235, 212, 48, 117, 123, 214, 52, 121, 123, 192, 50, 121, 123, 194, 62, 121, 123, 196, 57, 121, 123, 198, 53, 121, 123, 200, 57, 121, 123, 202, 50, 121, 123, 204, 53, 121, 123, 206, 59, 121, 123, 208, 55, 121, 123, 210, 62, 121, 123, 212, 59, 121, 123, 214, 55, 121, 123, 216, 50, 105, 123, 190, 55, 105, 123, 192, 53, 105, 123, 194, 51, 105, 124, 196, 50, 105, 123, 198, 48, 105, 123, 200, 46, 105, 123, 204, 43, 121, 123, 206, 41, 121, 123, 208, 44, 121, 123, 235, 214, 218, 43, 121, 124, 47, 121]', 20, 21, 0.2694444444444444, 0.0], [19, '[58, 118, 125, 235, 206, 54, 118, 125, 56, 118, 125, 210, 50, 118, 125, 55, 118, 125, 214, 48, 118, 129, 53, 118, 125, 1, 186, 48, 118, 129, 51, 118, 129, 194, 50, 118, 125, 198, 48, 118, 125, 206, 50, 118, 125, 210, 47, 118, 129, 214, 50, 118, 125, 1, 186, 52, 121, 124, 59, 121, 124, 62, 121, 124, 206, 218, 52, 121, 124, 55, 121, 124, 60, 121, 124, 210, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 214, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 1, 186, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 190, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 194, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 198, 218, 54, 121, 124, 59, 121, 124, 62, 121, 124, 202, 222, 55, 121, 124, 59, 121, 124, 62, 121, 124, 206, 218, 52, 121, 124, 60, 121, 124, 64, 121, 124, 210, 221, 57, 121, 124, 60, 121, 124, 64, 121, 124, 238, 206, 60, 121, 124, 64, 121, 124, 210, 59, 121, 124, 62, 121, 124, 214, 62, 121, 124, 1, 186, 52, 121, 127, 60, 121, 124, 190, 59, 121, 123, 192, 57, 121, 123, 194, 55, 121, 123, 62, 121, 123, 196, 60, 121, 123, 198, 58, 117, 127, 200, 59, 117, 123, 202, 60, 117, 123, 204, 63, 117, 123, 206, 62, 117, 123, 208, 63, 117, 123, 210, 58, 117, 123, 212]', 16, 17, 0.2027027027027027, 0.4285714285714285], [20, '[202, 49, 117, 128, 210, 48, 117, 128, 1, 186, 49, 117, 141, 190, 53, 117, 125, 194, 55, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 214, 65, 117, 125, 64, 121, 124, 206, 57, 121, 124, 65, 121, 124, 210, 55, 121, 124, 64, 121, 124, 214, 53, 121, 124, 62, 121, 124, 1, 186, 52, 121, 124, 61, 121, 124, 190, 57, 121, 124, 194, 65, 121, 127, 202, 57, 121, 124, 206, 54, 121, 124, 210, 56, 121, 124, 214, 58, 121, 124, 1, 186, 56, 121, 124, 190, 53, 121, 124, 194, 58, 121, 124, 198, 60, 121, 124, 202, 59, 121, 124, 206, 62, 121, 124, 210, 63, 121, 124, 214, 60, 121, 124, 1, 186, 65, 121, 134, 190, 60, 121, 124, 194, 58, 121, 124, 198, 51, 121, 124, 202, 60, 121, 124, 206, 51, 121, 124, 210, 58, 121, 124, 214, 51, 121, 124, 1, 186, 56, 121, 124, 190, 51, 121, 124, 194, 55, 121, 124, 198, 51, 121, 124, 202, 56, 121, 123, 188, 44, 118, 123, 51, 118, 123, 190, 48, 118, 123, 57, 118, 123, 192, 43, 118, 123, 52, 118, 123, 194, 45, 118, 129, 54, 118, 125, 198, 48, 118, 125, 202, 50, 118, 129, 53, 118, 129, 235, 214, 57, 118, 125, 59, 118, 125, 1, 186, 55, 118, 125, 60, 118, 125, 190, 53, 118, 125, 62, 118, 125, 194, 52, 118, 129, 55, 118, 129, 235, 206, 59, 118, 125]', 21, 22, 0.267156862745098, 0.0], [21, '[54, 121, 123, 190, 58, 121, 123, 192, 50, 121, 123, 194, 49, 121, 123, 196, 58, 117, 123, 206, 60, 117, 123, 208, 61, 117, 123, 210, 55, 117, 128, 1, 186, 60, 117, 139, 190, 55, 117, 125, 194, 51, 117, 128, 204, 60, 121, 122, 206, 59, 121, 122, 207, 62, 121, 122, 200, 61, 121, 122, 201, 62, 121, 127, 200, 44, 118, 123, 51, 118, 123, 202, 46, 118, 145, 49, 118, 127, 53, 118, 167, 208, 53, 118, 123, 210, 55, 118, 127, 216, 56, 118, 123, 1, 186, 51, 118, 153, 192, 53, 118, 123, 194, 51, 118, 127, 200, 49, 118, 123, 202, 51, 118, 127, 208, 44, 118, 123, 210, 48, 118, 127, 216, 48, 118, 123, 1, 186, 53, 118, 143, 192, 58, 118, 123, 194, 56, 118, 127, 200, 55, 118, 123, 202, 56, 118, 143, 208, 53, 118, 123, 210, 51, 118, 127, 216, 50, 118, 123, 1, 186, 48, 118, 143, 192, 58, 118, 123, 194, 56, 118, 127, 200, 55, 118, 123, 202, 56, 118, 143, 208, 53, 118, 123, 210, 55, 118, 127, 216, 53, 118, 123, 1, 186, 55, 118, 122, 187, 56, 118, 122, 188, 58, 118, 125, 192, 60, 118, 123, 194, 58, 118, 127, 200, 56, 118, 123, 202, 58, 118, 127, 208, 55, 118, 123, 210, 53, 118, 122, 211, 55, 118, 122, 212, 53, 118, 125, 216, 51, 118, 123, 1, 186, 56, 118, 127, 192, 58, 118, 123]', 15, 18, 0.2960526315789473, 0.0], [22, '[58, 118, 123, 202, 218, 45, 118, 129, 48, 118, 129, 52, 118, 129, 55, 118, 129, 60, 118, 129, 194, 218, 55, 113, 127, 64, 113, 124, 236, 206, 60, 113, 124, 210, 58, 113, 124, 214, 57, 113, 124, 1, 186, 55, 113, 127, 194, 50, 113, 124, 198, 52, 113, 124, 202, 49, 113, 131, 210, 43, 113, 127, 214, 50, 113, 124, 1, 186, 42, 113, 127, 50, 113, 127, 194, 45, 113, 127, 202, 46, 113, 127, 210, 48, 113, 127, 1, 186, 50, 113, 127, 194, 48, 113, 124, 198, 46, 113, 124, 202, 45, 113, 127, 210, 46, 113, 124, 214, 44, 113, 123, 53, 113, 124, 1, 186, 51, 113, 124, 190, 55, 113, 124, 194, 53, 113, 124, 198, 55, 113, 124, 202, 56, 113, 124, 206, 55, 113, 124, 210, 56, 113, 124, 214, 58, 113, 124, 1, 186, 55, 113, 124, 190, 53, 113, 124, 194, 55, 113, 124, 198, 56, 113, 124, 202, 55, 113, 134, 1, 186, 51, 113, 127, 194, 53, 113, 124, 198, 55, 113, 124, 202, 56, 113, 124, 206, 58, 113, 124, 210, 56, 113, 124, 214, 58, 113, 124, 1, 186, 55, 113, 124, 190, 53, 113, 124, 194, 55, 113, 124, 198, 56, 113, 124, 202, 55, 113, 134, 210, 44, 113, 124, 214, 44, 113, 124, 1, 186, 45, 113, 134, 50, 113, 127, 194, 53, 113, 124, 198, 55, 113, 124, 202, 50, 113, 134, 57, 113, 127, 210]', 17, 22, 0.269736842105263, 0.7333333333333333], [23, '[202, 42, 121, 123, 205, 41, 121, 122, 206, 45, 112, 123, 192, 42, 112, 123, 194, 47, 112, 123, 196, 45, 112, 123, 198, 47, 112, 123, 54, 112, 131, 194, 43, 112, 125, 196, 53, 112, 125, 198, 47, 112, 125, 54, 112, 131, 198, 47, 112, 125, 54, 112, 125, 202, 43, 112, 129, 52, 112, 125, 206, 55, 112, 125, 210, 42, 112, 129, 51, 112, 125, 214, 54, 112, 125, 202, 47, 112, 129, 54, 112, 125, 206, 45, 112, 125, 210, 43, 112, 129, 47, 112, 129, 214, 55, 112, 125, 59, 112, 125, 1, 186, 52, 112, 129, 57, 112, 129, 235, 198, 54, 112, 124, 57, 112, 124, 202, 50, 112, 129, 55, 112, 129, 235, 214, 47, 112, 124, 57, 112, 124, 1, 186, 50, 112, 129, 52, 112, 129, 235, 198, 49, 112, 124, 57, 112, 124, 202, 49, 112, 129, 52, 112, 129, 235, 214, 48, 112, 124, 1, 186, 49, 112, 129, 52, 112, 129, 194, 52, 112, 131, 196, 40, 112, 123, 198, 45, 112, 123, 200, 40, 112, 123, 202, 42, 112, 129, 204, 45, 112, 123, 206, 50, 112, 123, 208, 45, 112, 123, 210, 42, 112, 123, 212, 45, 112, 123, 214, 42, 112, 123, 57, 112, 124, 192, 45, 121, 123, 194, 55, 121, 123, 196, 52, 121, 123, 198, 57, 121, 124, 202, 55, 121, 122, 203, 54, 121, 122, 204, 55, 121, 122, 205, 54, 121, 122, 206, 52, 121, 123]', 16, 19, 0.2215909090909091, 0.0], [24, '[50, 121, 123, 196, 53, 121, 123, 198, 47, 121, 123, 55, 121, 124, 200, 45, 121, 123, 202, 38, 121, 124, 54, 121, 124, 206, 50, 121, 124, 210, 41, 121, 124, 53, 121, 124, 214, 50, 121, 124, 58, 121, 124, 1, 186, 218, 48, 121, 124, 53, 121, 124, 57, 121, 124, 60, 121, 124, 1, 186, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 190, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 194, 218, 52, 121, 124, 57, 121, 124, 60, 121, 124, 198, 218, 51, 121, 124, 57, 121, 124, 60, 121, 124, 202, 218, 51, 121, 124, 54, 121, 124, 60, 121, 124, 206, 218, 50, 121, 123, 55, 121, 124, 212, 44, 121, 123, 208, 50, 121, 123, 210, 54, 121, 123, 212, 57, 121, 123, 214, 60, 121, 123, 216, 59, 121, 123, 1, 186, 60, 121, 123, 198, 62, 113, 127, 202, 51, 113, 127, 210, 53, 113, 135, 1, 194, 51, 113, 127, 202, 48, 113, 127, 55, 113, 127, 210, 51, 113, 124, 214, 62, 121, 123, 216, 56, 121, 123, 1, 186, 48, 121, 124, 190, 55, 121, 122, 191, 57, 121, 122, 192, 56, 121, 123, 236, 198, 50, 121, 123, 200, 51, 121, 123, 202, 54, 121, 123, 204, 57, 121, 123, 206, 54, 121, 123, 208, 58, 121, 123, 210, 57, 121, 123, 212, 60, 121, 123, 214, 55, 121, 123, 216, 60, 121, 123, 1, 186, 53, 121, 123]', 18, 24, 0.1702898550724637, 0.2903225806451613], [25, '[236, 190, 50, 121, 124, 194, 50, 121, 124, 198, 54, 121, 123, 200, 57, 121, 123, 202, 60, 121, 123, 204, 59, 121, 123, 206, 60, 121, 123, 208, 57, 121, 123, 210, 65, 121, 123, 212, 62, 121, 123, 214, 57, 121, 123, 216, 62, 121, 123, 1, 186, 59, 121, 123, 188, 55, 121, 123, 190, 62, 121, 127, 196, 60, 121, 123, 198, 59, 121, 123, 200, 57, 121, 123, 202, 59, 121, 123, 204, 60, 121, 123, 206, 59, 121, 123, 208, 57, 121, 123, 210, 56, 121, 123, 212, 54, 121, 123, 214, 57, 121, 123, 216, 55, 121, 123, 1, 186, 54, 121, 124, 54, 113, 127, 1, 186, 45, 113, 147, 54, 113, 147, 190, 54, 113, 124, 194, 52, 113, 124, 198, 49, 113, 124, 202, 49, 113, 134, 53, 113, 127, 210, 55, 113, 124, 214, 54, 113, 124, 1, 186, 49, 113, 134, 52, 113, 124, 190, 53, 113, 124, 194, 55, 113, 124, 198, 56, 113, 124, 202, 55, 113, 134, 1, 186, 53, 113, 127, 194, 58, 113, 148, 1, 190, 58, 113, 124, 194, 56, 113, 124, 198, 55, 113, 124, 202, 56, 113, 124, 206, 58, 113, 124, 210, 56, 113, 124, 214, 58, 113, 124, 1, 186, 55, 113, 124, 190, 53, 113, 124, 194, 55, 113, 124, 198, 56, 113, 124, 202, 55, 113, 134, 1, 186, 53, 113, 127, 194, 58, 113, 148, 1, 190, 58, 113, 124, 194, 56, 113, 124]', 14, 20, 0.2923728813559321, 0.0], [26, '[58, 117, 123, 1, 186, 51, 117, 127, 60, 117, 127, 194, 48, 117, 127, 58, 117, 127, 202, 48, 117, 127, 56, 117, 127, 210, 46, 117, 127, 55, 117, 126, 216, 56, 117, 123, 1, 186, 218, 46, 117, 127, 49, 117, 127, 53, 117, 134, 56, 117, 134, 202, 49, 117, 131, 216, 53, 117, 123, 1, 186, 48, 117, 131, 54, 117, 134, 188, 54, 117, 123, 190, 48, 117, 124, 55, 117, 124, 194, 48, 117, 124, 60, 117, 124, 198, 48, 117, 124, 56, 117, 124, 202, 48, 117, 123, 55, 117, 127, 204, 49, 117, 123, 206, 51, 117, 123, 208, 53, 117, 123, 210, 48, 117, 124, 214, 53, 117, 124, 60, 117, 124, 1, 186, 53, 117, 123, 188, 56, 117, 123, 190, 53, 117, 123, 192, 56, 117, 123, 194, 44, 117, 127, 53, 117, 123, 196, 55, 117, 123, 198, 51, 117, 123, 200, 55, 117, 123, 202, 218, 48, 117, 127, 56, 117, 127, 235, 196, 44, 117, 123, 53, 117, 123, 198, 48, 117, 123, 56, 117, 123, 200, 44, 117, 123, 60, 117, 123, 202, 44, 117, 123, 60, 117, 123, 204, 46, 117, 123, 61, 117, 123, 206, 44, 117, 123, 60, 117, 123, 208, 46, 117, 123, 55, 117, 123, 210, 46, 117, 123, 58, 117, 123, 212, 46, 117, 123, 55, 117, 123, 214, 46, 117, 123, 55, 117, 123, 216, 44, 117, 123, 53, 117, 123, 1, 186, 41, 117, 123, 50]', 13, 20, 0.2083333333333333, 0.1764705882352941], [27, '[194, 39, 118, 123, 196, 43, 118, 123, 198, 41, 118, 129, 117, 123, 1, 186, 48, 117, 126, 192, 58, 117, 123, 194, 56, 117, 126, 200, 55, 117, 123, 202, 56, 117, 142, 208, 53, 117, 123, 210, 51, 117, 126, 216, 50, 117, 123, 194, 48, 117, 126, 53, 117, 166, 208, 51, 117, 123, 210, 49, 117, 126, 216, 48, 117, 123, 1, 186, 41, 117, 126, 53, 117, 134, 192, 51, 117, 123, 194, 50, 117, 126, 200, 48, 117, 123, 202, 49, 117, 131, 214, 48, 117, 123, 216, 50, 117, 123, 1, 186, 48, 117, 127, 243, 194, 48, 117, 127, 202, 42, 117, 134, 50, 117, 140, 1, 186, 43, 117, 147, 194, 48, 117, 127, 202, 50, 117, 127, 210, 46, 117, 127, 1, 186, 48, 117, 127, 194, 47, 117, 127, 202, 48, 117, 127, 55, 117, 134, 210, 51, 117, 127, 1, 186, 49, 117, 134, 202, 46, 117, 127, 210, 48, 117, 151, 1, 202, 41, 117, 127, 1, 186, 45, 117, 140, 202, 50, 117, 127, 210, 48, 117, 127, 1, 186, 46, 117, 127, 55, 117, 134, 194, 48, 117, 127, 202, 48, 117, 127, 210, 50, 117, 127, 53, 117, 127, 1, 186, 52, 117, 127, 194, 50, 117, 127, 202, 55, 117, 127, 210, 53, 117, 127, 1, 186, 52, 117, 131, 198, 50, 117, 123, 200, 52, 117, 123, 202, 48, 117, 140, 54, 117, 123, 196, 55, 117, 123, 198, 53, 117]', 17, 19, 0.4803030303030303, 0.0], [28, '[194, 50, 116, 128, 196, 55, 116, 123, 198, 47, 112, 123, 206, 47, 112, 123, 208, 48, 112, 123, 210, 54, 112, 123, 212, 47, 112, 123, 214, 52, 112, 123, 216, 44, 112, 123, 1, 186, 45, 112, 123, 188, 49, 112, 123, 190, 54, 112, 123, 192, 45, 112, 123, 194, 43, 112, 123, 196, 47, 112, 123, 198, 49, 112, 123, 200, 47, 112, 123, 202, 49, 112, 123, 204, 47, 112, 123, 206, 49, 112, 123, 208, 45, 112, 123, 236, 186, 50, 112, 123, 188, 49, 112, 123, 190, 50, 112, 123, 192, 47, 112, 123, 194, 52, 112, 123, 196, 47, 112, 123, 198, 47, 112, 123, 200, 47, 112, 123, 202, 49, 112, 123, 204, 47, 112, 123, 206, 49, 112, 123, 208, 45, 112, 123, 210, 50, 112, 131, 1, 186, 47, 112, 129, 188, 45, 112, 123, 190, 47, 112, 123, 192, 45, 112, 123, 194, 43, 112, 123, 196, 47, 112, 123, 198, 49, 112, 123, 200, 47, 112, 123, 202, 49, 112, 127, 210, 54, 112, 123, 212, 52, 112, 123, 214, 218, 45, 110, 129, 203, 47, 110, 122, 204, 49, 110, 122, 205, 50, 110, 122, 206, 52, 110, 122, 207, 53, 110, 122, 208, 55, 110, 122, 209, 57, 110, 122, 210, 227, 45, 110, 122, 52, 110, 122, 55, 110, 122, 215, 52, 110, 122, 57, 110, 122, 216, 227, 45, 110, 123, 49, 110, 122, 52, 110, 122, 55, 110, 122, 215, 52]', 12, 14, 0.1203703703703703, 0.5], [29, '[202, 42, 117, 128, 210, 55, 117, 141, 1, 190, 56, 117, 125, 194, 58, 117, 125, 198, 46, 117, 125, 202, 58, 117, 143, 1, 194, 48, 117, 132, 198, 55, 117, 125, 202, 41, 117, 125, 206, 41, 117, 125, 210, 46, 117, 128, 53, 117, 125, 214, 51, 117, 125, 1, 186, 51, 117, 125, 58, 117, 136, 190, 49, 117, 125, 194, 51, 117, 125, 198, 49, 117, 125, 202, 48, 117, 128, 206, 53, 117, 125, 210, 47, 117, 125, 214, 42, 117, 125, 47, 121, 123, 194, 55, 121, 123, 196, 50, 121, 123, 198, 45, 121, 124, 202, 55, 121, 150, 210, 45, 121, 127, 1, 186, 48, 121, 134, 53, 121, 140, 202, 45, 121, 134, 206, 55, 121, 124, 210, 53, 121, 124, 214, 52, 121, 124, 1, 186, 47, 121, 140, 50, 121, 124, 190, 51, 109, 124, 194, 53, 116, 135, 210, 58, 116, 135, 235, 196, 60, 116, 123, 198, 58, 116, 123, 200, 57, 116, 123, 202, 55, 116, 123, 204, 50, 116, 123, 206, 52, 116, 123, 208, 50, 116, 123, 210, 48, 117, 128, 210, 48, 117, 128, 1, 186, 49, 117, 141, 190, 53, 117, 125, 194, 55, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 214, 65, 117, 125, 1, 186, 64, 117, 125, 190, 65, 117, 125, 194, 64, 117, 125, 198, 62, 117, 125, 202, 63, 117, 125, 206, 63, 117, 125, 210, 57, 117, 128, 1, 186, 62]', 20, 24, 0.4187499999999999, 0.0], [30, '[202, 54, 121, 124, 190, 51, 121, 124, 194, 50, 121, 124, 198, 51, 121, 124, 202, 53, 121, 124, 206, 55, 121, 124, 210, 56, 121, 124, 214, 53, 121, 124, 1, 186, 55, 121, 123, 188, 43, 121, 123, 51, 121, 123, 190, 41, 121, 123, 50, 121, 123, 192, 43, 121, 123, 51, 121, 123, 194, 39, 121, 123, 46, 121, 123, 196, 43, 121, 123, 48, 121, 123, 198, 41, 121, 123, 50, 121, 123, 200, 44, 121, 123, 53, 121, 123, 202, 52, 121, 123, 62, 121, 123, 204, 47, 121, 123, 59, 121, 123, 206, 48, 121, 123, 64, 121, 124, 208, 48, 121, 123, 210, 52, 121, 123, 64, 121, 124, 212, 45, 121, 123, 214, 54, 121, 123, 216, 47, 121, 123, 1, 186, 55, 121, 123, 188, 47, 121, 123, 190, 54, 121, 123, 192, 47, 121, 123, 194, 45, 121, 123, 196, 43, 121, 123, 198, 47, 121, 123, 200, 47, 121, 123, 202, 48, 121, 123, 204, 50, 121, 123, 206, 43, 121, 123, 208, 47, 121, 123, 210, 50, 110, 123, 204, 48, 110, 123, 206, 53, 110, 123, 208, 56, 110, 123, 210, 57, 115, 123, 212, 53, 115, 123, 214, 51, 115, 123, 216, 50, 115, 123, 1, 186, 51, 115, 137, 55, 115, 139, 202, 48, 115, 137, 204, 55, 115, 123, 206, 56, 115, 123, 208, 57, 115, 123, 210, 55, 115, 123, 212, 53, 115, 123, 214, 51, 115, 123, 216, 50, 115]', 19, 25, 0.1875, 0.3846153846153846], [31, '[194, 57, 121, 127, 236, 235, 206, 54, 121, 124, 210, 51, 121, 124, 214, 47, 121, 124, 1, 186, 55, 121, 124, 190, 65, 117, 148, 1, 198, 60, 117, 124, 202, 58, 117, 123, 204, 57, 117, 123, 206, 59, 117, 123, 208, 60, 117, 123, 210, 64, 117, 123, 212, 60, 117, 123, 214, 59, 117, 123, 1, 186, 55, 117, 125, 59, 117, 125, 190, 57, 117, 125, 194, 58, 117, 125, 198, 59, 117, 125, 202, 58, 117, 125, 206, 56, 117, 125, 210, 54, 117, 125, 214, 53, 117, 125, 1, 190, 57, 117, 123, 192, 53, 117, 123, 194, 62, 117, 131, 204, 55, 117, 123, 198, 52, 117, 123, 200, 55, 117, 123, 202, 56, 117, 127, 204, 52, 117, 123, 206, 53, 117, 123, 208, 47, 117, 123, 210, 48, 117, 124, 55, 117, 124, 214, 219, 43, 117, 124, 46, 117, 124, 49, 117, 124, 1, 186, 223, 43, 117, 134, 46, 117, 134, 49, 117, 134, 202, 44, 117, 134, 48, 117, 134, 1, 186, 60, 117, 126, 192, 56, 117, 123, 194, 55, 117, 126, 200, 53, 117, 123, 202, 55, 117, 150, 1, 200, 46, 117, 123, 202, 44, 117, 131, 53, 117, 134, 56, 117, 150, 208, 51, 117, 123, 210, 53, 117, 126, 216, 55, 117, 123, 1, 186, 53, 117, 134, 192, 46, 117, 123, 194, 48, 117, 126, 200, 50, 117, 123, 202, 51, 117, 150, 208, 58, 117, 123, 210, 51, 117]', 20, 22, 0.3669871794871794, 0.0], [32, '[194, 56, 117, 128, 202, 51, 117, 133, 214, 50, 117, 123, 216, 48, 117, 123, 1, 186, 50, 117, 125, 190, 51, 117, 125, 194, 49, 117, 125, 198, 51, 117, 125, 202, 49, 117, 141, 58, 117, 131, 212, 56, 117, 123, 214, 58, 117, 123, 190, 60, 117, 123, 192, 61, 117, 123, 194, 60, 117, 126, 200, 58, 117, 123, 202, 60, 117, 126, 208, 55, 117, 123, 210, 53, 117, 126, 216, 51, 117, 123, 1, 186, 56, 117, 126, 192, 58, 117, 123, 194, 56, 117, 126, 200, 55, 117, 123, 202, 56, 117, 142, 208, 53, 117, 123, 210, 51, 117, 126, 216, 50, 117, 123, 1, 186, 48, 117, 142, 192, 58, 117, 123, 194, 56, 117, 126, 200, 55, 117, 123, 202, 56, 117, 142, 208, 55, 117, 123, 210, 53, 117, 126, 216, 51, 117, 123, 1, 186, 56, 117, 131, 192, 41, 117, 123, 194, 43, 117, 126, 196, 55, 117, 123, 198, 53, 117, 123, 200, 44, 117, 123, 53, 117, 123, 202, 49, 117, 127, 204, 55, 117, 123, 206, 53, 117, 123, 208, 52, 117, 123, 210, 53, 117, 127, 1, 186, 49, 117, 123, 1, 186, 55, 121, 127, 194, 55, 121, 127, 202, 55, 121, 124, 206, 54, 121, 123, 208, 52, 121, 123, 210, 50, 121, 123, 212, 55, 121, 123, 214, 48, 121, 123, 216, 55, 121, 123, 1, 186, 47, 121, 123, 188, 55, 121, 123, 190, 54, 121, 123, 192]', 16, 20, 0.2810734463276835, 1.0], [33, '[194, 58, 117, 128, 202, 60, 117, 132, 206, 53, 117, 125, 210, 55, 117, 125, 214, 56, 117, 125, 1, 186, 55, 117, 143, 61, 117, 155, 1, 198, 54, 117, 123, 200, 55, 117, 123, 202, 64, 117, 124, 206, 218, 55, 117, 124, 212, 58, 117, 123, 214, 55, 117, 123, 216, 58, 117, 123, 1, 186, 53, 117, 123, 188, 55, 117, 123, 190, 48, 117, 123, 192, 49, 117, 123, 194, 51, 117, 123, 196, 53, 117, 123, 198, 49, 117, 123, 200, 53, 117, 123, 202, 49, 117, 127, 204, 49, 117, 123, 206, 48, 117, 123, 208, 46, 117, 123, 210, 44, 117, 127, 212, 50, 117, 123, 214, 48, 117, 123, 216, 46, 117, 123, 1, 186, 48, 117, 139, 204, 50, 117, 123, 206, 48, 117, 123, 208, 50, 117, 123, 210, 47, 117, 127, 212, 43, 117, 123, 214, 41, 117, 123, 216, 43, 117, 123, 1, 186, 44, 117, 127, 194, 43, 117, 127, 196, 53, 117, 123, 198, 51, 117, 123, 200, 53, 117, 123, 202, 48, 117, 127, 204, 49, 117, 123, 206, 48, 117, 123, 208, 46, 117, 123, 194, 48, 117, 127, 196, 50, 117, 123, 198, 48, 118, 145, 53, 118, 145, 56, 118, 131, 214, 58, 118, 123, 216, 60, 118, 123, 1, 186, 58, 118, 123, 188, 60, 118, 123, 190, 58, 118, 123, 192, 56, 118, 123, 194, 58, 118, 133, 206, 59, 118, 123, 208, 57, 118, 123, 210, 55]', 19, 23, 0.2704545454545454, 1.0], [34, '[202, 45, 121, 150, 55, 121, 124, 63, 121, 140, 235, 210, 57, 121, 124, 190, 60, 121, 124, 194, 57, 121, 124, 198, 59, 121, 124, 202, 60, 121, 124, 206, 62, 121, 124, 210, 63, 121, 123, 212, 62, 121, 123, 214, 60, 121, 123, 216, 59, 121, 123, 1, 186, 57, 121, 124, 190, 56, 121, 124, 194, 57, 121, 124, 198, 57, 121, 124, 202, 59, 121, 124, 206, 62, 121, 123, 208, 60, 121, 123, 210, 59, 121, 123, 212, 57, 121, 123, 214, 55, 121, 123, 216, 54, 121, 123, 1, 186, 55, 121, 124, 58, 121, 126, 192, 57, 121, 123, 194, 55, 121, 123, 196, 54, 121, 123, 198, 48, 121, 124, 54, 121, 124, 202, 50, 121, 124, 53, 121, 124, 206, 51, 121, 123, 208, 50, 121, 123, 210, 51, 121, 123, 212, 48, 121, 123, 214, 47, 121, 123, 216, 45, 121, 123, 1, 186, 54, 121, 123, 188, 45, 121, 123, 190, 52, 121, 123, 192, 45, 121, 123, 194, 54, 121, 123, 196, 50, 121, 123, 198, 55, 121, 124, 202, 55, 121, 122, 205, 56, 121, 122, 206, 55, 121, 122, 207, 56, 121, 122, 212, 54, 121, 122, 213, 56, 121, 122, 214, 62, 121, 123, 216, 53, 121, 123, 1, 186, 52, 121, 124, 192, 60, 121, 123, 194, 59, 121, 123, 196, 62, 121, 123, 198, 57, 121, 123, 200, 62, 121, 123, 202, 53, 121, 123, 204, 57, 121, 123, 206, 56]', 16, 18, 0.1736111111111111, 0.0], [35, '[194, 46, 121, 127, 53, 121, 134, 60, 121, 124, 198, 58, 121, 124, 202, 46, 121, 134, 55, 121, 124, 206, 55, 121, 124, 210, 41, 121, 127, 57, 121, 124, 214, 58, 121, 124, 1, 186, 55, 121, 127, 60, 121, 127, 194, 50, 121, 127, 196, 53, 121, 124, 196, 56, 117, 124, 48, 113, 123, 200, 46, 113, 123, 202, 44, 113, 127, 53, 113, 123, 204, 53, 113, 123, 206, 51, 113, 123, 208, 49, 113, 123, 210, 51, 113, 123, 212, 49, 113, 123, 214, 48, 113, 123, 216, 46, 113, 123, 1, 186, 48, 113, 124, 240, 235, 202, 61, 113, 134, 210, 61, 113, 124, 198, 60, 113, 124, 206, 62, 113, 124, 210, 63, 113, 124, 214, 62, 113, 124, 1, 186, 63, 113, 124, 190, 65, 113, 124, 194, 63, 113, 124, 198, 62, 113, 124, 202, 60, 113, 124, 206, 64, 113, 124, 210, 63, 113, 124, 214, 62, 113, 124, 202, 63, 113, 124, 206, 62, 113, 124, 210, 63, 113, 124, 214, 62, 121, 127, 202, 58, 121, 124, 206, 56, 121, 124, 210, 58, 121, 124, 235, 1, 186, 56, 109, 122, 187, 55, 109, 122, 188, 56, 109, 122, 189, 55, 109, 122, 190, 56, 109, 122, 191, 55, 109, 122, 192, 56, 109, 122, 193, 55, 109, 122, 194, 56, 121, 140, 247, 235, 1, 186, 63, 121, 124, 190, 51, 121, 123, 192, 50, 121, 123, 53, 112, 125, 200, 50, 112, 123]', 18, 24, 0.2154088050314465, 0.0], [36, '[147, 1, 186, 46, 117, 134, 202, 51, 117, 134, 55, 117, 134, 1, 186, 44, 117, 127, 194, 43, 117, 127, 202, 48, 117, 134, 1, 186, 43, 117, 134, 202, 51, 117, 134, 55, 117, 134, 1, 186, 53, 117, 127, 58, 117, 134, 194, 48, 117, 127, 202, 50, 117, 127, 202, 48, 117, 127, 210, 55, 117, 127, 1, 186, 57, 117, 127, 194, 55, 117, 127, 202, 50, 117, 134, 57, 117, 127, 210, 58, 117, 127, 1, 186, 55, 117, 134, 57, 112, 137, 202, 50, 112, 129, 53, 115, 123, 194, 52, 115, 123, 196, 50, 115, 123, 198, 48, 115, 123, 200, 47, 115, 123, 202, 45, 115, 125, 48, 115, 123, 204, 50, 115, 123, 206, 47, 115, 123, 55, 115, 125, 208, 45, 115, 123, 210, 47, 115, 123, 212, 50, 115, 123, 214, 48, 115, 123, 216, 47, 115, 123, 1, 186, 45, 115, 129, 48, 115, 123, 188, 50, 115, 123, 190, 52, 115, 123, 192, 48, 115, 123, 194, 45, 115, 123, 196, 52, 115, 123, 198, 50, 115, 123, 200, 48, 115, 123, 202, 47, 115, 129, 206, 50, 121, 123, 208, 48, 121, 123, 210, 50, 121, 123, 212, 47, 121, 123, 214, 54, 121, 123, 216, 47, 121, 123, 1, 186, 55, 121, 123, 188, 47, 121, 123, 190, 54, 121, 123, 192, 47, 121, 123, 194, 55, 121, 123, 196, 47, 121, 123, 198, 218, 50, 121, 123, 55, 121, 129, 64, 121]', 14, 15, 0.3110119047619047, 0.0], [37, '[202, 45, 114, 123, 198, 38, 114, 123, 53, 114, 123, 200, 38, 114, 123, 52, 114, 123, 202, 37, 114, 125, 50, 114, 123, 204, 54, 114, 123, 206, 38, 114, 125, 55, 114, 123, 208, 48, 114, 123, 210, 36, 114, 125, 53, 114, 123, 212, 50, 114, 123, 214, 36, 114, 125, 53, 114, 123, 216, 47, 114, 123, 1, 186, 45, 114, 129, 48, 114, 123, 188, 57, 114, 123, 190, 62, 114, 123, 192, 63, 114, 123, 194, 62, 114, 124, 235, 202, 56, 114, 123, 204, 57, 114, 123, 206, 60, 114, 123, 208, 45, 114, 123, 210, 41, 114, 123, 212, 43, 114, 123, 214, 45, 114, 127, 50, 114, 123, 216, 49, 114, 123, 1, 186, 50, 114, 131, 196, 43, 114, 123, 198, 45, 114, 123, 200, 48, 114, 123, 202, 51, 114, 127, 208, 53, 114, 123, 210, 51, 114, 123, 212, 49, 114, 123, 53, 110, 123, 192, 44, 110, 123, 194, 48, 110, 127, 55, 110, 127, 235, 206, 56, 110, 124, 210, 58, 110, 124, 214, 46, 110, 131, 61, 117, 166, 208, 51, 117, 123, 210, 49, 117, 126, 216, 48, 117, 123, 1, 186, 41, 117, 126, 53, 105, 124, 200, 43, 105, 123, 204, 55, 105, 123, 206, 53, 105, 123, 208, 44, 105, 123, 210, 43, 105, 123, 212, 56, 113, 123, 190, 46, 113, 124, 194, 48, 113, 124, 56, 113, 124, 198, 52, 113, 124, 55, 113, 123, 200, 53, 113]', 24, 27, 0.2419590643274853, 0.0], [38, '[202, 44, 117, 128, 194, 41, 117, 127, 44, 117, 127, 202, 43, 117, 127, 204, 41, 117, 123, 206, 44, 117, 123, 208, 41, 117, 123, 210, 36, 117, 123, 212, 46, 117, 123, 214, 44, 117, 123, 216, 43, 117, 123, 1, 186, 41, 117, 127, 192, 53, 117, 123, 194, 55, 117, 126, 200, 56, 117, 123, 202, 58, 117, 150, 208, 46, 118, 123, 210, 44, 118, 127, 216, 43, 118, 123, 1, 186, 48, 118, 127, 192, 49, 118, 123, 194, 48, 118, 127, 200, 50, 118, 123, 202, 51, 118, 127, 208, 44, 118, 123, 210, 48, 118, 127, 216, 48, 118, 123, 1, 186, 46, 118, 143, 192, 58, 118, 123, 194, 56, 118, 167, 206, 58, 118, 153, 208, 53, 118, 123, 210, 55, 118, 127, 216, 53, 118, 123, 1, 186, 55, 118, 122, 53, 117, 123, 1, 186, 49, 117, 124, 188, 61, 117, 123, 190, 53, 117, 124, 60, 117, 123, 192, 61, 117, 123, 194, 52, 117, 124, 55, 117, 124, 198, 52, 117, 124, 55, 117, 124, 202, 47, 117, 127, 55, 117, 124, 206, 58, 117, 124, 210, 51, 117, 124, 214, 58, 117, 124, 1, 186, 48, 117, 124, 190, 51, 117, 124, 194, 56, 117, 124, 198, 48, 117, 124, 55, 117, 124, 202, 48, 117, 124, 56, 117, 124, 206, 48, 117, 124, 56, 117, 124, 210, 48, 117, 124, 55, 117, 124, 212, 61, 117, 123, 214, 58, 117, 123, 216, 60]', 17, 25, 0.3393939393939388, 0.0], [39, '[202, 53, 117, 141, 1, 198, 56, 117, 125, 202, 58, 117, 125, 206, 56, 117, 125, 210, 54, 117, 125, 214, 53, 117, 125, 1, 186, 54, 117, 135, 190, 50, 117, 125, 194, 52, 117, 125, 198, 53, 117, 125, 202, 51, 117, 128, 210, 60, 117, 133, 1, 186, 50, 117, 141, 190, 60, 117, 125, 194, 58, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 1, 190, 56, 117, 125, 194, 58, 117, 125, 198, 46, 117, 125, 202, 51, 117, 125, 206, 46, 117, 125, 210, 44, 117, 125, 214, 43, 117, 125, 1, 186, 41, 117, 125, 190, 44, 117, 125, 194, 44, 117, 125, 198, 46, 117, 125, 202, 44, 117, 125, 206, 43, 117, 125, 210, 40, 117, 125, 214, 38, 117, 125, 1, 186, 41, 117, 125, 190, 50, 117, 125, 194, 51, 117, 125, 198, 53, 117, 125, 202, 51, 117, 125, 206, 53, 117, 125, 210, 58, 117, 125, 214, 56, 117, 125, 1, 186, 55, 117, 141, 190, 46, 117, 125, 194, 51, 117, 125, 198, 49, 117, 125, 202, 48, 117, 125, 206, 46, 117, 125, 210, 44, 117, 125, 214, 43, 117, 125, 1, 186, 41, 117, 125, 190, 50, 117, 125, 194, 51, 117, 125, 198, 53, 117, 125, 202, 51, 117, 128, 210, 60, 116, 123, 212, 62, 116, 123, 214, 63, 116, 123, 216, 65, 116, 123, 1, 186, 63, 116, 123, 188, 62, 116, 123, 190, 60, 116, 123, 192]', 20, 27, 0.325, 0.6862745098039216], [40, '[202, 42, 118, 129, 46, 118, 129, 206, 56, 118, 129, 214, 54, 118, 127, 1, 188, 51, 118, 123, 190, 50, 118, 123, 192, 51, 118, 123, 194, 49, 118, 123, 196, 48, 118, 123, 56, 117, 123, 1, 186, 55, 117, 123, 188, 56, 117, 123, 190, 58, 117, 123, 192, 56, 117, 123, 194, 55, 117, 123, 196, 53, 117, 123, 198, 52, 117, 123, 200, 55, 117, 123, 202, 48, 117, 135, 54, 117, 135, 1, 186, 50, 117, 140, 194, 44, 117, 127, 210, 48, 117, 127, 1, 186, 46, 117, 127, 55, 117, 134, 194, 48, 117, 127, 202, 50, 117, 127, 210, 51, 117, 127, 1, 186, 48, 117, 127, 198, 58, 117, 127, 202, 49, 117, 127, 204, 56, 117, 123, 206, 54, 117, 123, 208, 53, 117, 123, 210, 52, 117, 123, 212, 49, 117, 123, 53, 117, 123, 214, 48, 117, 123, 55, 117, 127, 216, 48, 117, 123, 1, 186, 49, 117, 126, 192, 51, 117, 123, 194, 49, 117, 126, 200, 48, 117, 123, 202, 49, 117, 124, 1, 186, 60, 117, 126, 192, 58, 117, 123, 194, 56, 117, 126, 200, 55, 117, 123, 202, 56, 117, 142, 208, 48, 117, 123, 210, 50, 117, 126, 216, 51, 117, 123, 1, 186, 56, 117, 131, 192, 41, 117, 123, 194, 43, 117, 126, 196, 55, 117, 123, 198, 53, 117, 123, 200, 44, 121, 123, 200, 48, 121, 123, 202, 46, 121, 124, 206, 46, 121, 124]', 16, 19, 0.2723214285714286, 0.7073170731707317], [41, '[202, 59, 121, 134, 1, 186, 50, 121, 134, 59, 121, 124, 202, 62, 121, 134, 210, 55, 121, 127, 64, 121, 127, 1, 186, 57, 121, 143, 65, 121, 134, 202, 64, 121, 143, 210, 55, 121, 127, 1, 186, 53, 121, 134, 236, 210, 48, 121, 127, 1, 186, 46, 121, 147, 194, 48, 121, 127, 202, 50, 121, 127, 210, 49, 121, 127, 1, 186, 50, 121, 127, 194, 48, 121, 127, 202, 47, 121, 127, 210, 50, 121, 127, 1, 186, 49, 121, 147, 1, 1, 1, 1, 1, 202, 55, 117, 127, 210, 48, 117, 127, 1, 186, 50, 117, 127, 194, 50, 117, 127, 202, 51, 117, 140, 210, 43, 117, 147, 1, 194, 45, 110, 129, 52, 110, 125, 56, 110, 127, 206, 50, 110, 125, 208, 56, 110, 123, 210, 48, 110, 125, 57, 110, 125, 214, 50, 110, 125, 56, 110, 125, 1, 186, 48, 110, 129, 57, 110, 129, 235, 198, 52, 110, 125, 55, 110, 125, 235, 206, 52, 110, 125, 55, 110, 125, 235, 214, 52, 110, 125, 55, 110, 125, 1, 186, 50, 110, 129, 53, 110, 129, 237, 235, 214, 221, 46, 117, 124, 53, 117, 123, 188, 56, 117, 123, 190, 44, 117, 124, 55, 117, 123, 192, 53, 117, 123, 194, 51, 117, 123, 196, 53, 117, 123, 198, 45, 113, 124, 202, 43, 113, 127, 55, 113, 134, 1, 186, 50, 113, 134, 202, 43, 113, 127, 55, 113, 134, 1, 186, 50]', 18, 22, 0.5168269230769232, 0.0], [42, '[202, 62, 117, 153, 1, 202, 60, 117, 139, 1, 186, 60, 117, 154, 1, 190, 58, 117, 123, 192, 60, 117, 123, 194, 58, 117, 123, 196, 56, 117, 123, 198, 58, 117, 123, 200, 61, 117, 123, 202, 56, 117, 127, 61, 117, 127, 210, 55, 117, 162, 1, 54, 121, 123, 196, 51, 121, 123, 198, 49, 121, 123, 200, 51, 121, 123, 202, 48, 121, 124, 235, 210, 54, 121, 124, 235, 1, 186, 54, 121, 124, 235, 196, 49, 121, 123, 198, 47, 121, 123, 200, 49, 121, 123, 202, 44, 121, 123, 204, 49, 121, 123, 206, 47, 121, 123, 208, 49, 121, 123, 210, 53, 121, 123, 212, 49, 121, 123, 214, 47, 121, 123, 216, 49, 121, 123, 1, 186, 46, 121, 123, 188, 51, 121, 123, 190, 50, 121, 123, 192, 51, 121, 123, 194, 46, 121, 123, 196, 44, 121, 123, 198, 43, 121, 123, 200, 41, 121, 123, 202, 43, 121, 124, 206, 46, 121, 124, 210, 51, 121, 127, 1, 186, 46, 121, 124, 190, 55, 121, 124, 196, 56, 121, 123, 198, 58, 121, 123, 200, 60, 121, 123, 202, 61, 121, 123, 204, 63, 121, 123, 206, 65, 121, 123, 208, 63, 121, 123, 210, 61, 121, 123, 212, 60, 121, 123, 214, 58, 121, 123, 216, 61, 121, 123, 1, 186, 60, 121, 124, 190, 53, 121, 124, 194, 58, 121, 124, 198, 49, 121, 127, 206, 58, 121, 124, 210, 48, 121, 123, 212]', 19, 24, 0.2449712643678161, 0.5882352941176471], [43, '[202, 42, 110, 130, 47, 110, 122, 56, 110, 122, 199, 50, 110, 122, 57, 110, 122, 200, 226, 47, 110, 122, 52, 110, 122, 54, 110, 122, 201, 50, 110, 122, 56, 110, 122, 200, 218, 43, 110, 124, 47, 110, 122, 55, 110, 122, 187, 52, 110, 122, 56, 110, 122, 190, 54, 110, 122, 53, 110, 125, 194, 45, 110, 125, 198, 47, 110, 125, 202, 48, 110, 129, 210, 59, 116, 123, 212, 62, 116, 123, 214, 63, 116, 123, 216, 65, 116, 123, 1, 186, 63, 116, 123, 188, 62, 116, 123, 190, 60, 116, 123, 192, 58, 116, 123, 194, 57, 116, 123, 196, 55, 116, 123, 198, 53, 116, 123, 200, 52, 116, 123, 202, 58, 116, 123, 204, 60, 116, 123, 206, 58, 116, 123, 208, 60, 116, 123, 210, 58, 116, 123, 212, 57, 116, 123, 214, 55, 116, 123, 216, 53, 116, 123, 1, 186, 52, 116, 124, 190, 58, 116, 127, 196, 57, 116, 123, 198, 55, 116, 123, 200, 53, 116, 123, 202, 55, 116, 147, 1, 198, 55, 116, 124, 202, 57, 116, 124, 206, 60, 116, 124, 210, 58, 116, 123, 212, 57, 116, 123, 214, 59, 116, 123, 216, 60, 116, 123, 1, 186, 59, 116, 124, 190, 57, 116, 124, 194, 62, 116, 124, 198, 60, 116, 123, 200, 58, 116, 123, 202, 60, 116, 127, 237, 235, 198, 57, 116, 124, 202, 53, 116, 124, 206, 48, 116, 124, 210, 52, 116, 124]', 18, 23, 0.1721698113207546, 0.717391304347826], [44, '[202, 50, 121, 124, 53, 121, 124, 190, 52, 121, 124, 55, 121, 124, 194, 52, 121, 124, 62, 121, 124, 198, 53, 121, 124, 202, 53, 121, 124, 55, 121, 124, 206, 51, 121, 123, 57, 121, 124, 208, 48, 121, 123, 210, 50, 121, 124, 58, 121, 124, 214, 45, 121, 124, 54, 121, 124, 1, 186, 45, 121, 127, 55, 121, 124, 190, 54, 121, 123, 192, 52, 121, 123, 194, 50, 121, 123, 196, 48, 121, 123, 64, 121, 123, 198, 47, 121, 124, 61, 121, 123, 204, 62, 121, 123, 206, 59, 121, 123, 208, 60, 121, 123, 210, 57, 121, 124, 62, 121, 124, 214, 57, 121, 124, 62, 121, 124, 1, 186, 52, 121, 124, 61, 121, 123, 188, 64, 121, 123, 190, 62, 121, 123, 192, 60, 121, 123, 194, 59, 121, 123, 196, 57, 121, 123, 198, 55, 121, 123, 200, 54, 121, 123, 202, 55, 121, 123, 204, 59, 121, 123, 206, 62, 121, 123, 208, 59, 121, 123, 210, 65, 121, 123, 212, 62, 121, 123, 214, 57, 121, 123, 216, 62, 121, 123, 1, 186, 60, 121, 124, 190, 62, 121, 124, 194, 59, 121, 124, 198, 60, 121, 124, 235, 206, 52, 121, 123, 208, 50, 121, 123, 210, 53, 121, 123, 212, 52, 121, 123, 214, 50, 112, 123, 1, 186, 42, 119, 125, 50, 119, 125, 238, 235, 186, 50, 119, 123, 188, 55, 119, 123, 190, 57, 119, 123, 192, 54, 119, 123, 194]', 18, 23, 0.1512345679012346, 0.0], [45, '[58, 116, 123, 208, 57, 116, 123, 210, 58, 116, 123, 212, 57, 116, 123, 214, 55, 116, 123, 216, 53, 116, 123, 1, 186, 52, 116, 124, 190, 48, 116, 124, 196, 60, 116, 123, 198, 58, 116, 123, 200, 57, 116, 123, 202, 55, 116, 123, 204, 50, 116, 123, 206, 52, 116, 123, 208, 49, 116, 123, 210, 48, 116, 123, 212, 46, 116, 123, 214, 45, 116, 123, 216, 43, 116, 123, 1, 186, 46, 116, 123, 188, 43, 116, 123, 190, 47, 116, 123, 192, 50, 116, 123, 194, 51, 116, 124, 198, 43, 116, 124, 202, 45, 116, 124, 206, 53, 116, 124, 210, 43, 116, 124, 214, 52, 116, 124, 1, 186, 41, 116, 124, 198, 53, 116, 124, 202, 52, 116, 124, 206, 52, 116, 123, 208, 53, 116, 123, 210, 55, 116, 123, 212, 53, 116, 123, 214, 55, 116, 123, 216, 52, 116, 123, 1, 186, 53, 116, 123, 188, 52, 116, 123, 190, 53, 116, 123, 192, 55, 116, 123, 194, 53, 116, 123, 196, 55, 116, 123, 198, 57, 116, 123, 200, 53, 116, 123, 202, 55, 116, 124, 206, 52, 116, 123, 208, 53, 116, 123, 210, 52, 116, 123, 212, 53, 116, 123, 214, 55, 116, 123, 216, 53, 116, 123, 1, 186, 52, 116, 123, 188, 48, 116, 123, 190, 52, 116, 123, 192, 55, 116, 123, 194, 48, 116, 124, 198, 53, 116, 124, 202, 52, 116, 124, 206, 57, 116, 123, 208, 55]', 15, 19, 0.1352459016393442, 0.6], [46, '[202, 48, 117, 131, 204, 53, 117, 123, 206, 52, 117, 123, 208, 55, 117, 123, 210, 53, 117, 139, 212, 51, 117, 123, 214, 49, 117, 123, 216, 48, 117, 123, 1, 186, 46, 117, 123, 188, 44, 117, 123, 190, 42, 117, 123, 192, 41, 117, 123, 194, 42, 117, 123, 196, 46, 117, 123, 198, 43, 117, 123, 200, 48, 117, 123, 202, 43, 117, 123, 44, 117, 127, 204, 41, 117, 123, 206, 44, 117, 123, 208, 41, 117, 123, 208, 44, 117, 123, 210, 38, 117, 123, 46, 117, 135, 212, 44, 117, 123, 214, 38, 117, 123, 216, 44, 117, 123, 1, 186, 43, 117, 134, 202, 50, 117, 134, 55, 117, 134, 1, 186, 52, 117, 127, 194, 53, 117, 140, 235, 1, 186, 52, 117, 151, 1, 194, 41, 117, 127, 202, 46, 117, 127, 210, 50, 117, 151, 1, 186, 43, 117, 127, 194, 51, 117, 127, 202, 51, 117, 140, 235, 202, 50, 117, 127, 54, 117, 127, 210, 55, 117, 127, 1, 186, 53, 117, 127, 194, 55, 117, 127, 202, 50, 117, 134, 57, 117, 127, 210, 54, 117, 127, 1, 186, 55, 117, 140, 194, 43, 117, 127, 202, 48, 117, 151, 1, 202, 53, 117, 127, 210, 52, 117, 127, 1, 186, 53, 117, 134, 202, 46, 117, 127, 210, 48, 117, 140, 235, 202, 41, 117, 134, 1, 186, 41, 117, 140, 194, 41, 117, 127, 202, 46, 117, 156, 210, 53, 117, 127, 1]', 15, 19, 0.5242424242424242, 0.4594594594594595], [47, '[202, 58, 117, 128, 210, 60, 117, 149, 1, 190, 53, 117, 125, 194, 55, 117, 133, 198, 59, 117, 125, 202, 58, 117, 125, 206, 56, 117, 125, 210, 54, 117, 125, 214, 53, 117, 125, 1, 186, 52, 117, 135, 190, 57, 117, 123, 194, 46, 117, 124, 53, 117, 123, 196, 54, 117, 123, 198, 46, 117, 124, 53, 117, 123, 200, 54, 117, 123, 202, 53, 117, 139, 61, 117, 131, 212, 65, 117, 123, 214, 61, 117, 123, 216, 65, 117, 123, 1, 186, 62, 117, 123, 188, 62, 117, 123, 190, 55, 117, 123, 192, 60, 117, 123, 194, 57, 117, 123, 196, 60, 117, 123, 198, 57, 117, 123, 200, 60, 117, 123, 202, 41, 117, 127, 56, 117, 123, 204, 60, 117, 123, 206, 58, 117, 123, 208, 56, 117, 123, 210, 55, 117, 123, 212, 53, 117, 123, 214, 51, 117, 123, 216, 49, 117, 123, 1, 186, 48, 117, 123, 188, 49, 117, 123, 190, 48, 117, 123, 192, 51, 117, 123, 194, 49, 117, 134, 53, 117, 127, 202, 55, 117, 127, 210, 51, 117, 127, 1, 186, 56, 117, 127, 188, 49, 117, 123, 190, 48, 117, 123, 192, 51, 117, 123, 194, 49, 117, 123, 55, 117, 124, 196, 46, 117, 123, 198, 45, 117, 123, 48, 117, 123, 200, 48, 117, 123, 51, 117, 123, 200, 63, 117, 123, 202, 53, 117, 139, 61, 117, 131, 212, 65, 117, 123, 214, 61, 117, 123, 216, 65]', 19, 24, 0.2763888888888889, 0.0], [48, '[202, 43, 117, 123, 51, 117, 123, 196, 41, 117, 123, 56, 117, 123, 198, 43, 117, 123, 55, 117, 123, 200, 44, 117, 123, 55, 117, 123, 202, 43, 117, 123, 58, 117, 123, 204, 46, 117, 123, 55, 117, 123, 206, 43, 113, 123, 51, 113, 126, 216, 43, 113, 123, 46, 113, 123, 1, 186, 41, 113, 127, 48, 113, 127, 194, 44, 113, 127, 48, 113, 127, 198, 46, 113, 128, 202, 41, 113, 127, 204, 51, 113, 123, 206, 50, 113, 123, 208, 48, 113, 123, 210, 53, 113, 123, 212, 51, 113, 123, 214, 50, 113, 123, 216, 48, 113, 123, 1, 186, 44, 113, 127, 47, 113, 127, 235, 206, 44, 113, 124, 48, 113, 124, 210, 46, 113, 124, 55, 113, 124, 214, 46, 113, 124, 58, 113, 124, 1, 186, 46, 113, 124, 58, 113, 124, 190, 43, 113, 124, 58, 113, 124, 194, 41, 113, 140, 56, 113, 127, 202, 50, 113, 127, 58, 113, 134, 210, 50, 113, 124, 214, 53, 113, 124, 1, 186, 43, 113, 127, 52, 113, 134, 194, 45, 113, 127, 202, 46, 113, 127, 210, 48, 113, 127, 1, 186, 50, 113, 127, 194, 48, 113, 124, 198, 46, 113, 124, 202, 45, 113, 127, 210, 46, 113, 124, 214, 48, 113, 124, 1, 186, 42, 113, 134, 50, 113, 134, 202, 43, 113, 127, 210, 45, 113, 124, 214, 48, 113, 124, 1, 186, 43, 117, 134, 1, 186, 45, 117, 140, 202]', 15, 17, 0.3197278911564626, 0.0], [49, '[199, 40, 121, 123, 193, 43, 121, 122, 194, 39, 121, 122, 195, 38, 121, 123, 197, 43, 121, 122, 198, 45, 121, 122, 199, 42, 121, 123, 201, 43, 121, 122, 202, 35, 121, 122, 203, 38, 121, 123, 205, 42, 121, 122, 206, 50, 121, 122, 207, 42, 121, 123, 209, 43, 121, 122, 210, 37, 121, 122, 211, 38, 121, 123, 213, 43, 121, 122, 214, 40, 121, 122, 215, 40, 121, 123, 217, 43, 121, 122, 1, 186, 26, 121, 122, 187, 33, 121, 123, 189, 38, 121, 122, 190, 41, 121, 122, 191, 37, 121, 123, 193, 38, 121, 122, 194, 33, 121, 122, 195, 38, 121, 123, 197, 43, 121, 122, 198, 45, 121, 122, 199, 42, 121, 123, 201, 43, 121, 122, 202, 35, 121, 122, 203, 38, 121, 123, 205, 42, 121, 122, 206, 50, 121, 122, 207, 42, 121, 123, 209, 43, 121, 122, 210, 37, 121, 122, 211, 38, 121, 123, 213, 43, 121, 122, 214, 40, 121, 122, 215, 40, 121, 123, 216, 44, 121, 123, 1, 186, 48, 121, 139, 204, 43, 121, 123, 206, 47, 121, 123, 208, 50, 121, 123, 210, 43, 121, 123, 192, 47, 121, 123, 194, 50, 121, 123, 196, 47, 121, 123, 198, 55, 121, 128, 202, 53, 121, 123, 200, 57, 121, 123, 202, 60, 121, 124, 206, 57, 121, 124, 198, 59, 121, 124, 235, 206, 47, 121, 123, 208, 45, 121, 123, 210, 47, 121, 123, 212, 55, 121]', 20, 34, 0.0942982456140351, 0.0], [50, '[202, 221, 45, 117, 129, 48, 117, 129, 52, 117, 129, 236, 210, 221, 48, 117, 129, 52, 117, 129, 236, 210, 45, 112, 129, 54, 112, 129, 1, 186, 47, 112, 129, 55, 112, 129, 194, 52, 112, 131, 196, 40, 112, 123, 198, 45, 112, 123, 200, 40, 112, 123, 202, 42, 112, 129, 204, 45, 112, 123, 206, 50, 112, 123, 208, 45, 112, 123, 210, 42, 112, 123, 212, 45, 112, 123, 214, 38, 112, 123, 216, 35, 112, 123, 1, 188, 51, 105, 123, 194, 53, 105, 123, 196, 48, 105, 122, 198, 57, 105, 122, 200, 48, 105, 123, 204, 48, 105, 123, 206, 54, 105, 123, 208, 48, 105, 122, 210, 54, 105, 123, 212, 50, 105, 123, 216, 50, 105, 123, 1, 186, 42, 105, 123, 188, 45, 105, 123, 190, 48, 105, 123, 192, 57, 105, 123, 235, 202, 48, 117, 123, 53, 117, 124, 212, 51, 117, 123, 206, 49, 117, 123, 55, 117, 124, 208, 51, 117, 123, 210, 46, 117, 123, 212, 49, 117, 123, 214, 48, 117, 123, 216, 51, 117, 123, 1, 186, 49, 117, 123, 188, 53, 117, 123, 190, 51, 117, 123, 192, 53, 117, 123, 194, 51, 117, 123, 196, 54, 117, 123, 198, 53, 117, 123, 200, 51, 117, 123, 202, 49, 117, 123, 55, 117, 124, 204, 51, 117, 123, 206, 49, 117, 123, 55, 117, 124, 208, 51, 117, 123, 210, 48, 117, 123, 212, 49, 117, 123, 214, 48]', 16, 22, 0.1636904761904761, 0.3571428571428571], [51, '[195, 57, 121, 122, 214, 218, 46, 118, 125, 49, 118, 125, 53, 117, 124, 1, 186, 60, 117, 131, 196, 59, 117, 123, 198, 60, 117, 123, 200, 63, 117, 123, 202, 61, 117, 123, 204, 60, 117, 123, 206, 58, 117, 123, 208, 60, 117, 123, 210, 61, 117, 127, 212, 54, 117, 123, 214, 53, 117, 123, 216, 54, 117, 123, 1, 186, 51, 117, 123, 188, 49, 117, 123, 190, 48, 117, 123, 192, 46, 117, 123, 194, 45, 117, 123, 196, 48, 117, 123, 198, 45, 117, 123, 200, 48, 117, 123, 202, 45, 117, 123, 204, 50, 117, 123, 206, 45, 117, 123, 208, 47, 117, 123, 210, 43, 117, 123, 212, 50, 117, 123, 214, 43, 117, 124, 61, 117, 123, 216, 63, 117, 123, 1, 186, 44, 117, 124, 60, 117, 123, 188, 63, 117, 123, 190, 39, 117, 124, 61, 117, 123, 192, 65, 117, 123, 194, 61, 117, 123, 196, 65, 117, 123, 198, 46, 117, 124, 63, 117, 123, 200, 65, 117, 123, 202, 53, 117, 128, 60, 117, 128, 210, 53, 117, 128, 59, 117, 122, 211, 56, 117, 122, 212, 59, 117, 122, 213, 56, 117, 122, 214, 59, 117, 122, 215, 57, 117, 122, 217, 55, 117, 122, 1, 186, 55, 117, 122, 187, 56, 117, 122, 189, 58, 117, 122, 190, 58, 117, 122, 191, 60, 117, 122, 193, 61, 117, 122, 194, 58, 117, 126, 200, 56, 117, 122, 201, 55, 117, 122, 202]', 21, 26, 0.1215846994535519, 0.4761904761904761], [52, '[236, 235, 1, 186, 49, 117, 127, 194, 48, 117, 127, 202, 42, 117, 134, 50, 117, 140, 1, 186, 48, 117, 127, 194, 48, 117, 127, 202, 42, 117, 134, 50, 117, 127, 210, 50, 117, 127, 1, 186, 43, 117, 127, 194, 53, 117, 127, 202, 48, 117, 127, 210, 42, 117, 127, 50, 117, 127, 1, 186, 55, 117, 156, 1, 194, 43, 117, 127, 57, 117, 127, 202, 50, 117, 140, 54, 117, 140, 235, 194, 50, 117, 127, 54, 117, 127, 202, 50, 117, 127, 58, 117, 127, 210, 48, 117, 127, 56, 117, 127, 202, 55, 117, 156, 1, 210, 53, 117, 127, 1, 186, 51, 117, 127, 194, 53, 117, 127, 202, 55, 117, 156, 1, 210, 48, 117, 127, 1, 186, 58, 117, 156, 1, 194, 52, 117, 127, 202, 57, 117, 156, 1, 210, 54, 117, 127, 1, 186, 55, 117, 131, 198, 54, 117, 123, 200, 52, 117, 123, 202, 57, 117, 127, 210, 52, 117, 123, 212, 54, 117, 123, 214, 50, 117, 123, 216, 53, 117, 123, 1, 186, 50, 117, 124, 190, 62, 117, 124, 194, 58, 117, 124, 198, 62, 117, 124, 202, 63, 117, 131, 212, 60, 117, 123, 214, 58, 117, 123, 216, 60, 117, 123, 1, 186, 58, 117, 123, 216, 55, 112, 123, 1, 186, 57, 112, 123, 188, 55, 112, 123, 190, 57, 112, 123, 192, 54, 112, 123, 194, 59, 112, 123, 196, 57, 112, 123, 198, 218, 50, 112]', 17, 21, 0.5243710691823901, 0.4186046511627907], [53, '[39, 121, 122, 188, 56, 117, 123, 190, 46, 117, 124, 55, 117, 123, 192, 56, 117, 123, 194, 46, 117, 124, 53, 117, 123, 196, 54, 117, 123, 198, 46, 117, 124, 53, 117, 123, 200, 54, 117, 123, 202, 48, 117, 124, 51, 117, 124, 206, 219, 51, 117, 124, 55, 117, 124, 212, 53, 117, 123, 56, 117, 123, 214, 52, 117, 123, 55, 117, 123, 216, 50, 117, 123, 56, 117, 123, 1, 186, 52, 117, 123, 61, 117, 123, 188, 52, 117, 123, 64, 117, 123, 190, 55, 117, 123, 64, 117, 123, 192, 50, 117, 123, 58, 117, 123, 194, 43, 117, 123, 52, 117, 131, 196, 46, 117, 123, 198, 45, 117, 123, 200, 46, 117, 123, 202, 48, 117, 123, 204, 56, 117, 123, 206, 54, 117, 123, 208, 53, 117, 123, 210, 52, 117, 123, 212, 49, 117, 123, 53, 117, 123, 214, 48, 117, 123, 55, 117, 127, 216, 45, 117, 123, 1, 186, 48, 117, 126, 192, 49, 117, 123, 194, 48, 117, 126, 200, 46, 117, 123, 202, 48, 117, 126, 216, 56, 117, 123, 1, 186, 218, 46, 117, 127, 49, 117, 127, 53, 117, 134, 55, 117, 134, 236, 235, 214, 46, 117, 124, 61, 117, 124, 1, 186, 54, 117, 147, 188, 55, 117, 123, 190, 53, 117, 123, 192, 52, 117, 123, 194, 53, 117, 131, 204, 51, 117, 123, 206, 49, 117, 123, 208, 48, 117, 123, 210, 49, 117, 131, 1, 186]', 16, 25, 0.220352564102564, 0.5], [54, '[50, 116, 125, 190, 58, 116, 125, 194, 57, 116, 125, 198, 55, 116, 125, 202, 57, 116, 125, 206, 55, 116, 125, 60, 121, 128, 235, 206, 53, 121, 124, 210, 55, 121, 123, 212, 57, 121, 123, 214, 45, 121, 123, 216, 43, 121, 123, 1, 186, 55, 121, 124, 190, 60, 121, 124, 194, 57, 121, 124, 198, 59, 121, 124, 202, 54, 121, 124, 206, 50, 121, 124, 210, 55, 121, 124, 214, 59, 121, 124, 1, 186, 48, 121, 124, 190, 60, 121, 124, 194, 57, 121, 124, 198, 59, 121, 124, 235, 206, 47, 121, 123, 208, 45, 121, 123, 210, 47, 121, 124, 235, 1, 186, 54, 121, 123, 188, 57, 121, 123, 190, 62, 121, 123, 192, 57, 121, 123, 194, 65, 121, 123, 196, 62, 121, 123, 198, 57, 121, 123, 200, 62, 121, 123, 202, 53, 121, 123, 204, 59, 121, 123, 206, 57, 121, 123, 208, 59, 121, 123, 210, 55, 121, 123, 200, 59, 121, 123, 202, 220, 43, 117, 128, 48, 117, 124, 188, 58, 117, 123, 190, 51, 117, 124, 60, 117, 123, 192, 61, 117, 123, 194, 52, 117, 127, 200, 55, 117, 123, 202, 49, 117, 122, 203, 52, 117, 122, 204, 49, 117, 122, 205, 52, 117, 122, 206, 49, 117, 122, 207, 52, 117, 122, 208, 53, 117, 122, 209, 52, 117, 122, 210, 49, 117, 122, 211, 52, 117, 122, 212, 49, 117, 122, 213, 52, 117, 122, 214, 49, 117]', 18, 22, 0.1369047619047619, 0.3333333333333333], [55, '[52, 116, 123, 200, 54, 116, 123, 202, 55, 116, 123, 204, 54, 116, 123, 206, 55, 116, 123, 208, 53, 116, 123, 210, 52, 116, 123, 212, 50, 116, 123, 214, 48, 116, 123, 216, 47, 116, 123, 1, 186, 45, 116, 123, 188, 47, 116, 123, 190, 48, 116, 123, 192, 50, 116, 123, 194, 51, 116, 123, 196, 55, 116, 123, 198, 57, 116, 123, 200, 58, 116, 123, 202, 60, 116, 124, 206, 65, 116, 123, 208, 60, 116, 123, 210, 57, 116, 124, 214, 62, 116, 127, 1, 188, 61, 116, 123, 190, 59, 116, 123, 192, 61, 116, 123, 194, 62, 116, 124, 198, 59, 116, 123, 200, 57, 116, 123, 202, 55, 116, 124, 206, 57, 116, 124, 210, 60, 116, 124, 214, 50, 121, 124, 1, 186, 52, 121, 156, 55, 121, 156, 202, 37, 121, 127, 210, 45, 121, 132, 1, 190, 50, 121, 124, 194, 47, 121, 124, 198, 49, 121, 124, 202, 50, 121, 143, 210, 55, 121, 127, 1, 186, 48, 121, 134, 57, 121, 134, 202, 50, 121, 134, 210, 59, 121, 127, 1, 186, 48, 121, 134, 60, 121, 124, 190, 50, 121, 124, 194, 51, 121, 124, 198, 50, 121, 124, 202, 45, 121, 134, 50, 121, 150, 1, 186, 50, 121, 147, 202, 55, 121, 150, 1, 186, 48, 121, 147, 235, 1, 186, 46, 121, 134, 50, 121, 140, 202, 45, 121, 134, 210, 43, 121, 127, 1, 186, 43, 121, 140, 194]', 20, 28, 0.4058908045977011, 0.6444444444444445], [56, '[122, 217, 50, 118, 122, 1, 186, 44, 118, 129, 51, 118, 129, 188, 56, 118, 123, 190, 55, 118, 123, 192, 53, 118, 123, 194, 51, 118, 123, 196, 49, 118, 123, 198, 46, 118, 123, 200, 49, 118, 123, 202, 43, 118, 125, 46, 118, 125, 206, 48, 118, 125, 210, 41, 118, 129, 44, 118, 129, 214, 59, 118, 129, 1, 186, 44, 118, 129, 190, 48, 118, 129, 194, 44, 118, 129, 198, 51, 118, 127, 202, 46, 117, 127, 60, 117, 134, 194, 44, 117, 127, 56, 117, 127, 202, 46, 117, 127, 55, 117, 134, 58, 117, 134, 210, 48, 117, 124, 214, 58, 117, 124, 1, 186, 48, 117, 124, 190, 65, 117, 124, 194, 46, 121, 122, 188, 45, 121, 123, 190, 47, 121, 124, 194, 50, 121, 126, 200, 59, 121, 122, 201, 57, 121, 122, 202, 49, 121, 122, 203, 52, 121, 122, 204, 50, 121, 122, 205, 52, 121, 122, 206, 53, 121, 122, 207, 52, 121, 122, 208, 53, 121, 123, 210, 62, 121, 123, 212, 59, 121, 123, 214, 56, 121, 123, 216, 59, 121, 123, 1, 186, 52, 121, 123, 188, 55, 121, 123, 190, 60, 121, 123, 192, 57, 121, 123, 194, 54, 121, 123, 196, 57, 121, 123, 198, 62, 121, 123, 200, 56, 121, 123, 202, 57, 121, 124, 62, 121, 124, 206, 55, 121, 124, 60, 121, 123, 208, 59, 121, 123, 210, 55, 121, 124, 60, 121, 124, 214, 52, 121]', 21, 24, 0.1988993710691823, 0.0], [57, '[202, 55, 117, 125, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 48, 117, 128, 194, 58, 117, 133, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 47, 117, 128, 194, 56, 117, 133, 206, 61, 117, 153, 1, 202, 60, 117, 153, 1, 202, 58, 117, 130, 210, 56, 117, 130, 1, 186, 58, 117, 129, 194, 56, 117, 130, 202, 61, 117, 153, 1, 202, 55, 117, 138, 1, 186, 53, 117, 147, 235, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 202, 41, 113, 127, 48, 113, 127, 210, 50, 113, 135, 59, 116, 127, 235, 198, 57, 116, 124, 202, 53, 116, 124, 206, 48, 116, 124, 210, 53, 116, 131, 1, 188, 46, 116, 123, 190, 48, 116, 123, 192, 50, 116, 123, 194, 51, 116, 124, 198, 43, 116, 124, 202, 45, 116, 124, 206, 53, 116, 124, 210, 43, 116, 124, 214, 52, 116, 124, 1, 186, 41, 121, 123, 48, 121, 124, 47, 121, 124, 202, 43, 121, 127, 49, 121, 124, 206, 50, 121, 124, 210, 37, 121, 127, 57, 121, 124, 214, 55, 121, 124, 1, 186, 53, 121, 124, 190, 52, 121, 124, 194, 50, 121, 124, 198, 49, 121, 124, 202, 50, 121, 134, 1, 186, 48, 121, 134, 190, 43, 121, 124, 194, 45, 121, 124, 198, 42, 121, 124, 202, 39, 121, 124, 198, 43, 121, 124, 202]', 21, 24, 0.4344135802469127, 0.4761904761904761], [58, '[194, 52, 116, 128, 202, 53, 116, 135, 210, 58, 116, 128, 1, 186, 48, 116, 128, 194, 52, 116, 128, 202, 53, 121, 122, 205, 55, 121, 122, 206, 57, 121, 122, 207, 59, 121, 122, 208, 61, 121, 123, 210, 55, 112, 123, 212, 47, 112, 123, 214, 52, 112, 123, 216, 44, 112, 123, 1, 186, 45, 112, 123, 188, 49, 112, 123, 190, 54, 112, 123, 192, 45, 112, 123, 194, 42, 112, 123, 196, 45, 112, 123, 198, 47, 112, 123, 200, 49, 112, 123, 202, 49, 112, 123, 204, 47, 112, 123, 206, 49, 112, 123, 208, 54, 112, 123, 210, 51, 112, 123, 212, 54, 112, 123, 214, 50, 112, 123, 216, 52, 112, 123, 1, 186, 49, 112, 123, 188, 50, 112, 123, 190, 52, 112, 123, 192, 47, 112, 123, 194, 49, 112, 129, 204, 55, 112, 123, 206, 49, 112, 123, 208, 50, 112, 123, 210, 49, 112, 125, 214, 54, 112, 123, 216, 49, 112, 123, 1, 186, 50, 112, 123, 188, 49, 112, 123, 190, 50, 112, 123, 192, 47, 112, 123, 194, 47, 112, 123, 196, 50, 112, 123, 198, 47, 112, 123, 200, 43, 112, 131, 214, 45, 112, 123, 216, 47, 112, 123, 1, 186, 48, 112, 123, 188, 51, 112, 123, 190, 50, 112, 123, 192, 47, 112, 123, 194, 49, 112, 129, 196, 44, 112, 123, 198, 43, 112, 123, 200, 41, 112, 123, 202, 38, 112, 123, 204, 40, 112, 129, 47]', 20, 23, 0.1741803278688524, 0.4642857142857143], [59, '[194, 52, 117, 128, 236, 235, 214, 57, 121, 124, 1, 186, 55, 121, 124, 60, 121, 124, 190, 53, 121, 124, 62, 121, 124, 194, 55, 121, 124, 60, 121, 124, 198, 57, 121, 124, 60, 121, 124, 202, 43, 121, 134, 55, 121, 127, 210, 48, 121, 127, 57, 121, 127, 1, 186, 50, 121, 140, 55, 121, 127, 194, 57, 121, 127, 202, 58, 121, 134, 210, 52, 121, 124, 214, 50, 121, 124, 1, 186, 48, 112, 125, 200, 47, 112, 123, 202, 47, 121, 123, 206, 45, 121, 123, 208, 47, 121, 123, 210, 48, 121, 125, 214, 46, 121, 125, 1, 186, 45, 121, 123, 188, 50, 121, 123, 190, 58, 121, 123, 192, 57, 121, 123, 194, 54, 121, 123, 196, 52, 121, 123, 198, 60, 121, 124, 202, 59, 121, 124, 206, 62, 121, 123, 208, 60, 121, 123, 210, 59, 121, 123, 212, 57, 121, 123, 214, 55, 121, 123, 216, 54, 121, 123, 1, 186, 55, 121, 124, 190, 55, 121, 124, 194, 60, 121, 132, 206, 53, 121, 124, 210, 55, 121, 124, 214, 59, 121, 124, 1, 186, 60, 121, 124, 190, 62, 121, 124, 194, 59, 121, 124, 198, 57, 121, 124, 202, 55, 121, 124, 206, 58, 121, 123, 208, 57, 121, 123, 210, 60, 121, 132, 1, 190, 53, 121, 123, 192, 52, 121, 123, 194, 50, 121, 124, 198, 55, 121, 124, 61, 121, 123, 192, 57, 121, 123, 60, 121, 123, 204, 57]', 16, 19, 0.2348484848484846, 0.5581395348837209], [60, '[202, 48, 117, 139, 1, 186, 60, 117, 127, 190, 58, 117, 123, 192, 56, 117, 123, 194, 55, 117, 123, 196, 53, 117, 123, 198, 51, 117, 123, 200, 54, 117, 123, 202, 58, 117, 123, 204, 44, 121, 123, 198, 50, 121, 123, 200, 44, 121, 123, 202, 50, 121, 123, 204, 48, 121, 123, 206, 50, 121, 123, 208, 47, 121, 123, 210, 48, 121, 147, 214, 59, 121, 124, 1, 186, 62, 121, 124, 190, 60, 121, 124, 194, 57, 121, 124, 198, 59, 121, 124, 202, 60, 121, 124, 206, 62, 121, 124, 210, 63, 121, 124, 214, 60, 121, 124, 1, 186, 65, 121, 134, 190, 60, 121, 124, 194, 58, 121, 124, 198, 58, 121, 124, 202, 57, 121, 124, 206, 48, 121, 124, 210, 50, 121, 124, 214, 44, 121, 124, 1, 186, 43, 121, 127, 194, 45, 121, 124, 198, 46, 121, 124, 202, 45, 121, 127, 210, 43, 121, 124, 214, 42, 121, 124, 1, 186, 43, 121, 124, 190, 45, 121, 124, 194, 46, 121, 134, 210, 45, 121, 127, 1, 186, 46, 121, 147, 214, 43, 121, 124, 1, 186, 42, 121, 124, 190, 43, 121, 124, 194, 45, 121, 124, 57, 121, 124, 198, 46, 121, 124, 55, 121, 124, 202, 48, 121, 124, 57, 121, 124, 206, 58, 121, 123, 208, 57, 121, 123, 210, 55, 121, 124, 214, 53, 121, 124, 1, 186, 52, 121, 127, 55, 121, 156, 194, 40, 121, 124, 198, 41]', 22, 25, 0.3140432098765431, 0.6571428571428571], [61, '[202, 48, 121, 150, 55, 121, 134, 1, 186, 53, 121, 124, 190, 52, 121, 124, 194, 50, 121, 134, 210, 52, 121, 127, 1, 186, 48, 121, 127, 53, 121, 124, 190, 50, 121, 124, 194, 55, 121, 124, 198, 55, 121, 124, 202, 57, 121, 127, 60, 121, 150, 210, 52, 121, 127, 1, 186, 57, 121, 127, 194, 53, 121, 124, 198, 55, 121, 124, 202, 57, 121, 134, 210, 49, 121, 127, 1, 186, 50, 121, 134, 194, 57, 121, 127, 202, 53, 121, 143, 210, 46, 121, 132, 1, 190, 50, 113, 126, 192, 56, 113, 123, 194, 58, 113, 126, 200, 55, 113, 123, 202, 53, 113, 124, 206, 46, 113, 123, 55, 113, 123, 208, 48, 113, 123, 56, 113, 123, 210, 44, 113, 123, 53, 113, 123, 212, 44, 113, 123, 53, 113, 123, 214, 43, 113, 123, 58, 117, 123, 206, 56, 117, 123, 208, 58, 117, 123, 210, 55, 117, 123, 212, 54, 117, 123, 214, 52, 117, 123, 216, 54, 117, 123, 1, 186, 54, 117, 134, 202, 55, 117, 127, 243, 210, 52, 117, 127, 1, 186, 55, 117, 127, 194, 49, 117, 127, 202, 50, 117, 127, 210, 57, 117, 127, 1, 186, 57, 117, 127, 194, 55, 117, 127, 202, 50, 117, 134, 57, 117, 127, 210, 58, 117, 123, 212, 56, 117, 123, 214, 55, 117, 123, 216, 53, 117, 126, 1, 186, 43, 117, 127, 188, 56, 117, 123, 190, 55, 117, 123, 192]', 14, 17, 0.4078525641025641, 0.3448275862068966], [62, '[202, 45, 105, 153, 1, 1, 196, 50, 117, 123, 212, 51, 117, 123, 214, 53, 117, 123, 216, 51, 117, 123, 1, 186, 58, 117, 127, 194, 51, 117, 127, 60, 117, 127, 202, 50, 117, 127, 57, 117, 140, 210, 50, 117, 127, 1, 186, 45, 117, 140, 194, 60, 117, 127, 202, 58, 117, 127, 210, 55, 117, 127, 62, 112, 137, 206, 57, 112, 125, 210, 54, 112, 129, 1, 186, 49, 112, 129, 52, 112, 137, 194, 49, 112, 129, 202, 47, 112, 129, 52, 112, 137, 210, 45, 112, 129, 1, 186, 47, 112, 129, 52, 112, 137, 202, 45, 112, 129, 210, 45, 112, 129, 1, 186, 44, 112, 129, 47, 112, 129, 194, 45, 112, 129, 51, 112, 125, 198, 55, 112, 125, 202, 50, 112, 129, 52, 112, 129, 210, 45, 112, 129, 50, 112, 129, 1, 186, 43, 112, 129, 49, 112, 129, 194, 42, 112, 125, 49, 112, 129, 198, 40, 112, 125, 202, 42, 112, 129, 206, 43, 112, 129, 210, 38, 112, 129, 214, 43, 112, 125, 1, 186, 41, 112, 125, 194, 62, 117, 131, 204, 56, 117, 123, 206, 54, 117, 123, 208, 53, 117, 123, 210, 44, 118, 123, 204, 53, 118, 123, 206, 46, 118, 123, 55, 118, 133, 55, 104, 125, 235, 204, 55, 113, 123, 206, 53, 113, 123, 208, 51, 113, 123, 210, 53, 113, 123, 212, 55, 113, 123, 214, 53, 113, 123, 216, 51, 113, 123, 1, 186]', 21, 24, 0.4401515151515151, 0.0], [63, '[202, 38, 117, 128, 206, 56, 117, 125, 194, 55, 117, 125, 198, 53, 117, 125, 202, 52, 117, 125, 206, 50, 117, 129, 210, 47, 117, 125, 214, 47, 117, 125, 1, 186, 48, 117, 125, 190, 60, 117, 125, 194, 62, 117, 125, 198, 63, 117, 125, 202, 62, 117, 125, 206, 60, 117, 125, 210, 50, 117, 128, 59, 117, 125, 214, 62, 117, 125, 1, 186, 64, 117, 125, 190, 65, 117, 125, 194, 64, 117, 125, 198, 62, 117, 125, 202, 60, 117, 125, 206, 58, 117, 125, 210, 56, 117, 125, 214, 55, 117, 125, 1, 186, 57, 117, 125, 190, 58, 117, 125, 194, 60, 117, 125, 198, 61, 117, 125, 202, 60, 117, 125, 206, 58, 117, 125, 210, 55, 117, 135, 1, 186, 53, 117, 125, 190, 55, 117, 125, 194, 53, 117, 135, 198, 60, 117, 123, 200, 58, 117, 123, 202, 56, 117, 123, 204, 58, 117, 123, 206, 56, 117, 123, 208, 55, 117, 123, 210, 53, 117, 123, 212, 58, 117, 123, 214, 56, 117, 123, 216, 55, 117, 123, 1, 186, 53, 117, 125, 190, 55, 117, 125, 194, 53, 117, 135, 202, 50, 117, 135, 55, 117, 135, 1, 186, 53, 117, 147, 235, 1, 186, 51, 117, 155, 1, 194, 51, 117, 130, 202, 53, 117, 130, 210, 55, 117, 130, 1, 186, 56, 117, 129, 194, 53, 117, 130, 202, 58, 117, 129, 210, 56, 117, 130, 1, 186, 58, 117, 129, 194]', 18, 27, 0.3706896551724138, 0.0], [64, '[132, 208, 51, 113, 123, 210, 48, 113, 127, 212, 56, 113, 123, 214, 58, 113, 123, 216, 60, 113, 123, 1, 186, 61, 113, 127, 235, 196, 50, 113, 123, 198, 48, 113, 123, 200, 46, 113, 123, 202, 44, 113, 127, 53, 113, 123, 204, 56, 113, 123, 206, 58, 113, 123, 208, 60, 113, 123, 210, 58, 113, 123, 212, 56, 113, 123, 214, 55, 113, 123, 216, 53, 113, 123, 1, 186, 51, 113, 123, 188, 55, 113, 123, 190, 51, 113, 123, 192, 56, 113, 123, 194, 51, 113, 127, 198, 56, 113, 127, 202, 48, 113, 127, 204, 56, 113, 123, 206, 58, 113, 123, 208, 60, 113, 123, 210, 58, 113, 123, 212, 56, 113, 123, 214, 55, 113, 123, 216, 53, 113, 123, 1, 186, 51, 113, 123, 188, 55, 113, 123, 190, 51, 113, 123, 192, 49, 113, 123, 194, 48, 113, 123, 196, 49, 113, 123, 198, 48, 113, 123, 200, 46, 113, 123, 202, 44, 113, 127, 53, 113, 123, 204, 53, 113, 123, 206, 51, 113, 123, 208, 49, 113, 123, 210, 51, 113, 123, 212, 49, 113, 123, 214, 48, 113, 123, 216, 46, 113, 123, 1, 186, 48, 113, 124, 240, 235, 202, 61, 117, 134, 210, 58, 117, 135, 1, 186, 43, 117, 147, 52, 117, 127, 194, 53, 117, 127, 202, 55, 117, 127, 210, 53, 117, 127, 1, 186, 48, 117, 136, 53, 117, 127, 194, 55, 117, 127, 202, 50, 117, 134]', 14, 18, 0.2513661202185792, 0.0], [65, '[202, 46, 117, 128, 202, 46, 117, 160, 1, 198, 48, 117, 124, 202, 46, 117, 124, 206, 49, 117, 124, 210, 47, 117, 124, 214, 45, 117, 124, 1, 186, 45, 117, 147, 50, 117, 127, 194, 52, 117, 127, 202, 53, 117, 127, 210, 47, 117, 127, 1, 186, 53, 117, 147, 188, 51, 117, 134, 55, 121, 123, 198, 47, 121, 123, 200, 48, 121, 123, 202, 57, 121, 123, 204, 55, 121, 123, 206, 53, 121, 123, 208, 52, 121, 123, 210, 53, 121, 122, 212, 52, 121, 123, 214, 53, 121, 124, 1, 188, 43, 121, 123, 190, 47, 121, 123, 192, 50, 121, 123, 194, 62, 121, 124, 198, 62, 121, 124, 202, 62, 121, 124, 206, 60, 121, 124, 62, 121, 124, 210, 55, 121, 124, 64, 121, 124, 214, 52, 121, 124, 61, 121, 124, 1, 186, 53, 121, 123, 62, 121, 123, 188, 50, 121, 123, 190, 57, 121, 123, 192, 58, 121, 123, 194, 57, 121, 124, 198, 55, 121, 124, 202, 54, 121, 123, 204, 64, 121, 123, 206, 62, 121, 123, 208, 60, 121, 123, 210, 59, 121, 124, 214, 59, 121, 122, 215, 57, 121, 122, 216, 59, 121, 122, 217, 57, 121, 122, 1, 186, 55, 121, 122, 187, 54, 121, 122, 188, 52, 121, 123, 190, 51, 114, 123, 202, 53, 114, 123, 204, 56, 114, 123, 206, 55, 114, 123, 208, 53, 114, 123, 210, 55, 114, 131, 1, 186, 60, 114, 123, 188]', 20, 21, 0.2135416666666665, 0.6470588235294118], [66, '[202, 42, 121, 129, 203, 43, 121, 123, 217, 43, 121, 122, 1, 186, 29, 121, 122, 187, 33, 121, 123, 189, 38, 121, 122, 206, 40, 121, 122, 207, 37, 121, 123, 209, 38, 121, 122, 210, 31, 121, 122, 211, 37, 121, 123, 213, 40, 121, 122, 214, 46, 121, 122, 215, 45, 121, 123, 217, 43, 121, 122, 1, 186, 33, 121, 122, 187, 38, 121, 123, 189, 41, 121, 122, 190, 45, 121, 122, 191, 37, 121, 123, 193, 38, 121, 122, 194, 28, 121, 122, 195, 33, 121, 123, 197, 38, 121, 122, 198, 43, 121, 122, 199, 45, 121, 123, 201, 43, 121, 122, 202, 35, 121, 122, 203, 38, 121, 123, 205, 42, 121, 122, 206, 50, 121, 122, 207, 42, 121, 123, 209, 43, 121, 122, 210, 36, 121, 122, 211, 42, 121, 123, 213, 45, 121, 122, 214, 50, 121, 122, 215, 42, 121, 123, 217, 43, 121, 122, 1, 186, 38, 121, 129, 187, 41, 121, 123, 189, 45, 121, 123, 190, 46, 121, 123, 192, 45, 121, 123, 194, 47, 121, 123, 196, 50, 121, 123, 198, 48, 121, 123, 200, 47, 121, 123, 202, 45, 121, 124, 235, 210, 57, 121, 131, 1, 188, 50, 121, 123, 190, 52, 121, 123, 192, 54, 121, 123, 194, 55, 121, 123, 196, 57, 121, 123, 198, 54, 121, 123, 200, 57, 121, 123, 202, 48, 121, 123, 204, 52, 121, 123, 206, 57, 121, 123, 208, 52, 121, 123, 210]', 21, 29, 0.0949453551912568, 0.2244897959183673], [67, '[202, 41, 117, 123, 43, 117, 123, 204, 49, 117, 123, 53, 117, 123, 206, 48, 117, 123, 52, 117, 123, 208, 48, 117, 123, 51, 117, 123, 212, 50, 117, 123, 53, 117, 123, 214, 48, 117, 123, 52, 117, 124, 216, 50, 117, 123, 1, 186, 48, 117, 123, 52, 117, 124, 188, 49, 117, 123, 190, 46, 117, 123, 52, 117, 124, 192, 48, 117, 123, 194, 58, 117, 123, 196, 41, 117, 123, 56, 117, 123, 198, 43, 117, 124, 55, 117, 123, 200, 54, 117, 123, 202, 48, 117, 124, 52, 117, 123, 204, 54, 117, 123, 206, 48, 117, 124, 55, 117, 123, 208, 56, 117, 123, 210, 44, 117, 124, 53, 117, 123, 212, 53, 117, 123, 214, 41, 117, 124, 206, 55, 121, 124, 210, 45, 121, 124, 214, 54, 121, 124, 1, 186, 53, 121, 124, 190, 50, 121, 124, 194, 50, 121, 124, 198, 62, 121, 124, 202, 55, 121, 124, 64, 121, 124, 206, 64, 121, 124, 210, 64, 121, 124, 214, 48, 121, 124, 64, 121, 124, 1, 186, 52, 121, 124, 61, 121, 124, 190, 57, 121, 124, 60, 121, 124, 194, 62, 121, 124, 198, 52, 121, 124, 62, 121, 124, 202, 52, 121, 127, 62, 121, 127, 210, 50, 121, 135, 53, 121, 127, 1, 186, 52, 121, 134, 194, 48, 121, 127, 202, 50, 121, 127, 210, 52, 121, 124, 214, 49, 121, 124, 1, 186, 50, 121, 140, 194, 53, 121, 124, 198]', 20, 23, 0.2197712418300653, 0.0], [68, '[210, 41, 117, 127, 48, 117, 134, 194, 45, 117, 127, 202, 46, 117, 127, 55, 117, 127, 210, 44, 117, 124, 56, 117, 134, 214, 46, 117, 124, 1, 186, 48, 117, 124, 190, 49, 117, 124, 194, 51, 117, 124, 198, 53, 117, 124, 202, 55, 117, 124, 206, 53, 117, 124, 210, 51, 117, 124, 212, 56, 117, 123, 214, 44, 117, 124, 55, 117, 123, 216, 56, 117, 123, 1, 186, 53, 117, 123, 188, 56, 117, 123, 190, 53, 117, 123, 192, 56, 117, 123, 194, 48, 117, 123, 196, 63, 117, 123, 198, 46, 117, 124, 63, 117, 123, 200, 65, 117, 123, 202, 53, 117, 139, 61, 117, 131, 212, 65, 117, 123, 214, 61, 117, 123, 216, 65, 117, 123, 1, 186, 61, 117, 123, 188, 60, 117, 122, 190, 65, 117, 146, 214, 63, 117, 125, 1, 186, 61, 117, 143, 202, 44, 112, 123, 194, 48, 105, 124, 196, 48, 105, 123, 198, 60, 105, 123, 200, 50, 105, 122, 204, 50, 105, 123, 216, 48, 105, 123, 1, 186, 42, 105, 123, 188, 45, 105, 123, 190, 48, 121, 127, 235, 198, 48, 121, 124, 202, 46, 121, 124, 206, 45, 121, 124, 210, 43, 121, 124, 214, 46, 121, 124, 1, 186, 43, 121, 131, 198, 45, 121, 124, 202, 45, 121, 131, 214, 43, 121, 124, 1, 186, 45, 121, 124, 190, 43, 121, 124, 194, 43, 121, 124, 198, 46, 121, 124, 202, 45, 121, 124]', 17, 24, 0.2605932203389829, 0.0], [69, '[210, 62, 112, 125, 235, 202, 50, 112, 131, 212, 56, 112, 123, 214, 55, 112, 123, 216, 53, 112, 123, 1, 186, 52, 112, 123, 188, 55, 112, 123, 190, 56, 112, 123, 192, 58, 112, 123, 194, 60, 112, 123, 196, 58, 112, 123, 198, 56, 112, 123, 200, 58, 112, 123, 52, 110, 123, 192, 50, 110, 123, 194, 45, 110, 125, 48, 110, 123, 196, 50, 110, 123, 198, 43, 110, 125, 47, 110, 123, 200, 48, 110, 123, 202, 42, 110, 129, 48, 110, 127, 235, 214, 53, 110, 131, 216, 43, 117, 123, 1, 186, 39, 117, 147, 43, 117, 134, 202, 44, 117, 131, 212, 48, 117, 123, 214, 46, 117, 123, 216, 44, 117, 123, 1, 186, 51, 117, 127, 194, 53, 117, 127, 56, 117, 127, 202, 49, 117, 127, 55, 117, 127, 210, 48, 117, 127, 56, 117, 127, 1, 186, 55, 117, 123, 192, 56, 117, 123, 194, 63, 117, 131, 204, 55, 117, 123, 206, 53, 117, 123, 208, 52, 117, 123, 210, 53, 117, 126, 216, 51, 117, 123, 1, 186, 56, 117, 150, 192, 49, 117, 123, 194, 48, 117, 131, 204, 51, 117, 123, 56, 117, 123, 190, 46, 117, 124, 55, 117, 123, 192, 56, 117, 123, 194, 46, 117, 124, 53, 117, 123, 196, 54, 117, 123, 198, 46, 117, 124, 53, 117, 123, 196, 54, 117, 123, 202, 46, 117, 124, 53, 117, 123, 204, 54, 117, 123, 206, 48, 117, 124]', 20, 24, 0.2769607843137254, 0.0], [70, '[194, 58, 117, 123, 196, 50, 117, 123, 198, 52, 117, 123, 200, 53, 117, 123, 202, 55, 117, 145, 1, 190, 53, 117, 125, 194, 55, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 1, 190, 56, 117, 125, 194, 58, 117, 125, 198, 46, 117, 125, 202, 44, 117, 125, 206, 43, 117, 125, 210, 49, 117, 132, 1, 186, 48, 117, 135, 53, 117, 128, 194, 52, 117, 128, 236, 210, 48, 116, 128, 1, 186, 53, 116, 128, 194, 53, 116, 128, 202, 55, 116, 128, 210, 53, 116, 128, 1, 186, 51, 116, 128, 194, 50, 116, 125, 198, 51, 116, 125, 202, 48, 116, 125, 206, 60, 116, 128, 214, 48, 116, 128, 1, 186, 53, 116, 135, 59, 116, 125, 206, 61, 105, 125, 214, 55, 105, 125, 1, 186, 53, 105, 125, 190, 52, 105, 125, 194, 53, 105, 125, 198, 55, 105, 125, 198, 57, 105, 125, 202, 55, 105, 125, 206, 53, 105, 125, 210, 55, 105, 125, 214, 57, 105, 125, 1, 186, 58, 105, 126, 190, 57, 105, 125, 194, 55, 105, 125, 198, 57, 105, 125, 202, 55, 105, 125, 210, 57, 105, 126, 214, 54, 105, 125, 1, 186, 49, 117, 132, 1, 186, 50, 117, 125, 190, 53, 117, 125, 194, 58, 117, 125, 198, 60, 117, 125, 202, 58, 117, 125, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 48, 117, 128, 194, 53, 117, 128]', 17, 18, 0.3432203389830508, 0.0], [71, '[39, 121, 123, 56, 121, 123, 200, 45, 121, 123, 202, 38, 121, 123, 204, 43, 121, 123, 206, 41, 121, 123, 208, 40, 121, 123, 192, 38, 121, 123, 194, 36, 121, 129, 52, 121, 122, 195, 51, 121, 122, 196, 52, 121, 131, 206, 50, 121, 129, 53, 121, 131, 210, 49, 121, 129, 212, 58, 121, 123, 214, 57, 121, 123, 216, 55, 121, 123, 1, 186, 54, 121, 123, 188, 48, 121, 123, 190, 57, 121, 127, 196, 55, 121, 123, 198, 55, 121, 123, 200, 54, 121, 123, 202, 55, 121, 139, 1, 188, 52, 121, 123, 190, 53, 121, 123, 192, 55, 121, 123, 194, 50, 121, 123, 196, 53, 121, 123, 198, 50, 121, 123, 200, 48, 121, 123, 202, 47, 121, 123, 204, 45, 121, 123, 206, 47, 121, 123, 208, 45, 121, 123, 235, 198, 64, 112, 124, 202, 57, 112, 124, 235, 210, 62, 112, 127, 216, 60, 112, 123, 1, 186, 59, 112, 122, 187, 60, 112, 122, 188, 62, 112, 124, 192, 56, 112, 123, 194, 57, 112, 122, 195, 59, 112, 122, 196, 60, 112, 122, 197, 62, 112, 122, 198, 63, 112, 124, 204, 54, 112, 123, 206, 55, 112, 122, 207, 57, 112, 122, 208, 58, 112, 122, 209, 60, 112, 122, 210, 61, 112, 124, 216, 52, 112, 123, 1, 186, 53, 117, 123, 60, 117, 145, 192, 52, 117, 123, 194, 53, 117, 125, 235, 204, 51, 117, 123, 198, 56, 117, 123]', 25, 28, 0.1878415300546448, 0.0], [72, '[210, 40, 110, 125, 56, 110, 125, 214, 52, 110, 125, 60, 110, 123, 216, 56, 110, 123, 1, 186, 53, 110, 125, 57, 110, 125, 190, 54, 110, 125, 62, 110, 123, 192, 54, 110, 123, 194, 47, 110, 125, 198, 45, 110, 125, 202, 52, 110, 133, 214, 47, 110, 125, 1, 186, 48, 110, 125, 190, 53, 110, 123, 192, 50, 110, 123, 194, 44, 110, 125, 198, 43, 110, 125, 202, 43, 110, 129, 51, 110, 153, 210, 45, 110, 125, 214, 47, 110, 125, 1, 186, 48, 110, 125, 190, 46, 110, 125, 194, 44, 110, 125, 198, 43, 110, 125, 202, 41, 110, 125, 53, 110, 137, 206, 48, 110, 125, 210, 46, 110, 125, 214, 44, 110, 125, 1, 186, 43, 110, 125, 55, 110, 137, 190, 50, 110, 125, 194, 47, 110, 125, 198, 45, 110, 125, 202, 43, 110, 129, 51, 110, 153, 210, 45, 110, 125, 214, 47, 110, 125, 1, 186, 48, 110, 125, 190, 46, 110, 125, 194, 47, 110, 125, 198, 43, 110, 125, 202, 44, 110, 129, 51, 110, 137, 210, 44, 110, 125, 1, 186, 43, 110, 137, 55, 110, 153, 190, 43, 110, 125, 194, 43, 110, 125, 198, 41, 110, 125, 202, 43, 110, 129, 51, 110, 153, 210, 44, 110, 125, 214, 46, 110, 125, 1, 186, 48, 110, 141, 202, 50, 110, 137, 210, 48, 110, 137, 1, 186, 43, 110, 125, 190, 43, 110, 125, 194, 45, 110, 125, 198]', 18, 22, 0.4676724137931034, 0.0], [73, '[194, 50, 121, 128, 202, 52, 121, 127, 210, 53, 121, 127, 62, 121, 123, 190, 61, 121, 123, 192, 60, 121, 123, 194, 59, 121, 124, 198, 40, 117, 124, 48, 117, 124, 55, 117, 138, 1, 186, 48, 117, 147, 202, 53, 117, 150, 1, 186, 49, 117, 126, 192, 51, 117, 123, 194, 49, 117, 126, 200, 48, 117, 123, 202, 49, 117, 142, 208, 48, 117, 123, 210, 50, 117, 126, 216, 51, 117, 123, 1, 186, 56, 117, 131, 192, 41, 117, 123, 194, 43, 117, 126, 196, 55, 117, 123, 198, 53, 117, 123, 200, 44, 117, 123, 53, 117, 123, 202, 52, 121, 127, 194, 45, 113, 127, 202, 46, 113, 127, 210, 48, 113, 127, 55, 113, 127, 1, 186, 50, 113, 134, 58, 113, 127, 194, 57, 113, 124, 198, 55, 113, 124, 202, 45, 113, 134, 48, 112, 134, 60, 113, 127, 210, 62, 113, 124, 214, 60, 113, 124, 1, 186, 65, 113, 124, 190, 60, 113, 124, 194, 61, 113, 124, 198, 60, 113, 124, 202, 58, 113, 124, 206, 60, 113, 124, 210, 58, 113, 124, 214, 57, 113, 124, 1, 186, 55, 112, 122, 193, 50, 112, 122, 194, 49, 112, 122, 195, 50, 112, 122, 196, 57, 112, 122, 197, 50, 112, 122, 198, 49, 112, 122, 199, 50, 112, 122, 200, 58, 112, 122, 201, 53, 112, 122, 204, 60, 112, 122, 205, 53, 112, 122, 206, 52, 112, 124, 56, 112, 124, 1]', 21, 25, 0.2858187134502922, 1.0], [74, '[202, 57, 115, 125, 206, 50, 115, 125, 210, 62, 115, 133, 1, 190, 60, 115, 123, 192, 59, 115, 123, 194, 60, 115, 125, 198, 55, 115, 125, 202, 56, 113, 124, 1, 186, 51, 113, 127, 55, 113, 127, 194, 218, 48, 113, 127, 56, 113, 127, 56, 113, 127, 202, 49, 113, 127, 55, 113, 127, 210, 48, 113, 127, 60, 113, 127, 1, 186, 50, 113, 140, 57, 113, 127, 194, 55, 113, 124, 198, 53, 113, 124, 202, 52, 113, 134, 210, 45, 113, 127, 1, 186, 218, 43, 113, 127, 48, 113, 127, 194, 43, 113, 127, 196, 51, 113, 123, 198, 53, 113, 123, 200, 50, 113, 123, 202, 41, 113, 127, 46, 113, 124, 202, 45, 113, 127, 62, 113, 124, 206, 60, 113, 124, 210, 46, 113, 127, 58, 113, 124, 214, 63, 113, 124, 1, 186, 60, 113, 124, 190, 58, 113, 124, 194, 61, 113, 124, 198, 60, 113, 124, 202, 58, 113, 124, 206, 60, 113, 124, 210, 58, 113, 124, 214, 57, 113, 124, 1, 186, 55, 113, 134, 194, 51, 113, 134, 202, 53, 113, 134, 210, 50, 113, 135, 1, 186, 51, 113, 147, 194, 48, 113, 124, 198, 46, 113, 124, 202, 45, 113, 134, 48, 113, 143, 1, 186, 49, 113, 159, 194, 57, 113, 124, 198, 55, 113, 124, 202, 56, 113, 124, 206, 58, 113, 124, 210, 56, 113, 124, 214, 58, 113, 124, 1, 186, 55, 113, 124, 190, 53]', 19, 22, 0.4106060606060606, 0.0], [75, '[202, 59, 121, 123, 204, 46, 121, 123, 206, 51, 121, 124, 210, 53, 121, 147, 206, 54, 121, 123, 208, 52, 121, 123, 210, 50, 121, 123, 212, 55, 121, 123, 214, 48, 121, 123, 192, 55, 117, 123, 63, 117, 123, 194, 44, 117, 127, 60, 117, 123, 196, 61, 117, 123, 198, 58, 117, 123, 200, 60, 117, 123, 202, 62, 117, 139, 212, 57, 117, 123, 214, 55, 117, 123, 216, 53, 117, 123, 1, 186, 52, 117, 124, 190, 58, 117, 127, 196, 57, 117, 123, 198, 55, 117, 123, 200, 53, 117, 123, 202, 52, 117, 127, 204, 49, 117, 123, 206, 48, 117, 123, 208, 53, 117, 123, 210, 48, 117, 127, 212, 60, 112, 125, 1, 186, 218, 46, 112, 129, 52, 112, 129, 235, 198, 52, 112, 125, 55, 112, 125, 202, 50, 112, 129, 60, 112, 123, 204, 47, 112, 123, 206, 49, 112, 123, 208, 54, 112, 123, 210, 51, 112, 123, 212, 56, 117, 123, 214, 52, 117, 123, 200, 54, 117, 123, 202, 56, 117, 145, 1, 206, 55, 117, 146, 1, 194, 54, 117, 130, 202, 57, 117, 137, 1, 186, 55, 117, 136, 202, 54, 121, 123, 1, 186, 54, 121, 124, 60, 121, 124, 190, 54, 121, 124, 58, 121, 123, 192, 57, 121, 123, 194, 55, 121, 124, 58, 121, 124, 198, 55, 121, 124, 58, 121, 124, 202, 52, 121, 132, 55, 121, 127, 206, 57, 121, 123, 208, 55, 121, 123]', 19, 19, 0.2790880503144654, 0.3333333333333333], [76, '[202, 43, 112, 123, 204, 47, 112, 123, 206, 52, 112, 123, 208, 47, 112, 123, 210, 43, 112, 123, 212, 47, 112, 123, 214, 52, 112, 123, 216, 47, 112, 123, 235, 190, 60, 118, 125, 210, 222, 51, 118, 125, 54, 118, 125, 60, 118, 125, 64, 118, 125, 198, 218, 55, 118, 125, 60, 118, 125, 64, 118, 125, 202, 218, 55, 118, 125, 60, 118, 125, 64, 118, 125, 206, 218, 55, 118, 125, 60, 118, 125, 64, 118, 125, 210, 221, 57, 118, 125, 62, 118, 125, 214, 55, 118, 125, 64, 118, 125, 1, 186, 59, 118, 123, 188, 57, 118, 123, 190, 59, 118, 123, 192, 60, 118, 123, 194, 51, 118, 129, 198, 55, 118, 127, 202, 53, 118, 129, 204, 55, 118, 123, 206, 53, 118, 123, 208, 51, 118, 123, 210, 46, 118, 125, 212, 55, 118, 123, 214, 60, 118, 123, 216, 61, 118, 123, 1, 186, 60, 118, 123, 188, 58, 118, 123, 190, 56, 118, 123, 192, 55, 118, 123, 194, 56, 118, 123, 196, 58, 118, 123, 198, 55, 118, 123, 200, 56, 118, 123, 202, 43, 118, 145, 58, 118, 135, 214, 63, 118, 123, 216, 62, 118, 123, 1, 186, 65, 118, 123, 188, 64, 118, 123, 190, 62, 118, 123, 192, 60, 118, 123, 194, 59, 118, 123, 196, 57, 118, 123, 198, 55, 118, 123, 200, 57, 118, 123, 202, 54, 118, 123, 204, 57, 118, 123, 206, 51, 118, 123, 208]', 18, 22, 0.2, 0.0], [77, '[202, 55, 117, 136, 1, 186, 54, 117, 147, 235, 1, 1, 1, 1, 1, 194, 51, 113, 124, 198, 52, 113, 124, 202, 46, 113, 134, 48, 113, 127, 210, 50, 113, 124, 214, 48, 113, 124, 1, 186, 47, 113, 124, 190, 51, 113, 124, 194, 53, 113, 124, 198, 55, 113, 124, 202, 56, 113, 124, 206, 58, 113, 124, 210, 60, 113, 124, 214, 218, 40, 113, 124, 49, 113, 124, 58, 113, 124, 1, 186, 218, 41, 113, 124, 48, 113, 124, 56, 113, 124, 190, 218, 49, 113, 124, 55, 113, 124, 194, 44, 113, 124, 53, 113, 124, 198, 218, 43, 113, 124, 46, 113, 124, 52, 113, 124, 202, 218, 41, 113, 127, 48, 113, 127, 56, 113, 127, 210, 218, 46, 113, 127, 55, 113, 127, 58, 113, 127, 1, 186, 222, 51, 113, 127, 55, 113, 127, 58, 113, 127, 1, 186, 222, 51, 113, 127, 55, 113, 127, 56, 113, 127, 194, 49, 113, 127, 58, 113, 127, 202, 48, 113, 127, 60, 113, 127, 210, 37, 113, 127, 60, 113, 127, 63, 113, 127, 210, 48, 113, 127, 59, 113, 127, 1, 186, 50, 113, 134, 62, 113, 127, 194, 58, 113, 147, 198, 56, 113, 124, 202, 55, 113, 134, 210, 45, 113, 124, 214, 47, 113, 124, 1, 186, 48, 113, 134, 51, 113, 134, 202, 48, 113, 134, 1, 186, 53, 113, 124, 190, 55, 113, 124, 194, 56, 113, 124, 198, 55, 113, 124]', 22, 26, 0.3875, 0.0], [78, '[194, 54, 112, 123, 196, 47, 112, 123, 198, 49, 112, 123, 200, 47, 112, 123, 202, 49, 112, 127, 210, 54, 112, 123, 212, 52, 112, 123, 214, 218, 45, 112, 124, 49, 112, 124, 54, 112, 123, 216, 49, 112, 122, 1, 186, 42, 112, 129, 47, 112, 129, 47, 112, 129, 238, 186, 42, 112, 129, 47, 112, 129, 235, 198, 43, 112, 124, 61, 112, 124, 202, 43, 112, 129, 57, 112, 129, 235, 214, 42, 112, 124, 57, 112, 124, 1, 186, 42, 112, 129, 60, 112, 129, 235, 198, 43, 112, 124, 61, 117, 128, 236, 210, 58, 117, 123, 208, 45, 121, 123, 210, 57, 121, 123, 204, 46, 121, 123, 214, 51, 121, 123, 216, 48, 121, 123, 1, 186, 49, 121, 123, 188, 48, 121, 123, 190, 49, 121, 123, 192, 53, 121, 123, 194, 51, 121, 123, 196, 49, 121, 123, 198, 48, 121, 123, 200, 46, 121, 123, 202, 54, 121, 139, 1, 188, 43, 121, 123, 190, 46, 121, 123, 192, 56, 121, 123, 194, 55, 121, 123, 196, 58, 121, 123, 198, 56, 121, 123, 200, 60, 121, 123, 202, 63, 121, 123, 204, 60, 121, 123, 206, 56, 121, 123, 208, 51, 121, 123, 210, 60, 121, 123, 212, 58, 121, 123, 214, 51, 121, 123, 216, 58, 121, 123, 1, 186, 51, 121, 123, 188, 55, 121, 123, 190, 51, 121, 123, 192, 49, 121, 123, 194, 48, 121, 123, 196, 55, 121, 123, 198]', 18, 21, 0.1918238993710692, 0.4186046511627907], [79, '[198, 43, 113, 123, 200, 44, 113, 123, 202, 41, 113, 123, 204, 46, 113, 123, 206, 41, 113, 123, 208, 46, 113, 123, 210, 40, 113, 123, 212, 46, 113, 123, 194, 44, 113, 123, 196, 48, 113, 123, 198, 44, 113, 123, 200, 48, 113, 123, 202, 53, 113, 139, 1, 188, 44, 113, 123, 190, 43, 113, 123, 192, 41, 113, 123, 194, 43, 113, 123, 196, 46, 113, 123, 198, 40, 113, 123, 200, 43, 113, 123, 202, 39, 113, 123, 204, 44, 113, 123, 206, 39, 113, 123, 208, 44, 113, 123, 210, 38, 113, 123, 212, 44, 113, 123, 214, 38, 113, 123, 216, 38, 113, 123, 1, 186, 38, 113, 124, 44, 113, 123, 190, 44, 113, 123, 192, 48, 113, 123, 194, 41, 113, 127, 48, 113, 123, 196, 51, 113, 123, 198, 41, 113, 123, 200, 44, 113, 123, 202, 41, 113, 123, 204, 46, 113, 123, 206, 41, 113, 123, 208, 46, 113, 123, 210, 40, 113, 123, 212, 46, 113, 123, 214, 40, 113, 123, 216, 38, 113, 123, 1, 186, 39, 113, 127, 237, 210, 222, 51, 113, 127, 55, 113, 127, 58, 113, 127, 1, 186, 222, 51, 113, 127, 55, 113, 127, 58, 113, 127, 194, 218, 48, 113, 127, 51, 113, 127, 56, 113, 127, 235, 204, 56, 113, 124, 210, 46, 113, 132, 212, 51, 113, 123, 214, 53, 113, 123, 216, 55, 113, 123, 1, 186, 56, 113, 127, 235, 196, 44, 113]', 13, 20, 0.1826923076923076, 0.1578947368421052], [80, '[55, 105, 125, 235, 202, 60, 105, 126, 210, 59, 105, 125, 214, 62, 105, 122, 216, 53, 105, 122, 1, 186, 55, 105, 125, 235, 194, 51, 105, 129, 202, 46, 105, 137, 1, 186, 45, 105, 137, 235, 190, 45, 105, 126, 194, 49, 105, 125, 198, 52, 105, 125, 202, 55, 105, 146, 235, 194, 54, 105, 129, 202, 53, 105, 137, 1, 194, 51, 105, 138, 206, 53, 115, 123, 57, 115, 123, 200, 47, 112, 123, 202, 45, 112, 123, 55, 112, 129, 238, 214, 218, 48, 118, 123, 52, 118, 123, 55, 117, 122, 212, 49, 117, 122, 213, 51, 117, 122, 214, 49, 117, 122, 215, 51, 117, 122, 216, 49, 117, 122, 217, 48, 117, 122, 1, 186, 49, 117, 126, 192, 51, 117, 123, 194, 49, 117, 126, 200, 48, 117, 123, 202, 49, 117, 124, 1, 192, 42, 117, 123, 206, 42, 117, 123, 208, 40, 117, 123, 210, 53, 117, 127, 1, 186, 51, 117, 127, 55, 117, 127, 188, 48, 117, 123, 56, 117, 123, 204, 44, 118, 127, 200, 46, 118, 123, 202, 48, 118, 137, 52, 121, 123, 216, 56, 121, 123, 1, 186, 48, 121, 123, 188, 46, 121, 123, 190, 48, 121, 123, 192, 56, 121, 123, 194, 49, 121, 123, 196, 48, 121, 123, 198, 49, 121, 123, 200, 56, 121, 123, 202, 48, 121, 127, 235, 212, 60, 121, 123, 214, 58, 121, 123, 216, 56, 121, 123, 1, 186, 55, 121]', 19, 22, 0.2623456790123457, 0.5], [81, '[56, 114, 123, 200, 55, 114, 123, 202, 53, 114, 123, 204, 56, 114, 123, 206, 62, 114, 127, 212, 65, 114, 123, 214, 63, 114, 123, 216, 62, 114, 123, 1, 186, 63, 114, 123, 188, 60, 114, 123, 190, 59, 114, 123, 192, 56, 114, 123, 194, 59, 114, 123, 196, 57, 114, 123, 198, 55, 114, 123, 200, 53, 114, 123, 202, 59, 114, 123, 204, 55, 114, 123, 206, 38, 114, 125, 55, 114, 123, 208, 48, 114, 123, 210, 36, 114, 125, 53, 114, 123, 212, 50, 114, 123, 214, 36, 114, 125, 53, 114, 123, 216, 47, 114, 123, 1, 186, 45, 114, 129, 48, 114, 123, 188, 50, 114, 123, 190, 52, 114, 123, 192, 45, 114, 123, 194, 46, 114, 123, 196, 45, 114, 123, 198, 47, 114, 123, 200, 48, 114, 123, 202, 45, 114, 125, 53, 114, 123, 204, 50, 114, 123, 206, 47, 114, 125, 55, 114, 133, 210, 49, 114, 124, 235, 190, 60, 121, 123, 192, 57, 121, 123, 194, 54, 121, 123, 196, 57, 121, 123, 198, 60, 121, 123, 200, 52, 121, 123, 202, 54, 121, 123, 204, 57, 121, 123, 206, 51, 121, 123, 208, 57, 121, 123, 210, 50, 121, 123, 212, 45, 121, 123, 214, 52, 121, 123, 216, 45, 121, 123, 1, 186, 54, 121, 123, 188, 45, 121, 123, 190, 52, 121, 123, 192, 45, 121, 123, 194, 54, 121, 123, 196, 50, 121, 123, 198, 55, 121, 124, 194]', 20, 29, 0.1545698924731182, 0.3469387755102041], [82, '[194, 55, 112, 129, 202, 52, 112, 129, 55, 112, 137, 210, 49, 112, 129, 1, 186, 50, 112, 129, 55, 112, 129, 194, 49, 112, 129, 54, 112, 137, 202, 47, 112, 129, 210, 48, 112, 129, 54, 112, 137, 1, 186, 50, 112, 125, 194, 52, 112, 125, 198, 50, 112, 125, 202, 52, 112, 125, 206, 53, 112, 125, 210, 50, 112, 125, 214, 53, 112, 125, 1, 186, 52, 112, 145, 190, 43, 112, 125, 194, 54, 112, 137, 198, 49, 112, 125, 202, 47, 112, 125, 206, 45, 112, 125, 210, 43, 112, 125, 214, 46, 112, 125, 1, 186, 44, 112, 123, 188, 43, 112, 123, 190, 44, 112, 123, 192, 43, 112, 123, 194, 45, 112, 122, 195, 48, 112, 122, 196, 52, 112, 122, 197, 53, 112, 122, 198, 52, 112, 124, 203, 48, 112, 122, 204, 47, 112, 122, 205, 48, 112, 122, 206, 50, 112, 122, 207, 51, 112, 122, 212, 60, 117, 125, 214, 55, 117, 125, 1, 186, 53, 117, 125, 190, 51, 117, 125, 194, 53, 117, 125, 198, 54, 117, 125, 202, 53, 117, 125, 206, 51, 117, 125, 210, 49, 117, 125, 214, 48, 117, 125, 1, 186, 49, 117, 141, 190, 53, 117, 125, 194, 55, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 1, 190, 56, 117, 125, 194, 58, 117, 125, 198, 60, 110, 123, 200, 54, 110, 123, 202, 54, 110, 141, 55, 110, 125, 206, 57, 110, 123]', 17, 17, 0.3449453551912568, 0.0], [83, '[202, 46, 112, 129, 52, 112, 129, 210, 50, 112, 129, 235, 190, 49, 112, 129, 52, 112, 129, 202, 52, 112, 129, 55, 112, 137, 210, 49, 112, 129, 210, 55, 112, 129, 1, 186, 50, 112, 129, 55, 112, 129, 194, 49, 112, 129, 54, 112, 129, 202, 52, 112, 129, 55, 112, 137, 210, 52, 112, 137, 1, 186, 54, 112, 137, 194, 50, 112, 137, 202, 52, 112, 137, 210, 53, 112, 137, 1, 186, 51, 112, 137, 194, 48, 112, 129, 202, 47, 112, 137, 210, 56, 112, 129, 1, 186, 50, 112, 127, 194, 50, 112, 122, 195, 52, 112, 122, 196, 50, 112, 122, 197, 52, 112, 122, 198, 55, 112, 123, 200, 60, 112, 125, 204, 57, 112, 122, 205, 55, 112, 122, 206, 57, 112, 123, 208, 54, 112, 123, 210, 59, 112, 123, 212, 57, 112, 123, 214, 55, 112, 123, 216, 54, 112, 123, 1, 186, 53, 112, 123, 188, 57, 112, 122, 189, 60, 112, 122, 190, 65, 112, 124, 195, 53, 112, 122, 196, 55, 112, 122, 197, 57, 112, 122, 212, 55, 116, 123, 212, 57, 116, 123, 214, 55, 116, 123, 216, 57, 116, 123, 1, 186, 55, 116, 124, 190, 57, 116, 123, 192, 59, 116, 123, 194, 60, 116, 124, 198, 62, 116, 124, 202, 55, 116, 124, 206, 59, 116, 124, 210, 57, 116, 124, 214, 62, 116, 124, 1, 186, 56, 116, 124, 190, 52, 116, 124, 194, 57, 116, 132]', 16, 19, 0.3196969696969695, 0.5116279069767442], [84, '[202, 62, 121, 123, 194, 50, 121, 123, 196, 47, 121, 123, 198, 57, 121, 123, 200, 55, 121, 123, 202, 58, 121, 124, 206, 57, 121, 124, 235, 214, 50, 121, 128, 1, 190, 50, 118, 125, 235, 194, 49, 118, 125, 58, 118, 125, 235, 202, 48, 118, 125, 54, 118, 122, 203, 56, 118, 122, 204, 54, 118, 124, 210, 49, 118, 125, 53, 118, 125, 236, 235, 210, 49, 104, 123, 212, 53, 104, 123, 214, 51, 104, 123, 216, 49, 104, 123, 1, 186, 48, 117, 124, 52, 117, 124, 55, 117, 124, 204, 53, 117, 123, 56, 117, 123, 206, 51, 117, 123, 55, 117, 123, 208, 53, 117, 123, 56, 117, 123, 212, 53, 117, 123, 56, 117, 123, 214, 53, 117, 123, 58, 117, 123, 216, 56, 117, 123, 1, 186, 55, 117, 123, 188, 56, 117, 123, 190, 44, 117, 124, 48, 117, 123, 192, 56, 117, 123, 194, 46, 117, 124, 53, 117, 123, 196, 54, 117, 123, 198, 46, 117, 124, 53, 117, 123, 200, 54, 117, 123, 202, 48, 117, 124, 51, 117, 124, 235, 210, 218, 43, 118, 124, 48, 118, 124, 55, 121, 122, 215, 54, 121, 122, 216, 52, 121, 123, 1, 186, 54, 121, 124, 235, 194, 53, 121, 122, 195, 52, 121, 122, 196, 53, 121, 124, 200, 56, 121, 124, 204, 59, 121, 123, 206, 62, 121, 123, 208, 53, 121, 123, 210, 52, 121, 126, 216, 51, 121, 123, 1, 186]', 17, 19, 0.1386217948717948, 0.5483870967741935], [85, '[194, 48, 110, 125, 198, 50, 110, 125, 202, 43, 110, 129, 51, 110, 153, 55, 110, 137, 1, 186, 54, 110, 153, 190, 50, 110, 125, 194, 48, 110, 125, 198, 50, 110, 125, 202, 51, 110, 141, 55, 110, 153, 206, 47, 110, 125, 210, 42, 110, 125, 1, 186, 43, 110, 137, 55, 110, 153, 194, 41, 110, 129, 202, 46, 110, 125, 206, 48, 110, 125, 210, 46, 110, 125, 214, 44, 110, 125, 1, 186, 43, 110, 125, 55, 110, 177, 190, 47, 110, 125, 194, 48, 110, 133, 198, 47, 110, 125, 202, 48, 110, 133, 55, 110, 137, 206, 50, 110, 125, 210, 48, 110, 125, 214, 50, 110, 125, 1, 186, 48, 110, 129, 55, 117, 123, 214, 45, 117, 123, 216, 47, 117, 123, 1, 186, 48, 117, 127, 194, 55, 117, 134, 198, 51, 117, 124, 206, 49, 117, 124, 210, 48, 117, 124, 214, 53, 117, 124, 1, 186, 48, 117, 124, 190, 49, 117, 124, 194, 51, 117, 124, 198, 53, 117, 124, 202, 55, 117, 124, 206, 58, 117, 124, 210, 53, 117, 123, 212, 58, 117, 123, 214, 55, 117, 123, 216, 58, 117, 123, 1, 186, 51, 117, 124, 190, 50, 116, 122, 212, 56, 112, 123, 214, 55, 112, 123, 216, 53, 112, 123, 1, 186, 43, 112, 125, 52, 112, 123, 188, 57, 112, 123, 190, 45, 112, 125, 196, 55, 112, 124, 198, 47, 112, 125, 200, 55, 112, 123, 202, 45]', 18, 17, 0.3516949152542368, 0.0], [86, '[1, 186, 54, 105, 125, 190, 55, 105, 125, 194, 53, 105, 125, 198, 52, 105, 125, 202, 54, 105, 137, 235, 190, 43, 105, 125, 194, 47, 105, 125, 198, 50, 105, 124, 202, 54, 105, 136, 1, 190, 54, 105, 125, 194, 53, 105, 125, 198, 55, 105, 125, 202, 57, 105, 142, 206, 53, 105, 125, 210, 50, 105, 125, 214, 49, 105, 125, 1, 186, 49, 105, 124, 190, 50, 105, 123, 194, 50, 105, 125, 198, 52, 105, 125, 202, 55, 105, 129, 210, 46, 105, 130, 1, 1, 188, 45, 117, 127, 55, 117, 127, 194, 51, 117, 127, 202, 50, 117, 127, 210, 55, 117, 127, 1, 186, 57, 117, 127, 194, 58, 117, 127, 64, 117, 127, 202, 52, 117, 127, 61, 117, 140, 210, 54, 117, 127, 1, 186, 55, 117, 140, 235, 210, 53, 117, 127, 1, 186, 51, 117, 127, 194, 48, 117, 127, 202, 55, 117, 140, 235, 194, 53, 117, 135, 210, 51, 117, 135, 1, 186, 53, 117, 155, 1, 194, 51, 117, 128, 202, 53, 117, 128, 210, 53, 117, 128, 1, 186, 56, 117, 128, 194, 55, 117, 128, 202, 53, 117, 128, 210, 58, 117, 141, 1, 190, 56, 117, 125, 194, 55, 117, 125, 198, 56, 117, 125, 202, 58, 117, 141, 214, 65, 117, 125, 1, 186, 64, 117, 125, 190, 65, 117, 125, 194, 64, 117, 128, 198, 63, 117, 129, 206, 53, 117, 128, 58, 117, 128, 210, 52]', 19, 22, 0.4873511904761904, 0.55], [87, '[202, 54, 121, 124, 190, 50, 121, 124, 236, 235, 206, 50, 121, 124, 210, 52, 121, 124, 214, 54, 121, 124, 1, 186, 55, 121, 124, 190, 50, 121, 124, 241, 235, 214, 59, 121, 124, 1, 186, 57, 121, 124, 65, 121, 124, 190, 60, 121, 123, 192, 59, 121, 123, 194, 61, 121, 123, 196, 62, 121, 123, 198, 59, 121, 123, 200, 62, 121, 123, 202, 53, 121, 123, 204, 57, 121, 123, 206, 54, 121, 123, 208, 57, 121, 123, 210, 50, 121, 123, 212, 59, 121, 123, 214, 57, 121, 123, 216, 60, 121, 123, 1, 186, 50, 121, 123, 188, 53, 121, 123, 190, 57, 121, 123, 192, 53, 121, 123, 194, 62, 121, 123, 196, 57, 121, 123, 198, 53, 121, 123, 200, 57, 121, 123, 202, 50, 121, 123, 204, 54, 121, 123, 206, 57, 121, 123, 208, 54, 121, 123, 210, 60, 121, 123, 212, 57, 121, 123, 214, 54, 121, 123, 216, 57, 121, 123, 1, 186, 50, 121, 123, 188, 55, 121, 123, 190, 59, 121, 123, 192, 55, 121, 123, 194, 64, 121, 123, 196, 60, 121, 123, 198, 55, 121, 123, 200, 60, 121, 123, 202, 52, 121, 123, 204, 55, 121, 123, 206, 60, 121, 123, 208, 57, 121, 123, 210, 60, 121, 123, 212, 57, 121, 123, 214, 54, 121, 123, 216, 57, 121, 123, 1, 186, 50, 121, 123, 188, 55, 121, 123, 190, 59, 121, 123, 192, 62, 121, 123, 194, 60]', 12, 15, 0.1313559322033898, 0.1403508771929824], [88, '[130, 125, 204, 52, 115, 123, 206, 50, 115, 123, 208, 52, 115, 123, 210, 53, 115, 123, 212, 55, 115, 123, 214, 52, 115, 123, 216, 53, 115, 123, 1, 186, 55, 115, 123, 188, 59, 115, 123, 190, 55, 115, 123, 192, 50, 115, 123, 194, 62, 115, 123, 196, 59, 115, 123, 198, 55, 115, 123, 200, 50, 115, 123, 202, 62, 115, 123, 204, 59, 115, 123, 206, 55, 115, 123, 208, 50, 115, 123, 210, 62, 115, 123, 212, 53, 115, 123, 214, 52, 115, 123, 216, 50, 115, 123, 1, 186, 49, 115, 123, 188, 52, 115, 123, 60, 115, 123, 190, 50, 115, 123, 59, 115, 123, 192, 48, 115, 123, 57, 115, 123, 194, 47, 115, 123, 56, 115, 127, 196, 48, 115, 123, 198, 49, 115, 133, 210, 46, 115, 129, 212, 61, 115, 123, 214, 60, 115, 123, 216, 58, 115, 123, 1, 186, 55, 115, 127, 235, 196, 60, 115, 123, 198, 58, 115, 123, 200, 56, 115, 123, 202, 58, 115, 153, 208, 46, 115, 123, 210, 47, 115, 131, 1, 186, 54, 115, 123, 188, 55, 115, 123, 190, 53, 115, 123, 192, 55, 115, 123, 194, 45, 115, 125, 48, 115, 131, 198, 43, 117, 123, 216, 49, 117, 123, 1, 186, 48, 117, 123, 188, 51, 117, 123, 190, 53, 117, 123, 192, 48, 117, 123, 194, 49, 117, 127, 196, 56, 117, 123, 198, 55, 117, 123, 200, 58, 117, 123, 202, 48, 117]', 19, 19, 0.2068965517241379, 0.0], [89, '[202, 44, 117, 127, 206, 48, 117, 127, 64, 121, 123, 190, 62, 121, 123, 192, 60, 121, 123, 194, 59, 121, 123, 196, 64, 121, 123, 198, 62, 121, 123, 200, 64, 121, 123, 202, 45, 121, 123, 60, 121, 150, 204, 45, 121, 123, 206, 47, 121, 123, 208, 50, 121, 123, 210, 53, 121, 123, 212, 52, 121, 123, 214, 53, 121, 147, 190, 52, 121, 123, 192, 55, 121, 123, 194, 64, 121, 123, 196, 60, 121, 123, 198, 62, 121, 123, 200, 64, 121, 123, 202, 45, 121, 123, 60, 121, 150, 204, 48, 121, 123, 206, 47, 121, 123, 208, 45, 121, 123, 210, 47, 121, 123, 212, 45, 121, 123, 214, 43, 121, 123, 216, 41, 121, 123, 1, 186, 32, 121, 123, 188, 41, 121, 123, 190, 40, 121, 123, 192, 38, 121, 123, 194, 36, 121, 129, 52, 117, 127, 1, 186, 55, 117, 140, 235, 210, 53, 113, 123, 188, 45, 113, 123, 49, 113, 123, 190, 48, 113, 123, 51, 113, 123, 192, 48, 113, 123, 51, 113, 123, 194, 48, 113, 123, 56, 113, 127, 235, 204, 51, 113, 123, 206, 49, 113, 123, 208, 48, 113, 123, 210, 46, 113, 123, 212, 44, 113, 123, 214, 43, 113, 123, 216, 49, 113, 123, 1, 186, 48, 113, 123, 188, 46, 113, 123, 190, 51, 113, 123, 192, 53, 113, 123, 194, 51, 113, 123, 196, 55, 113, 123, 198, 53, 113, 123, 200, 51, 113, 123, 202]', 22, 32, 0.2453703703703704, 0.0], [90, '[58, 112, 122, 196, 48, 112, 122, 197, 47, 112, 122, 198, 49, 112, 122, 199, 50, 112, 122, 200, 58, 112, 122, 201, 53, 112, 122, 202, 52, 112, 122, 203, 53, 112, 122, 204, 60, 112, 122, 205, 53, 112, 122, 206, 52, 112, 123, 208, 50, 112, 123, 210, 49, 112, 123, 212, 52, 112, 123, 214, 54, 112, 123, 216, 49, 112, 123, 1, 186, 50, 112, 123, 188, 49, 112, 123, 190, 50, 112, 123, 192, 47, 112, 123, 194, 49, 112, 129, 196, 44, 121, 123, 194, 52, 112, 131, 204, 47, 112, 123, 206, 49, 112, 123, 208, 45, 112, 123, 210, 42, 112, 123, 57, 112, 125, 212, 40, 112, 123, 214, 42, 112, 123, 57, 112, 125, 216, 37, 112, 123, 214, 33, 112, 123, 59, 112, 123, 216, 43, 112, 123, 52, 112, 123, 1, 186, 54, 112, 123, 188, 52, 112, 123, 190, 54, 112, 123, 192, 52, 112, 123, 194, 50, 112, 123, 196, 52, 112, 123, 198, 54, 112, 123, 200, 52, 112, 123, 202, 50, 112, 131, 212, 52, 112, 123, 214, 50, 112, 123, 216, 52, 112, 123, 1, 186, 49, 112, 123, 188, 50, 112, 123, 190, 52, 112, 123, 192, 47, 112, 123, 194, 49, 112, 123, 196, 47, 112, 123, 198, 49, 112, 123, 200, 47, 112, 123, 202, 49, 112, 127, 210, 54, 112, 123, 212, 52, 112, 123, 214, 54, 112, 123, 216, 49, 112, 123, 1, 186, 50, 112]', 18, 27, 0.1336206896551724, 0.0], [91, '[202, 49, 104, 122, 203, 52, 117, 122, 216, 49, 117, 122, 213, 52, 117, 122, 214, 49, 117, 122, 215, 52, 117, 122, 216, 50, 117, 122, 217, 52, 117, 122, 1, 186, 49, 117, 122, 187, 52, 117, 122, 188, 49, 117, 122, 189, 52, 117, 122, 190, 49, 117, 123, 192, 53, 117, 123, 194, 49, 117, 127, 53, 117, 131, 54, 105, 122, 202, 38, 117, 127, 45, 117, 135, 1, 186, 40, 117, 134, 1, 186, 55, 117, 134, 194, 45, 117, 127, 202, 46, 117, 127, 52, 117, 150, 210, 43, 117, 134, 1, 186, 45, 117, 140, 202, 50, 117, 134, 210, 43, 117, 127, 1, 186, 45, 117, 127, 53, 117, 134, 194, 41, 117, 127, 202, 43, 117, 140, 47, 117, 134, 210, 50, 117, 127, 1, 186, 43, 117, 134, 52, 117, 140, 202, 45, 117, 147, 210, 50, 117, 127, 1, 186, 51, 117, 127, 55, 117, 127, 194, 51, 117, 127, 57, 117, 127, 202, 46, 117, 140, 58, 117, 127, 210, 62, 117, 127, 1, 186, 60, 117, 127, 60, 117, 127, 194, 55, 117, 127, 202, 50, 117, 127, 53, 117, 127, 210, 52, 117, 127, 1, 186, 53, 117, 124, 190, 55, 117, 124, 194, 48, 117, 124, 198, 52, 117, 124, 202, 202, 48, 118, 145, 58, 118, 127, 208, 56, 118, 123, 210, 55, 118, 127, 216, 56, 118, 123, 1, 186, 218, 46, 118, 137, 51, 118, 137, 55, 118, 137, 236]', 20, 24, 0.4613636363636365, 0.0], [92, '[50, 110, 123, 204, 53, 110, 123, 206, 60, 110, 123, 208, 56, 110, 123, 210, 57, 110, 123, 212, 50, 110, 123, 214, 58, 110, 123, 216, 65, 110, 123, 235, 190, 52, 110, 123, 192, 60, 110, 123, 194, 48, 110, 123, 196, 53, 110, 123, 198, 61, 110, 123, 200, 65, 110, 123, 236, 194, 46, 110, 123, 196, 49, 110, 123, 198, 49, 110, 123, 200, 48, 110, 123, 202, 43, 110, 123, 204, 40, 110, 123, 206, 45, 110, 123, 208, 47, 110, 123, 210, 40, 110, 129, 44, 110, 127, 216, 44, 110, 123, 1, 186, 40, 110, 127, 45, 110, 125, 53, 110, 137, 190, 50, 110, 125, 194, 47, 110, 125, 198, 45, 110, 125, 202, 44, 110, 141, 1, 186, 50, 110, 137, 190, 45, 110, 125, 194, 47, 110, 125, 198, 50, 110, 125, 202, 51, 110, 153, 1, 186, 51, 110, 137, 190, 43, 110, 125, 194, 45, 110, 125, 198, 47, 110, 125, 202, 48, 110, 133, 55, 110, 137, 206, 48, 110, 125, 210, 47, 110, 125, 214, 45, 110, 125, 1, 186, 43, 110, 141, 194, 53, 110, 129, 202, 51, 110, 137, 55, 58, 121, 124, 214, 60, 121, 123, 216, 58, 121, 123, 1, 186, 57, 121, 124, 192, 60, 121, 123, 194, 59, 121, 124, 198, 47, 112, 124, 202, 50, 112, 129, 52, 112, 129, 235, 214, 45, 112, 124, 49, 112, 124, 1, 186, 43, 112, 129, 52, 112, 129, 235]', 20, 25, 0.3189102564102563, 0.0], [93, '[202, 58, 117, 141, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 47, 117, 128, 194, 56, 117, 133, 206, 61, 117, 125, 210, 60, 117, 125, 214, 58, 117, 125, 1, 186, 48, 117, 128, 56, 117, 125, 190, 60, 117, 125, 194, 48, 117, 128, 52, 117, 128, 194, 52, 117, 128, 61, 117, 128, 202, 53, 117, 128, 60, 117, 128, 210, 55, 117, 133, 58, 117, 128, 1, 186, 56, 117, 135, 190, 55, 117, 125, 194, 53, 117, 125, 198, 55, 117, 125, 202, 53, 117, 125, 206, 51, 117, 125, 210, 49, 117, 125, 214, 48, 117, 125, 1, 186, 49, 117, 152, 1, 186, 51, 117, 136, 239, 202, 56, 117, 137, 1, 186, 55, 117, 136, 202, 60, 117, 138, 1, 186, 58, 117, 134, 202, 63, 117, 153, 1, 202, 61, 117, 153, 1, 202, 60, 117, 153, 1, 202, 58, 117, 130, 210, 56, 117, 130, 1, 186, 58, 117, 137, 202, 53, 117, 130, 210, 55, 117, 130, 1, 186, 56, 117, 136, 202, 51, 114, 123, 59, 114, 125, 204, 50, 114, 123, 206, 36, 114, 125, 52, 114, 123, 208, 50, 114, 123, 210, 36, 114, 125, 53, 114, 123, 212, 50, 114, 123, 214, 36, 114, 125, 53, 114, 123, 216, 47, 114, 123, 1, 186, 52, 114, 129, 188, 45, 114, 123, 190, 43, 114, 123, 192, 41, 114, 123, 194, 40, 114, 125, 198, 45, 114, 123, 200]', 19, 27, 0.5057471264367815, 0.5675675675675675], [94, '[202, 50, 121, 124, 59, 121, 124, 235, 202, 50, 121, 124, 60, 121, 124, 235, 210, 50, 121, 124, 59, 121, 124, 235, 190, 54, 117, 131, 194, 44, 117, 127, 53, 117, 127, 202, 43, 117, 127, 49, 117, 127, 210, 44, 117, 127, 48, 117, 127, 238, 210, 49, 117, 127, 212, 49, 117, 123, 214, 48, 117, 123, 216, 46, 117, 123, 1, 186, 45, 117, 123, 188, 48, 117, 123, 190, 45, 117, 123, 192, 48, 117, 123, 194, 45, 117, 123, 196, 48, 117, 123, 198, 47, 117, 123, 200, 48, 117, 123, 202, 45, 117, 123, 204, 50, 117, 123, 206, 43, 117, 123, 208, 47, 117, 123, 210, 48, 117, 123, 212, 50, 117, 123, 214, 52, 117, 123, 216, 54, 117, 123, 1, 186, 55, 117, 125, 192, 53, 117, 123, 194, 62, 117, 131, 204, 55, 117, 123, 198, 52, 117, 123, 200, 55, 117, 123, 202, 64, 117, 131, 212, 65, 117, 123, 214, 64, 117, 123, 216, 62, 117, 123, 1, 186, 58, 117, 123, 188, 62, 117, 123, 190, 60, 117, 123, 192, 62, 117, 123, 194, 63, 117, 123, 196, 62, 117, 123, 198, 60, 117, 123, 200, 62, 117, 123, 202, 58, 117, 123, 204, 60, 117, 123, 206, 59, 117, 123, 208, 60, 117, 123, 210, 64, 117, 123, 212, 60, 117, 123, 214, 59, 117, 123, 200, 60, 117, 123, 202, 62, 117, 124, 206, 57, 117, 123, 208, 55, 117, 123, 210]', 20, 22, 0.1802884615384615, 0.0], [95, '[202, 48, 117, 141, 206, 55, 117, 125, 210, 56, 117, 125, 214, 58, 117, 125, 1, 186, 61, 117, 125, 190, 51, 117, 125, 194, 54, 117, 125, 60, 117, 129, 198, 49, 117, 125, 202, 46, 117, 125, 206, 44, 117, 125, 210, 43, 117, 125, 214, 41, 117, 125, 1, 190, 47, 117, 125, 194, 48, 117, 125, 198, 49, 117, 125, 202, 48, 117, 125, 206, 46, 117, 125, 210, 44, 117, 125, 214, 43, 117, 125, 1, 186, 41, 117, 125, 190, 50, 117, 125, 194, 51, 117, 125, 198, 53, 117, 125, 202, 51, 117, 125, 206, 53, 117, 125, 210, 58, 117, 125, 214, 56, 117, 125, 1, 186, 55, 117, 141, 190, 46, 117, 125, 194, 51, 117, 125, 198, 49, 117, 125, 202, 48, 117, 141, 206, 58, 117, 125, 210, 56, 117, 125, 214, 55, 117, 125, 1, 186, 56, 117, 132, 190, 49, 117, 125, 194, 53, 117, 125, 198, 52, 117, 125, 202, 50, 117, 135, 206, 59, 117, 125, 210, 52, 117, 128, 1, 186, 53, 117, 125, 190, 55, 117, 125, 194, 53, 117, 135, 198, 60, 117, 123, 200, 58, 117, 123, 202, 56, 117, 127, 60, 117, 127, 210, 58, 117, 127, 61, 117, 127, 1, 186, 58, 117, 127, 60, 117, 127, 194, 58, 117, 127, 64, 117, 127, 202, 52, 117, 127, 61, 117, 140, 210, 54, 117, 127, 1, 186, 55, 117, 140, 235, 210, 53, 117, 127, 1, 186, 51]', 19, 23, 0.3815789473684211, 0.7222222222222222], [96, '[196, 48, 117, 123, 192, 49, 117, 123, 194, 46, 117, 123, 58, 117, 127, 196, 51, 117, 123, 198, 49, 117, 123, 200, 51, 117, 123, 202, 48, 117, 123, 206, 47, 117, 123, 55, 117, 123, 208, 48, 117, 123, 56, 117, 123, 210, 44, 117, 123, 60, 117, 123, 212, 48, 117, 123, 56, 117, 123, 214, 44, 117, 123, 60, 117, 123, 216, 46, 117, 123, 61, 117, 123, 1, 186, 58, 117, 124, 61, 117, 124, 190, 53, 117, 124, 61, 117, 124, 194, 53, 117, 123, 60, 117, 123, 196, 54, 117, 123, 63, 117, 123, 198, 53, 117, 123, 61, 117, 123, 200, 53, 117, 123, 58, 117, 123, 202, 49, 117, 127, 58, 117, 123, 204, 60, 117, 123, 206, 58, 117, 123, 208, 60, 117, 123, 210, 56, 117, 127, 212, 50, 117, 123, 214, 48, 117, 123, 216, 50, 117, 123, 1, 186, 48, 117, 127, 243, 194, 50, 117, 127, 202, 55, 117, 127, 210, 53, 117, 127, 1, 186, 51, 117, 127, 194, 55, 117, 127, 202, 56, 117, 134, 237, 194, 50, 117, 127, 202, 48, 117, 127, 210, 57, 118, 123, 216, 58, 118, 123, 1, 186, 58, 118, 123, 188, 56, 118, 123, 190, 55, 118, 123, 192, 53, 118, 123, 194, 54, 118, 123, 196, 53, 118, 123, 198, 49, 118, 123, 200, 53, 118, 123, 202, 46, 118, 125, 58, 118, 139, 206, 48, 118, 125, 55, 117, 125, 214, 56, 117, 125]', 16, 19, 0.2214052287581699, 0.4827586206896552], [97, '[202, 55, 117, 125, 206, 56, 117, 125, 210, 55, 117, 125, 214, 53, 117, 125, 1, 186, 48, 117, 131, 53, 117, 135, 188, 58, 117, 123, 190, 56, 117, 123, 192, 58, 117, 123, 194, 44, 117, 127, 56, 117, 123, 196, 58, 117, 123, 198, 56, 117, 123, 200, 58, 117, 123, 202, 48, 117, 127, 53, 117, 123, 204, 56, 117, 123, 206, 54, 117, 123, 208, 53, 117, 123, 210, 52, 117, 123, 212, 49, 117, 123, 53, 117, 123, 214, 48, 117, 123, 55, 117, 127, 216, 43, 117, 123, 46, 117, 123, 1, 186, 42, 117, 129, 47, 117, 129, 237, 210, 45, 117, 122, 211, 47, 117, 122, 53, 117, 122, 213, 52, 117, 122, 198, 52, 118, 123, 1, 186, 50, 118, 129, 53, 118, 147, 198, 45, 118, 125, 202, 50, 118, 125, 206, 48, 118, 125, 210, 47, 118, 129, 212, 52, 118, 123, 214, 55, 117, 123, 198, 45, 117, 123, 48, 117, 123, 200, 43, 117, 123, 51, 117, 123, 202, 43, 117, 132, 50, 117, 123, 204, 53, 117, 123, 206, 48, 117, 124, 55, 117, 123, 208, 56, 117, 123, 210, 44, 117, 123, 53, 117, 123, 212, 43, 117, 123, 55, 117, 123, 214, 44, 117, 123, 53, 117, 123, 216, 43, 117, 123, 52, 117, 123, 1, 186, 41, 117, 123, 48, 117, 123, 188, 47, 117, 123, 190, 65, 117, 124, 194, 48, 117, 124, 60, 117, 124, 198, 48, 117, 124]', 19, 24, 0.2474489795918367, 0.0], [98, '[194, 52, 116, 128, 202, 55, 116, 160, 1, 202, 53, 116, 125, 190, 55, 121, 124, 198, 57, 121, 124, 202, 62, 121, 127, 210, 50, 121, 135, 53, 121, 132, 57, 121, 127, 1, 186, 58, 121, 154, 190, 57, 121, 124, 194, 62, 121, 124, 198, 61, 121, 124, 202, 62, 121, 127, 210, 50, 121, 135, 53, 121, 127, 1, 186, 52, 121, 134, 194, 48, 121, 127, 202, 50, 121, 127, 210, 52, 121, 127, 210, 50, 121, 127, 1, 186, 49, 121, 147, 194, 48, 121, 127, 202, 50, 121, 127, 210, 52, 121, 127, 1, 186, 53, 121, 127, 194, 55, 121, 124, 198, 57, 121, 124, 202, 58, 121, 127, 210, 57, 121, 124, 214, 55, 121, 124, 1, 186, 57, 121, 124, 190, 55, 121, 124, 194, 58, 121, 124, 198, 57, 121, 124, 202, 55, 121, 150, 58, 121, 154, 210, 50, 121, 127, 1, 186, 52, 121, 134, 206, 58, 121, 124, 210, 57, 121, 124, 214, 55, 121, 124, 1, 186, 57, 121, 124, 190, 55, 121, 124, 194, 54, 121, 124, 198, 57, 121, 124, 202, 62, 121, 127, 210, 48, 121, 123, 212, 47, 121, 123, 214, 48, 121, 123, 216, 47, 121, 123, 1, 186, 45, 121, 123, 188, 43, 121, 123, 190, 46, 121, 123, 192, 45, 121, 123, 194, 43, 121, 124, 198, 55, 121, 124, 202, 57, 121, 124, 206, 62, 121, 124, 210, 63, 121, 124, 214, 62, 121, 124, 1]', 16, 20, 0.4081920903954802, 0.0], [99, '[202, 59, 110, 123, 196, 62, 110, 123, 198, 64, 110, 123, 200, 65, 110, 123, 236, 210, 48, 117, 131, 1, 190, 46, 117, 124, 194, 44, 117, 123, 196, 48, 117, 123, 198, 46, 117, 123, 200, 44, 117, 123, 202, 43, 117, 131, 212, 43, 117, 123, 214, 41, 117, 123, 216, 43, 117, 126, 1, 186, 40, 117, 123, 188, 43, 117, 123, 190, 41, 117, 123, 192, 41, 117, 123, 194, 39, 117, 123, 196, 42, 117, 123, 198, 39, 117, 123, 200, 42, 117, 123, 202, 39, 117, 123, 47, 117, 127, 204, 44, 117, 123, 206, 43, 117, 123, 208, 41, 117, 123, 210, 48, 117, 126, 216, 48, 117, 123, 1, 186, 56, 117, 127, 188, 52, 117, 123, 190, 53, 117, 123, 192, 47, 117, 123, 194, 48, 117, 124, 55, 117, 124, 198, 227, 48, 117, 123, 55, 117, 124, 192, 51, 117, 123, 194, 50, 117, 123, 196, 53, 117, 123, 198, 51, 117, 123, 200, 50, 117, 123, 202, 47, 117, 127, 204, 50, 117, 123, 206, 48, 117, 123, 208, 47, 117, 123, 210, 48, 117, 123, 212, 50, 117, 123, 214, 52, 117, 123, 216, 54, 117, 123, 1, 186, 55, 117, 123, 188, 53, 117, 123, 190, 52, 117, 123, 192, 50, 117, 123, 194, 48, 115, 123, 196, 50, 117, 123, 62, 117, 123, 212, 52, 117, 123, 55, 117, 123, 214, 50, 117, 123, 54, 117, 123, 216, 47, 117, 123, 1, 186]', 20, 26, 0.1651515151515151, 0.5853658536585366]], columns=['Unnamed: 0', 'midiTokens', 'pitchCount', 'pitchRange', 'averageNoteDuration', 'stepwiseMotion'])

#============================================

# Make the sliders
reggae_pitchCountSlider = 7 #@param {type:"slider", min:1, max:27, step:1.0}
reggae_pitchRangeSlider = 23 #@param {type:"slider", min:0, max:53, step:1.0}
reggae_averageNoteDurationSlider = 0.6 #@param {type:"slider", min:0, max:3.5, step:0.1}
reggae_stepwiseMotionSlider = 0.4 #@param {type:"slider", min:0, max:0.6, step:0.1}

bach_pitchCountSlider = 14 #@param {type:"slider", min:12, max:25, step:1.0}
bach_pitchRangeSlider = 32 #@param {type:"slider", min:14, max:34, step:1.0}
bach_averageNoteDurationSlider = 0.2 #@param {type:"slider", min:0, max:5.5, step:0.1}
bach_stepwiseMotionSlider = 0.6 #@param {type:"slider", min:0, max:1.0, step:0.1}

#============================================
# Narrow down the music based on values closet to the users metrics 
reggaeNarrowIdx = reggaeDf['pitchCount'].sub(reggae_pitchCountSlider).abs().nsmallest(30).index.values.tolist()
reggaeNarrowDf = reggaeDf.loc[reggaeNarrowIdx] #<---start with main reggae df
reggaeNarrowIdx = reggaeNarrowDf['pitchRange'].sub(reggae_pitchRangeSlider).abs().nsmallest(20).index.values.tolist()
reggaeNarrowDf = reggaeNarrowDf.loc[reggaeNarrowIdx] #<--- narrow it down
reggaeNarrowIdx = reggaeNarrowDf['averageNoteDuration'].sub(reggae_averageNoteDurationSlider).abs().nsmallest(10).index.values.tolist()
reggaeNarrowDf = reggaeNarrowDf.loc[reggaeNarrowIdx]
reggaeNarrowIdx = reggaeNarrowDf['stepwiseMotion'].sub(reggae_stepwiseMotionSlider).abs().nsmallest(1).index.values.tolist()
reggaeNarrowDf = reggaeNarrowDf.loc[reggaeNarrowIdx]

bachNarrowIdx = bachDf['pitchCount'].sub(bach_pitchCountSlider).abs().nsmallest(30).index.values.tolist()
bachNarrowDf = bachDf.loc[bachNarrowIdx] #<---start with main bach df
bachNarrowIdx = bachNarrowDf['pitchRange'].sub(bach_pitchRangeSlider).abs().nsmallest(20).index.values.tolist()
bachNarrowDf = bachNarrowDf.loc[bachNarrowIdx] #<--- narrow it down
bachNarrowIdx = bachNarrowDf['averageNoteDuration'].sub(bach_averageNoteDurationSlider).abs().nsmallest(10).index.values.tolist()
bachNarrowDf = bachNarrowDf.loc[bachNarrowIdx]
bachNarrowIdx = bachNarrowDf['stepwiseMotion'].sub(bach_stepwiseMotionSlider).abs().nsmallest(1).index.values.tolist()
bachNarrowDf = bachNarrowDf.loc[bachNarrowIdx]

#============================================
# Load the music from the df to tokens 
from ast import literal_eval #<--- for getting tokens from csv
programs = [(0, False), (41, False), (61, False), (0, True)]
print(":::REGGAE MOTIF:::")
reggaeTokens = literal_eval(reggaeNarrowDf["midiTokens"].iloc[0])
convertedReggaePath = "My Drive/generatedReggae.mid"
covnvertedReggae = tokenizer.tokens_to_midi([reggaeTokens], programs)
covnvertedReggae.dump(convertedReggaePath)
awesomeNotesReggae = note_seq.midi_file_to_sequence_proto(convertedReggaePath)
note_seq.plot_sequence(awesomeNotesReggae)
note_seq.play_sequence(awesomeNotesReggae, synth=note_seq.synthesize)

print("\n:::BACH MOTIF:::")
bachTokens = literal_eval(bachNarrowDf["midiTokens"].iloc[0])
convertedBachPath = "My Drive/generatedbach.mid"
covnvertedbach = tokenizer.tokens_to_midi([bachTokens], programs)
covnvertedbach.dump(convertedBachPath)
awesomeNotesBach = note_seq.midi_file_to_sequence_proto(convertedBachPath)
note_seq.plot_sequence(awesomeNotesBach)
note_seq.play_sequence(awesomeNotesBach, synth=note_seq.synthesize)


#============================================
print("\n:::INTERPOLATION:::")
# Clean the note sequences 
b = awesomeNotesBach
b.time_signatures.pop()
b.instrument_infos.pop()

r = awesomeNotesReggae
r.time_signatures.pop()
r.instrument_infos.pop()

#============================================
# Process note sequences for music vae, extracting melodies 
reggae_mels = []
reggae_mels.extend(mel_2bar_config.data_converter.from_tensors(mel_2bar_config.data_converter.to_tensors(r)[1]))
bach_mels = []
bach_mels.extend(mel_2bar_config.data_converter.from_tensors(mel_2bar_config.data_converter.to_tensors(b)[1]))

#============================================
# Generate and show sequences based on a random melody from above
num_steps = 16;
note_sequences = music_vae.interpolate(
      reggae_mels[random.randint(0, len(reggae_mels) -1)], 
      bach_mels[random.randint(0, len(bach_mels)-1)], 
      num_steps,
      length=32
)
interp_seq = note_seq.sequences_lib.concatenate_sequences(note_sequences)

# Show 
note_seq.plot_sequence(interp_seq)
note_seq.play_sequence(interp_seq, synth=note_seq.synthesize)

:::REGGAE MOTIF:::



:::BACH MOTIF:::



:::INTERPOLATION:::


## Appendix

The initial plan for the project was to provide 2 VAE models for both the bach and reggae data, and allow users to explore their latent vectors to produce the 2 distinct extracts. Below is the code attempting this. I ran into an error where the calculations (possibly for the KL divergence loss, or the Z vectors) didn't work with the version of torch which I was running, and hope to return to this one day. 

The pivot towards using pre-generated examples and musical metrics may have actually been a positive thing, however, as the slider values are much more intuitive than an empty latent vectory, supporting casual creator design principles. 

In [ ]:
# # define params 
# latent_units = 2
# lstm_units = 1#1024
# embedding_dim = 256

In [ ]:
# # build encoder=================================================================
# x = Input(shape=(my_batch.shape[1]))
# emb = Embedding(len(vocab), embedding_dim, input_length=my_batch.shape[0])(x)
# b = Bidirectional(LSTM(lstm_units, return_sequences=True))(emb)
# mu = Dense(units=latent_units, activation=None)(b) 
# mu = LayerNormalization(axis=1, name="mu_final")(mu)

# std = Dense(units=latent_units, activation="softplus", name="std_final")(b) 

# eps = tf.random.normal(tf.shape(std)) #<-- reparametarisation 
# z = mu + (std * eps) #<--- latent vector

# encoder =  Model(x, [mu, std, z], name="encoder")
# #===============================================================================


# # build decoder=================================================================
# x1 = Input(shape=(my_batch.shape[1], latent_units))
# x2 = Dense(units=my_batch.shape[1], activation="softmax")(x1) 
# decoder = Model(x1, y, name="decoder")
# #===============================================================================

# # print summaries 
# encoder.summary()
# decoder.summary()

In [ ]:
# #define custom loss function 
# class LossLayer(keras.layers.Layer):
#   def vae_loss(self, x, z_decoded):
#         #guess these need to be flattened
#         x = K.flatten(x)
#         z_decoded = K.flatten(z_decoded)

#         # Reconstruction loss (as we used sigmoid activation we can use binarycrossentropy)
#         recon_loss = tf.keras.losses.binary_crossentropy(x, z_decoded)
        
#         # KL divergence
#         kl_loss = -5e-4 * K.mean(1 + std - K.square(mu) - K.exp(std), axis=-1)
#         return K.mean(recon_loss + kl_loss)

#   def call(self, inputs):
#     x = inputs[0]
#     z_decoded = inputs[1]
#     loss = self.vae_loss(x, z_decoded)
#     self.add_loss(loss, inputs=inputs)
#     return x


# # build the loss layer into the model
# z_decoded = decoder(z)
# y1 = LossLayer()([x, z_decoded])

# # Compile the VAE
# vae = Model(x, y1, name='vae')
# vae.compile(optimizer='adam', loss=None)
# vae.summary()

In [ ]:
# n_epochs =  50

# vae.fit(my_batch, None)